In [ ]:
import os
# replace PATH_TO_DOWNLOADED_CODE_DIR with the path of our code that you downloaded. This is the directory
# of the README file, among others.
os.chdir(PATH_TO_DOWNLOADED_CODE_DIR) 

import matplotlib
%matplotlib inline
# %matplotlib widget

# import matplotlib
# matplotlib.use('TkAgg')
# matplotlib.use('TKAgg')
# matplotlib.use('Agg')
# %matplotlib tk

# %matplotlib qt
# %pylab qt
# %matplotlib gtk
# %matplotlib wx


# IIUC, according to https://stackoverflow.com/questions/51922480/javascript-error-ipython-is-not-defined-in-jupyterlab/56416229#56416229, the way
# to do this is to use "%matplotlib widget"
# import matplotlib
# %matplotlib notebook 

import matplotlib.pyplot as plt
import importlib

# dpi = None
dpi = 100
dpi = 300 # the figures look a lot better (but load a little slower)
if dpi:
    matplotlib.rcParams['figure.dpi'] = dpi
    matplotlib.rc("savefig", dpi=dpi)
    
if 0:
    plt.close('all')

In [ ]:
import pickle
import collections
import subprocess
import importlib
import math
import operator
import pathlib
import Bio
import io
import itertools
import matplotlib.colors
import matplotlib.patches
import matplotlib.ticker
import matplotlib.image
import matplotlib.transforms
import string
from Bio import SeqRecord

from generic import generic_utils
importlib.reload(generic_utils)
from generic import bio_utils
importlib.reload(bio_utils)
from generic import blast_interface_and_utils
importlib.reload(blast_interface_and_utils)
from generic import vsearch_interface_and_utils
importlib.reload(vsearch_interface_and_utils)
from generic import mauve_interface_and_utils
importlib.reload(mauve_interface_and_utils)
from searching_for_pis import massive_screening_configuration
importlib.reload(massive_screening_configuration)
from searching_for_pis import massive_screening_stage_1
importlib.reload(massive_screening_stage_1)
from searching_for_pis import index_column_names
importlib.reload(index_column_names)
# from generic import samtools_and_sam_files_interface
# importlib.reload(samtools_and_sam_files_interface)


import scipy.stats
import scipy.integrate
import scipy.interpolate
import statsmodels.api as sm
import statsmodels.stats
import random
import pandas as pd
import numpy as np
# 200624: It seems that this is the default (what I get from np.geterr() after importing numpy): {'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}
# https://numpy.org/doc/stable/reference/generated/numpy.seterr.html says: Underflow: result so close to zero that some precision was lost.
# so I guess it is Ok that I just ignore underflow problems?
# np.seterr(all='raise')
np.seterr(divide='raise', over='raise', invalid='raise')
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('display.expand_frame_repr', False)


search_for_pis_args = massive_screening_configuration.SEARCH_FOR_PROGRAMMED_INVERSIONS_ARGS_DICT

debug___num_of_taxa_to_go_over = search_for_pis_args['debug___num_of_taxa_to_go_over']
debug___taxon_uids = search_for_pis_args['debug___taxon_uids']

stage_out_file_name_suffix = massive_screening_stage_1.get_stage_out_file_name_suffix(
    debug___num_of_taxa_to_go_over=debug___num_of_taxa_to_go_over,
    debug___taxon_uids=debug___taxon_uids,
)

paper_figs_and_tables_dir_path = 'paper_figs_and_tables'
pathlib.Path(paper_figs_and_tables_dir_path).mkdir(parents=True, exist_ok=True)

In [ ]:
stage1_out_dir_path = search_for_pis_args['stage1']['output_dir_path']
stage1_results_info_pickle_file_path = os.path.join(stage1_out_dir_path, search_for_pis_args['stage1']['results_pickle_file_name'])
stage1_results_info_pickle_file_path = generic_utils.add_suffix_to_file_name_while_keeping_extension(
    stage1_results_info_pickle_file_path, stage_out_file_name_suffix)

with open(stage1_results_info_pickle_file_path, 'rb') as f:
    stage1_results_info = pickle.load(f)

taxa_for_which_download_of_primary_assembly_nuccore_entries_failed_uids = stage1_results_info[
    'taxa_for_which_download_of_primary_assembly_nuccore_entries_failed_uids']
print(f'len(taxa_for_which_download_of_primary_assembly_nuccore_entries_failed_uids): '
      f'{len(taxa_for_which_download_of_primary_assembly_nuccore_entries_failed_uids)}')
    

num_of_all_cds = stage1_results_info['num_of_all_cds']
num_of_taxa = stage1_results_info['num_of_taxa']

taxa_df_csv_file_path = stage1_results_info['taxa_df_csv_file_path']
taxa_df = pd.read_csv(taxa_df_csv_file_path, sep='\t', low_memory=False)
print(f'len(taxa_df): {len(taxa_df)}')




In [ ]:
stage2_out_dir_path = search_for_pis_args['stage2']['output_dir_path']
stage2_results_info_pickle_file_path = os.path.join(stage2_out_dir_path, search_for_pis_args['stage2']['results_pickle_file_name'])
stage2_results_info_pickle_file_path = generic_utils.add_suffix_to_file_name_while_keeping_extension(
    stage2_results_info_pickle_file_path, stage_out_file_name_suffix)

with open(stage2_results_info_pickle_file_path, 'rb') as f:
    stage2_results_info = pickle.load(f)

num_of_all_ir_pairs = stage2_results_info['num_of_all_ir_pairs']

In [ ]:
stage3_out_dir_path = search_for_pis_args['stage3']['output_dir_path']
stage3_results_info_pickle_file_path = os.path.join(stage3_out_dir_path, search_for_pis_args['stage3']['results_pickle_file_name'])
stage3_results_info_pickle_file_path = generic_utils.add_suffix_to_file_name_while_keeping_extension(
    stage3_results_info_pickle_file_path, stage_out_file_name_suffix)

with open(stage3_results_info_pickle_file_path, 'rb') as f:
    stage3_results_info = pickle.load(f)

ir_pairs_linked_df_csv_file_path = stage3_results_info['ir_pairs_linked_df_csv_file_path']
ir_pairs_linked_df = pd.read_csv(ir_pairs_linked_df_csv_file_path, sep='\t', low_memory=False)

# pairs_df = pairs_df.sample(frac=1, random_state=0)
discard_ir_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_info = stage3_results_info[
    'discard_ir_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_info']


pairs_linked_with_evidence_for_repeat_overlapping_mobile_element_df_csv_file_path = stage3_results_info[
    'pairs_linked_with_evidence_for_repeat_overlapping_mobile_element_df_csv_file_path']
pairs_linked_with_evidence_for_repeat_overlapping_mobile_element_df = pd.read_csv(
    pairs_linked_with_evidence_for_repeat_overlapping_mobile_element_df_csv_file_path, sep='\t', low_memory=False)
print(f'len(pairs_linked_with_evidence_for_repeat_overlapping_mobile_element_df): '
      f'{len(pairs_linked_with_evidence_for_repeat_overlapping_mobile_element_df)}')

pairs_after_discarding_ir_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_df_csv_file_path = stage3_results_info[
    'pairs_after_discarding_ir_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_df_csv_file_path']
pairs_after_discarding_ir_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_df = pd.read_csv(
    pairs_after_discarding_ir_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_df_csv_file_path, sep='\t', low_memory=False)



In [ ]:
stage4_out_dir_path = search_for_pis_args['stage4']['output_dir_path']
stage4_results_info_pickle_file_path = os.path.join(stage4_out_dir_path, search_for_pis_args['stage4']['results_pickle_file_name'])
stage4_results_info_pickle_file_path = generic_utils.add_suffix_to_file_name_while_keeping_extension(
    stage4_results_info_pickle_file_path, stage_out_file_name_suffix)

with open(stage4_results_info_pickle_file_path, 'rb') as f:
    stage4_results_info = pickle.load(f)
    
taxa_potential_evidence_for_pi_info_pickle_file_path = stage4_results_info['taxa_potential_evidence_for_pi_info_pickle_file_path']
with open(taxa_potential_evidence_for_pi_info_pickle_file_path, 'rb') as f:
    taxa_potential_evidence_for_pi_info = pickle.load(f)
taxon_uid_to_taxon_potential_evidence_for_pi_info = taxa_potential_evidence_for_pi_info['taxon_uid_to_taxon_potential_evidence_for_pi_info']

num_of_taxa_for_which_search_for_wgs_nuccore_entries_failed = taxa_potential_evidence_for_pi_info[
    'num_of_taxa_for_which_search_for_wgs_nuccore_entries_failed']
print(f'num_of_taxa_for_which_search_for_wgs_nuccore_entries_failed: {num_of_taxa_for_which_search_for_wgs_nuccore_entries_failed}')

num_of_taxa_for_which_search_for_local_blast_nuccore_entries_failed = taxa_potential_evidence_for_pi_info[
    'num_of_taxa_for_which_search_for_local_blast_nuccore_entries_failed']
print(f'num_of_taxa_for_which_search_for_local_blast_nuccore_entries_failed: {num_of_taxa_for_which_search_for_local_blast_nuccore_entries_failed}')

In [ ]:
stage5_out_dir_path = search_for_pis_args['stage5']['output_dir_path']
stage5_results_info_pickle_file_path = os.path.join(stage5_out_dir_path, search_for_pis_args['stage5']['results_pickle_file_name'])
stage5_results_info_pickle_file_path = generic_utils.add_suffix_to_file_name_while_keeping_extension(
    stage5_results_info_pickle_file_path, stage_out_file_name_suffix)

with open(stage5_results_info_pickle_file_path, 'rb') as f:
    stage5_results_info = pickle.load(f)

taxon_uid_to_nuccore_accession_to_other_nuccore_entries_evidence_for_pis_in_nuccore_entry_info_pickle_file_path = stage5_results_info[
    'taxon_uid_to_nuccore_accession_to_other_nuccore_entries_evidence_for_pis_in_nuccore_entry_info_pickle_file_path']
with open(taxon_uid_to_nuccore_accession_to_other_nuccore_entries_evidence_for_pis_in_nuccore_entry_info_pickle_file_path, 'rb') as f:
    taxon_uid_to_nuccore_accession_to_other_nuccore_entries_evidence_for_pis_in_nuccore_entry_info = pickle.load(f)

extended_merged_cds_pair_region_df_csv_file_path = stage5_results_info['extended_merged_cds_pair_region_df_csv_file_path']
extended_merged_cds_pair_region_df = pd.read_csv(extended_merged_cds_pair_region_df_csv_file_path, sep='\t', low_memory=False)

region_in_other_nuccore_df_csv_file_path = stage5_results_info['region_in_other_nuccore_df_csv_file_path']
region_in_other_nuccore_df = pd.read_csv(region_in_other_nuccore_df_csv_file_path, sep='\t', low_memory=False)

potential_breakpoint_df_csv_file_path = stage5_results_info['potential_breakpoint_df_csv_file_path']
potential_breakpoint_df = pd.read_csv(potential_breakpoint_df_csv_file_path, sep='\t', low_memory=False)

num_of_pairs_with_high_confidence_bps = stage5_results_info['num_of_pairs_with_high_confidence_bps']

minimal_ir_pairs_linked_to_breakpoint_pairs_df_csv_file_path = stage5_results_info['minimal_ir_pairs_linked_to_breakpoint_pairs_df_csv_file_path']
minimal_ir_pairs_linked_to_breakpoint_pairs_df = pd.read_csv(minimal_ir_pairs_linked_to_breakpoint_pairs_df_csv_file_path, sep='\t', low_memory=False)

filtered_minimal_ir_pairs_linked_to_breakpoint_pairs_df_csv_file_path = stage5_results_info['filtered_minimal_ir_pairs_linked_to_breakpoint_pairs_df_csv_file_path']
filtered_minimal_ir_pairs_linked_to_breakpoint_pairs_df = pd.read_csv(filtered_minimal_ir_pairs_linked_to_breakpoint_pairs_df_csv_file_path, sep='\t', low_memory=False)

pairs_with_highest_confidence_bps_df_csv_file_path = stage5_results_info['pairs_with_highest_confidence_bps_df_csv_file_path']
pairs_with_highest_confidence_bps_df = pd.read_csv(pairs_with_highest_confidence_bps_df_csv_file_path, sep='\t', low_memory=False)

lens_of_spanning_regions_ratios_df_csv_file_path = stage5_results_info['lens_of_spanning_regions_ratios_df_csv_file_path']
lens_of_spanning_regions_ratios_df = pd.read_csv(lens_of_spanning_regions_ratios_df_csv_file_path, sep='\t', low_memory=False)

filtered_pairs_with_highest_confidence_bps_df_csv_file_path = stage5_results_info['filtered_pairs_with_highest_confidence_bps_df_csv_file_path']
filtered_pairs_with_highest_confidence_bps_df = pd.read_csv(filtered_pairs_with_highest_confidence_bps_df_csv_file_path, sep='\t', low_memory=False)

taxon_uid_to_downloaded_taxon_wgs_nuccore_entries_info_pickle_file_path_pickle_file_path = stage5_results_info['taxon_uid_to_downloaded_taxon_wgs_nuccore_entries_info_pickle_file_path_pickle_file_path']
with open(taxon_uid_to_downloaded_taxon_wgs_nuccore_entries_info_pickle_file_path_pickle_file_path, 'rb') as f:
    taxon_uid_to_downloaded_taxon_wgs_nuccore_entries_info_pickle_file_path = pickle.load(f)

products_with_high_confidence_bps_value_counts = pd.concat([
    pairs_with_highest_confidence_bps_df[pairs_with_highest_confidence_bps_df['high_confidence_bp_for_both_repeats']][f'repeat{repeat_num}_cds_product']
    for repeat_num in (1, 2)
], ignore_index=True).value_counts()
print(f'\nproducts_with_high_confidence_bps_value_counts:\n{products_with_high_confidence_bps_value_counts}\n')

curr_df = pairs_with_highest_confidence_bps_df.merge(extended_merged_cds_pair_region_df)
stage5_num_of_cds_pairs_with_any_homologous_locus = len(
    curr_df[curr_df['num_of_regions_in_other_nuccores_satisfying_mauve_total_match_proportion_and_min_sub_alignment_min_match_proportion_thresholds'] >= 1][
        index_column_names.CDS_PAIR_INDEX_COLUMN_NAMES].drop_duplicates())





In [ ]:
enrichment_analysis_out_dir_path = search_for_pis_args['enrichment_analysis']['output_dir_path']
enrichment_analysis_results_info_pickle_file_path = os.path.join(enrichment_analysis_out_dir_path, search_for_pis_args['enrichment_analysis']['results_pickle_file_name'])
enrichment_analysis_results_info_pickle_file_path = generic_utils.add_suffix_to_file_name_while_keeping_extension(
    enrichment_analysis_results_info_pickle_file_path, stage_out_file_name_suffix)

with open(enrichment_analysis_results_info_pickle_file_path, 'rb') as f:
    enrichment_analysis_results_info = pickle.load(f)


extended1_cds_pairs_for_enrichment_analysis_and_logistic_regression_df_csv_file_path = enrichment_analysis_results_info[
    'extended1_cds_pairs_for_enrichment_analysis_and_logistic_regression_df_csv_file_path']
extended1_cds_pairs_for_enrichment_analysis_and_logistic_regression_df = pd.read_csv(
    extended1_cds_pairs_for_enrichment_analysis_and_logistic_regression_df_csv_file_path, sep='\t', low_memory=False)

column_name_to_critical_val_and_passed_threshold_column_name_pickle_file_path = enrichment_analysis_results_info[
    'column_name_to_critical_val_and_passed_threshold_column_name_pickle_file_path']
with open(column_name_to_critical_val_and_passed_threshold_column_name_pickle_file_path, 'rb') as f:
    column_name_to_critical_val_and_passed_threshold_column_name = pickle.load(f)

if enrichment_analysis_results_info['logistic_regression_converged']:
    if not (
            search_for_pis_args['enrichment_analysis']['DEBUG___RANDOM_ANY_HIGH_CONFIDENCE_IR_PAIR_LINKED_TO_CDS_PAIR'] or 
            search_for_pis_args['enrichment_analysis']['DEBUG___SHUFFLE_OPERON_ASYMMETRY']
    ):
        repeat_cds_product_fisher_or_g_result_df_csv_file_path = enrichment_analysis_results_info['repeat_cds_product_fisher_or_g_result_df_csv_file_path']
        repeat_cds_product_fisher_or_g_result_df = pd.read_csv(repeat_cds_product_fisher_or_g_result_df_csv_file_path, sep='\t', low_memory=False)

        adjacent_repeat_cds_product_fisher_or_g_result_df_csv_file_path = enrichment_analysis_results_info['adjacent_repeat_cds_product_fisher_or_g_result_df_csv_file_path']
        adjacent_repeat_cds_product_fisher_or_g_result_df = pd.read_csv(adjacent_repeat_cds_product_fisher_or_g_result_df_csv_file_path, sep='\t', low_memory=False)

    logistic_regression_fit_result_df_csv_file_path = enrichment_analysis_results_info['logistic_regression_fit_result_df_csv_file_path']
    logistic_regression_fit_result_df = pd.read_csv(logistic_regression_fit_result_df_csv_file_path, sep='\t', low_memory=False)
    
    unified_roc_curve_df_csv_file_path = enrichment_analysis_results_info['unified_roc_curve_df_csv_file_path']
    unified_roc_curve_df = pd.read_csv(unified_roc_curve_df_csv_file_path, sep='\t', low_memory=False)


    if not (
            search_for_pis_args['enrichment_analysis']['DEBUG___RANDOM_ANY_HIGH_CONFIDENCE_IR_PAIR_LINKED_TO_CDS_PAIR'] or 
            search_for_pis_args['enrichment_analysis']['DEBUG___SHUFFLE_OPERON_ASYMMETRY']
    ):
        extended3_all_cds_pairs_df_csv_file_path = (
            enrichment_analysis_results_info['extended3_all_cds_pairs_df_csv_file_path'])
        extended3_all_cds_pairs_df = pd.read_csv(
            extended3_all_cds_pairs_df_csv_file_path, sep='\t', low_memory=False)


else:
    logistic_regression_fit_result_df = None
    unified_roc_curve_df = None

In [ ]:
stage6_out_dir_path = search_for_pis_args['stage6']['output_dir_path']
stage6_results_info_pickle_file_path = os.path.join(stage6_out_dir_path, search_for_pis_args['stage6']['results_pickle_file_name'])
stage6_results_info_pickle_file_path = generic_utils.add_suffix_to_file_name_while_keeping_extension(
    stage6_results_info_pickle_file_path, stage_out_file_name_suffix)

with open(stage6_results_info_pickle_file_path, 'rb') as f:
    stage6_results_info = pickle.load(f)

all_raw_read_alignment_results_df_csv_file_path = stage6_results_info['all_raw_read_alignment_results_df_csv_file_path']
all_raw_read_alignment_results_df = pd.read_csv(all_raw_read_alignment_results_df_csv_file_path, sep='\t', low_memory=False)

putative_pi_locus_descriptions_df_csv_file_path = stage6_results_info['putative_pi_locus_descriptions_df_csv_file_path']
putative_pi_locus_descriptions_df = pd.read_csv(putative_pi_locus_descriptions_df_csv_file_path, sep='\t', low_memory=False)

all_all_diff_score_alignments_df_csv_file_path = stage6_results_info['all_all_diff_score_alignments_df_csv_file_path']
all_all_diff_score_alignments_df = pd.read_csv(all_all_diff_score_alignments_df_csv_file_path, sep='\t', low_memory=False)

rna_seq_summary_df_csv_file_path = stage6_results_info['rna_seq_summary_df_csv_file_path']
rna_seq_summary_df = pd.read_csv(rna_seq_summary_df_csv_file_path, sep='\t', low_memory=False)

good_rna_sra_accessions_and_predicted_invertible_regions_df = rna_seq_summary_df[
            (rna_seq_summary_df['num_of_reads_matching_ref_better'] > 0) &
            (rna_seq_summary_df['num_of_reads_matching_non_ref_better'] > 0)
][['sra_accession', 'in_silico_inverted_region']].drop_duplicates()

good_rna_sra_accessions_df = good_rna_sra_accessions_and_predicted_invertible_regions_df['sra_accession'].drop_duplicates()

semi_min_rna_seq_summary_df = rna_seq_summary_df.merge(good_rna_sra_accessions_df)
semi_min_all_all_diff_score_alignments_df = all_all_diff_score_alignments_df.merge(good_rna_sra_accessions_df)
min_all_all_diff_score_alignments_df = all_all_diff_score_alignments_df.merge(good_rna_sra_accessions_and_predicted_invertible_regions_df)




all_homologs_df_csv_file_path = stage6_results_info['all_homologs_df_csv_file_path']
if generic_utils.is_file_empty(all_homologs_df_csv_file_path):
    all_homologs_df = None
else:
    all_homologs_df = pd.read_csv(all_homologs_df_csv_file_path, sep='\t', low_memory=False)
    all_homologs_extended_df = all_homologs_df.merge(taxa_df)
    assert len(all_homologs_extended_df) == len(all_homologs_df)
    

    
all_all_repeat_cds_covered_bases_proportions_pickle_file_path = stage6_results_info['all_all_repeat_cds_covered_bases_proportions_pickle_file_path']
with open(all_all_repeat_cds_covered_bases_proportions_pickle_file_path, 'rb') as f:
    all_all_repeat_cds_covered_bases_proportions = pickle.load(f)
    
all_all_alignment_bases_covered_by_cds_proportions_pickle_file_path = stage6_results_info['all_all_alignment_bases_covered_by_cds_proportions_pickle_file_path']
with open(all_all_alignment_bases_covered_by_cds_proportions_pickle_file_path, 'rb') as f:
    all_all_alignment_bases_covered_by_cds_proportions = pickle.load(f)


In [ ]:
def write_xlsx_for_df(table_file_path, sheet_name, df, ordered_column_name_and_final_name_and_width, column_name_to_num_format_specifier={}, writer=None):
    use_existing_writer = writer is not None
    
    if use_existing_writer:
        assert table_file_path is None
    else:
        writer = pd.ExcelWriter(table_file_path, engine='xlsxwriter')
    
    df = df.copy()
    df = df[[x[0] for x in ordered_column_name_and_final_name_and_width]]
    column_name_to_final_column_name = {x[0]: x[1] for x in ordered_column_name_and_final_name_and_width}
    df.rename(columns=column_name_to_final_column_name, inplace=True)
    
    
    # https://stackoverflow.com/questions/55183760/text-wrap-format-gets-ignored-using-worksheet-formatting/55185388#55185388
    df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1, header=False)
    
    
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    shared_cell_format_attribute_dict = {'text_wrap': True, 'border': 1, 'border_color': 'black', 'align': 'left'}
    normal_cell_format = workbook.add_format(shared_cell_format_attribute_dict)
    header_cell_format = workbook.add_format({
        **shared_cell_format_attribute_dict, 
        'bold': True, 
        # 'valign': 'top',
    })
    
    final_column_name_to_num_format = {
        column_name_to_final_column_name[column_name]: workbook.add_format(
            {**shared_cell_format_attribute_dict, 'num_format': num_format_specifier})
        for column_name, num_format_specifier in column_name_to_num_format_specifier.items()
    }
    
    
    num_of_columns = len(list(df))
    for i in range(num_of_columns):
        final_column_name = ordered_column_name_and_final_name_and_width[i][1]
        column_cell_format = final_column_name_to_num_format.get(final_column_name, normal_cell_format)
        worksheet.set_column(i, i, ordered_column_name_and_final_name_and_width[i][2], column_cell_format)

    # https://stackoverflow.com/questions/55183760/text-wrap-format-gets-ignored-using-worksheet-formatting/55185388#55185388
    for i, column_name in enumerate(list(df)):
        worksheet.write(0, i, column_name, header_cell_format)

    if not use_existing_writer:
        writer.save()
    


In [ ]:
matplotlib.rcParams["legend.frameon"] = False
matplotlib.rcParams['hatch.linewidth'] = 0.3
matplotlib.rcParams["savefig.facecolor"] = 'white'
matplotlib.rcParams["path.snap"] = False

# according to https://matplotlib.org/stable/gallery/color/color_cycle_default.html
prop_cycle = plt.rcParams['axes.prop_cycle']
TEN_DEFAULT_MATPLOTLIB_COLORS = prop_cycle.by_key()['color']
TEN_DEFAULT_MATPLOTLIB_COLORS_WITHOUT_GREY = TEN_DEFAULT_MATPLOTLIB_COLORS[:7] + TEN_DEFAULT_MATPLOTLIB_COLORS[8:]
TEN_DEFAULT_MATPLOTLIB_COLORS_WITHOUT_GREY_AND_PURPLE = TEN_DEFAULT_MATPLOTLIB_COLORS_WITHOUT_GREY[:4] + TEN_DEFAULT_MATPLOTLIB_COLORS_WITHOUT_GREY[5:]

COLOR_OF_HACKY_LINE_TO_OVERWRITE_FRACTION_LINE_OF_ORIENTATION_MATCHING = 'red'
COLOR_OF_HACKY_LINE_TO_OVERWRITE_FRACTION_LINE_OF_ORIENTATION_MATCHING = 'white'
ALIGNMENT_PROJECTION_COLOR = 'orange'
ALIGNMENT_PROJECTION_ALPHA = 0.3
ALIGNMENT_PROJECTION_EDGE_COLOR = (*matplotlib.colors.to_rgb('black'), 0.5)
ALIGNMENT_PROJECTION_LINE_WIDTH = 0.5
VOLCANO_PLOT_EDGE_COLOR = '#222222'
VOLCANO_PLOT_LINEWIDTH = 0.5
GREY_HIST_COLOR = (0.7,0.7,0.7)
THRESHOLD_DASHED_LINE_ALPHA = 0.7
FAKE_VAL_THRESHOLD_LINE_ALPHA = 0.7
FAKE_VAL_THRESHOLD_LINE_COLOR = 'black'
FAKE_VAL_THRESHOLD_LINE_STYLE = ':'
# https://colorbrewer2.org
COLORBREWER_SET3 = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']
COLORBREWER_SET3_GREY = '#d9d9d9'
COLORBREWER_SET3_WITHOUT_GREY = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#bc80bd','#ccebc5','#ffed6f']
COLORBREWER_SET3_WITHOUT_GREY_AND_YELLOW = ['#8dd3c7' ,'#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#bc80bd','#ccebc5','#ffed6f']
COLORBREWER_SET3_WITHOUT_GREY_AND_YELLOW_AND_ORANGE = ['#8dd3c7' ,'#bebada','#fb8072','#80b1d3','#b3de69','#fccde5','#bc80bd','#ccebc5','#ffed6f']
COLORBREWER_SET1 = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf','#999999']
COLORBREWER_SET1_WITHOUT_GREY = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf']
COLORBREWER_SET1_WITHOUT_GREY_AND_YELLOW = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00' ,'#a65628','#f781bf']
COLORBREWER_SET1_WITHOUT_GREY_AND_YELLOW_AND_ORANGE = ['#e41a1c','#377eb8','#4daf4a','#984ea3' ,'#a65628','#f781bf']
COLORBREWER_PASTEL = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
COLORBREWER_PASTEL_WITHOUT_GREY = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec']
COLORBREWER_PASTEL_WITHOUT_GREY_AND_WITH_GREEN_AND_BROWN_SWITCHED = ['#fbb4ae','#b3cde3','#e5d8bd','#decbe4','#fed9a6','#ffffcc','#ccebc5','#fddaec']
COLORBREWER_PASTEL_WITH_GREEN_AND_BROWN_SWITCHED = ['#fbb4ae','#b3cde3','#e5d8bd','#decbe4','#fed9a6','#ffffcc','#ccebc5','#fddaec', '#f2f2f2']
PRODUCT_CLASS_COLORS = [
    '#fbb4ae','#b3cde3','#e5d8bd','#decbe4','#fed9a6','#ffffcc','#ccebc5',
    # '#fddaec', # pink, i think
    
    # 'paleturquoise',
    # tuple(np.array(matplotlib.colors.to_rgb('paleturquoise')) * 0.7),
    (*matplotlib.colors.to_rgb('paleturquoise'), 0.7),
    # tuple(np.array(matplotlib.colors.to_rgb('fuchsia')) * 0.3),
    # (*matplotlib.colors.to_rgb('fuchsia'), 0.15),
    (*matplotlib.colors.to_rgb('tab:brown'), 0.4),
    (*matplotlib.colors.to_rgb('darkcyan'), 0.45),
]

NON_REF_VARIANT_COLORS = COLORBREWER_SET1_WITHOUT_GREY_AND_YELLOW + [
    'gold',
    'grey',
    'teal',
    'olive',
    'olivedrab',
    'lawngreen',
    'cadetblue',
    'mediumseagreen',
    # 'mediumorchid',
    'mediumpurple',
    'deepskyblue',
    'indianred',
    'tan',
    'darkkhaki',
    'steelblue',
    'lightgreen',
    'powderblue',
    'plum',
    'mediumaquamarine',
    'navajowhite',
]

HISTOGRAM_WITH_OVERLAP_ALPHA = 0.5
HIGH_CONFIDENCE_BP_COLOR = 'black'
NO_HIGH_CONFIDENCE_BP_COLOR = '#aaaaaa'

OTHER_PRODUCT_CLASS_COLOR = 'white'

# alternatives - 'outer membrane'
# 
# https://www.annualreviews.org/doi/10.1146/annurev.micro.112408.134247 says:
# "TonB-dependent transporters (TBDTs) are bacterial outer membrane proteins that bind and transport ferric chelates, 
# called siderophores, as well as vitamin B12, nickel complexes, and carbohydrates. The transport process requires energy 
# in the form of proton motive force and a complex of three inner membrane proteins, TonB-ExbB-ExbD, to transduce this 
# energy to the outer membrane. The siderophore substrates range in complexity from simple small molecules such as citrate
# to large proteins such as serum transferrin and hemoglobin."
# 
# https://www.ebi.ac.uk/interpro/entry/InterPro/IPR023996/ says:
# This entry describes a clade of TonB-linked outer membrane proteins (OMP). Members of the family are restricted to the 
# Bacteriodetes lineage (except for Gemmatimonas aurantiaca T-27 from the novel phylum Gemmatimonadetes) and occur in high
# copy numbers, with over 100 members from Bacteroides thetaiotaomicron VPI-5482 alone. Published descriptions of members 
# of the family are available for RagA from Porphyromonas gingivalis [1], SusC from Bacteroides thetaiotaomicron [2], and 
# OmpW from Bacteroides caccae [3]. Members form pairs with members of the SusD/RagB family (PF07980). Transporter complexes 
# including these outer membrane proteins are likely to import large degradation products of proteins (e.g. RagA) or 
# carbohydrates (e.g. SusC) as nutrients, rather than siderophores.
TONB_DEPENDENT_TRANSPORTER_CLASS_NAME = 'TonB-linked OMP'

LIST_OF_PRODUCT_AND_PRODUCT_CLASS = [
    ('BREX-1 system adenine-specific DNA-methyltransferase PglX', 'DNA MTase'),
    ('Eco57I restriction-modification methylase domain-containing protein', 'DNA MTase'),
    ('N-6 DNA methylase', 'DNA MTase'),
    ('SAM-dependent DNA methyltransferase', 'DNA MTase'),
    ('class I SAM-dependent DNA methyltransferase', 'DNA MTase'),
    ('type I restriction-modification system subunit M', 'DNA MTase'),
    ('eco57I restriction-modification methylase family protein', 'DNA MTase'),
    # blastp https://www.ncbi.nlm.nih.gov/protein/WP_128833139.1 to 
    # https://www.ncbi.nlm.nih.gov/protein/WP_155765198.1, and you will get 87% alignment coverage.
    ('type II restriction endonuclease', 'DNA MTase'),

    ('DEAD/DEAH box helicase', 'helicase'),
    ('DEAD/DEAH box helicase family protein', 'helicase'),
    ('DISARM system SNF2-like helicase DrmD', 'helicase'),
    ('DISARM system helicase DrmA', 'helicase'),
    ('helicase', 'helicase'),
    ('ATP-dependent DNA helicase RecG', 'helicase'),

    ('GPW/gp25 family protein', 'phage'),
    ('tail fiber assembly protein', 'phage'),
    ('tail fiber domain-containing protein', 'phage'),
    ('tail fiber protein', 'phage'),
    ('baseplate J/gp47 family protein', 'phage'),
    ('baseplate assembly protein', 'phage'),
    ('phage GP46 family protein', 'phage'),
    ('phage baseplate assembly protein V', 'phage'),
    ('phage tail protein', 'phage'),
    ('phage tail protein I', 'phage'),

    ('TcpQ domain-containing protein', 'pili'),
    ('Flp pilus assembly complex ATPase component TadA', 'pili'),
    ('PilN family type IVB pilus formation outer membrane protein', 'pili'),
    ('pilin', 'pili'),
    ('pilus assembly protein PilX', 'pili'),
    ('prepilin peptidase', 'pili'),
    ('shufflon system plasmid conjugative transfer pilus tip adhesin PilV', 'pili'),
    ('type 4b pilus protein PilO2', 'pili'),
    ('type IV pilus biogenesis protein PilM', 'pili'),
    ('type IV pilus biogenesis protein PilP', 'pili'),

    ('restriction endonuclease subunit S', 'HsdS'),

    ('HsdR family type I site-specific deoxyribonuclease', 'HsdR'),
    ('type I restriction endonuclease subunit R', 'HsdR'),

    ('TonB-dependent siderophore receptor', TONB_DEPENDENT_TRANSPORTER_CLASS_NAME),
    ('RagB/SusD family nutrient uptake outer membrane protein', TONB_DEPENDENT_TRANSPORTER_CLASS_NAME),
    ('SusC/RagA family TonB-linked outer membrane protein', TONB_DEPENDENT_TRANSPORTER_CLASS_NAME),
    ('SusD/RagB family nutrient-binding outer membrane lipoprotein', TONB_DEPENDENT_TRANSPORTER_CLASS_NAME),
    ('TonB-dependent receptor', TONB_DEPENDENT_TRANSPORTER_CLASS_NAME),

    ('integrase core domain-containing protein', 'recombinase'),
    ('master DNA invertase Mpi family serine-type recombinase', 'recombinase'),
    ('recombinase family protein', 'recombinase'),
    ('site-specific integrase', 'recombinase'),
    ('tyrosine-type DNA invertase PsrA', 'recombinase'),
    ('tyrosine-type recombinase/integrase', 'recombinase'),
    ('invertase', 'recombinase'),
    ('Tsr0667 family tyrosine-type DNA invertase', 'recombinase'),
    
    # The ATP-binding cassette family: a structural perspective (https://link.springer.com/article/10.1007/s00018-009-0064-9) says:
    # "The ATP-binding cassette family is one of the largest groupings of membrane proteins, moving allocrites across lipid
    # membranes, using energy from ATP. In bacteria, they reside in the inner membrane and are involved in both uptake and export."
    # but Structure, Function, and Evolution of Bacterial ATP-Binding Cassette Systems
    # (https://journals.asm.org/doi/full/10.1128/MMBR.00031-07) says:
    # "ABC systems couple the energy of ATP hydrolysis to an impressively large variety of essential biological phenomena,
    # comprising not only transmembrane (TM) transport, for which they are best known, but also several non-transport-related
    # processes, such as translation elongation (62) and DNA repair (174)."
    # and most importantly, it seems, ABC-F Proteins Mediate Antibiotic Resistance through Ribosomal Protection 
    # (https://journals.asm.org/doi/10.1128/mBio.01975-15) says:
    # "ABC-F proteins are found across all three domains of life, and comprise a single polypeptide containing two 
    # ATP-binding cassette (ABC) domains separated by a linker of ~80 amino acids. In contrast to canonical ABC transporters, 
    # the ABC portions of ABC-F proteins are not fused to transmembrane domains (TMDs), nor are they genetically associated 
    # with TMDs in operons (4). While it may be that some ABC-F proteins associate with TMDs to mediate transport across 
    # membranes, it is nevertheless apparent that members of this family participate in biological processes other than 
    # transport, including DNA repair, enzyme regulation, and translational control (5). In Gram-positive bacteria, a 
    # subgroup of the ABC-F proteins mediates resistance to antibiotics that exert their action on the ribosome." 
    #('ABC-F family ATP-binding cassette domain-containing protein', 'ABC transporter'),
    #('ATP-binding cassette domain-containing protein', 'ABC transporter'),

    ('ABC transporter ATP-binding protein', 'ABC transporter'),
    ('ABC transporter permease', 'ABC transporter'),
    ('ABC transporter permease subunit', 'ABC transporter'),
    ('ABC transporter substrate-binding protein', 'ABC transporter'),
    ('amino acid ABC transporter permease', 'ABC transporter'),
    ('carbohydrate ABC transporter permease', 'ABC transporter'),
    ('sn-glycerol-3-phosphate ABC transporter ATP-binding protein UgpC', 'ABC transporter'),
    ('sugar ABC transporter ATP-binding protein', 'ABC transporter'),
    ('sugar ABC transporter permease', 'ABC transporter'),

    # ('hypothetical protein', 'hypothetical'),
]
PRODUCT_CLASS_DF = pd.DataFrame(LIST_OF_PRODUCT_AND_PRODUCT_CLASS, columns=['product', 'product_class'])

PRODUCT_CLASSES_WITHOUT_OTHER_ORDERED = [
    'recombinase',
    'HsdS',
    'HsdR',
    'DNA MTase',
    TONB_DEPENDENT_TRANSPORTER_CLASS_NAME,
    'phage',
    'helicase',
    'pili',
    'ABC transporter',
    # 'hypothetical',
]
assert set(PRODUCT_CLASSES_WITHOUT_OTHER_ORDERED) == set(PRODUCT_CLASS_DF['product_class'])
PRODUCT_CLASSES_ORDERED = PRODUCT_CLASSES_WITHOUT_OTHER_ORDERED + [
    'other',
]

assert len(PRODUCT_CLASSES_WITHOUT_OTHER_ORDERED) <= len(PRODUCT_CLASS_COLORS)
PRODUCT_CLASS_TO_COLOR = {
    **{
        x: c
        for x, c in zip(PRODUCT_CLASSES_WITHOUT_OTHER_ORDERED, PRODUCT_CLASS_COLORS) 
    },
    'other': 'white',
    # 'hypothetical protein': 'white',
    # 'hypothetical': (0.8,0.8,0.8),
}

PRODUCT_CLASS_AND_COLOR_DF = pd.DataFrame(list(PRODUCT_CLASS_TO_COLOR.items()), columns=['product_class', 'color'])

ATTR_NAME_TO_VAL_TO_COLOR = {
    'any_high_confidence_ir_pair_linked_to_cds_pair': {
        True: HIGH_CONFIDENCE_BP_COLOR,
        False: NO_HIGH_CONFIDENCE_BP_COLOR,
    },
}
ATTR_NAME_TO_VAL_TO_LABEL = {
    'any_high_confidence_ir_pair_linked_to_cds_pair': {
        True: 'variation identified',
        False: 'no variation identified',
        # True: 'rearrangements identified',
        # False: 'no rearrangements identified',
    },
}

LOGISTIC_REGRESSION_COEFFICIENT_BOOL_COLUMN_NAME_TO_LABEL = {
    # 'any_adjacent_recombinase': 'adjacent/nrecombinase',
    'any_adjacent_recombinase': 'proximal\nrecombinase',
    'high_operon_asymmetry': 
        'high\nasymmetry',
    'high_operon_strand_efficiency': 
        'high strand\nefficiency',
    'high_max_repeat_len': 
        'long\nrepeats',
    'low_operon_spacer_len': 
        'short CDS\ndistance',
    'high_operon_closest_repeat_position_orientation_matching': 'high orientation\nmatching',
}

def get_pvalue_repr(pvalue, verbose=True):
    pval_exponent = math.floor(np.log10(pvalue))
    if verbose:
        print(f'pvalue: {pvalue}')
    if pval_exponent >= -4:
        pvalue_repr = f'{pvalue:.4f}'
    else:
        pval_exponent_repr = f'\\mathdefault{{10^{{{pval_exponent}}}}}'
        pval_without_exponent = pvalue / (10 ** pval_exponent)
        if pval_without_exponent == 1:
            pvalue_repr = f'${pval_exponent_repr}$'
        else:
            pval_repr_without_exponent = f'{pval_without_exponent:.1f}'
            pvalue_repr = f'${pval_repr_without_exponent}\cdot{pval_exponent_repr}$'
        if verbose:
            print(pval_repr_without_exponent)
            print(f'pval_repr_without_exponent: {pval_repr_without_exponent}')
    
    if verbose:
        print(f'pvalue_repr: {pvalue_repr}')
    
    return pvalue_repr
    
def remove_trailing_zeros(num_str_repr):
    if '.' not in num_str_repr:
        return num_str_repr
    for i, c in enumerate(num_str_repr[::-1]):
        if c != '0':
            break
    num_of_trailing_zeros = i
    if num_of_trailing_zeros == 0:
        return num_str_repr
    if c == '.':
        return num_str_repr[:(-num_of_trailing_zeros - 1)]
    return num_str_repr[:(-num_of_trailing_zeros)]

def add_fake_bin_tick_labels_and_dashed_lines_to_histogram(ax, df, column_name, bin_size, last_real_bin_end=np.inf, first_real_bin_start=-np.inf, 
                                                           fake_val_threshold_line_color=FAKE_VAL_THRESHOLD_LINE_COLOR, 
                                                           fake_val_threshold_line_style=FAKE_VAL_THRESHOLD_LINE_STYLE, 
                                                           fake_val_threshold_line_alpha=FAKE_VAL_THRESHOLD_LINE_ALPHA):
    orig_ylim = ax.get_ylim()
    
    anything_in_fake_last_bin = (df[column_name] > last_real_bin_end).sum() > 0
    if anything_in_fake_last_bin:
        ax.plot([last_real_bin_end, last_real_bin_end], orig_ylim,
                linestyle=fake_val_threshold_line_style,color=fake_val_threshold_line_color,
                alpha=0.7)

    anything_in_fake_first_bin = (df[column_name] < first_real_bin_start).sum() > 0
    if anything_in_fake_first_bin:
        ax.plot([first_real_bin_start, first_real_bin_start], orig_ylim,
                linestyle=fake_val_threshold_line_style,color=fake_val_threshold_line_color,
                alpha=0.7)

    ax.set_ylim(orig_ylim)

    orig_x_tick_positions = ax.get_xticks()

    min_position_for_keeping_ticks = (first_real_bin_start + bin_size / 2) if anything_in_fake_first_bin else -np.inf

    max_position_for_keeping_ticks = (last_real_bin_end - bin_size / 2) if anything_in_fake_last_bin else np.inf

    num_of_smallest_ticks_to_discard = (orig_x_tick_positions < min_position_for_keeping_ticks).sum()
    num_of_ticks_to_keep = ((orig_x_tick_positions >= min_position_for_keeping_ticks) & (orig_x_tick_positions <= max_position_for_keeping_ticks)).sum()
    new_x_tick_positions_without_fake = list(orig_x_tick_positions[num_of_smallest_ticks_to_discard:][:num_of_ticks_to_keep])

    new_x_tick_positions = new_x_tick_positions_without_fake
    new_x_tick_labels = [remove_trailing_zeros(str(f'{x:.2f}')) for x in new_x_tick_positions_without_fake]
    new_xlim = list(ax.get_xlim())
    if anything_in_fake_first_bin:
        fake_first_bin_middle = first_real_bin_start - bin_size / 2
        new_x_tick_positions = [fake_first_bin_middle] + new_x_tick_positions
        new_x_tick_labels = [f'<{first_real_bin_start}'] + new_x_tick_labels
    else:
        new_xlim[0] = first_real_bin_start
        
    if anything_in_fake_last_bin:
        fake_last_bin_middle = last_real_bin_end + bin_size / 2
        new_x_tick_positions.append(fake_last_bin_middle)
        new_x_tick_labels.append(f'$\geqslant${last_real_bin_end}')
        # new_x_tick_labels.append(f'$\geq${last_real_bin_end}')
    else:
        new_xlim[1] = last_real_bin_end


    ax.set_xticks(new_x_tick_positions)
    ax.set_xticklabels(new_x_tick_labels)

    ax.set_xlim(new_xlim)
    

def remove_all_ax_spines_an_x_and_y_axes(ax):
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)
    
def add_product_class_column(df, product_column_name, product_class_column_name, other_product_class='other'):
    df = df.copy()
    assert product_class_column_name not in df
    df = df.merge(PRODUCT_CLASS_DF.rename(columns={'product': product_column_name}), how='left')
    
    if other_product_class is None:
        df.loc[df[product_class_column_name].isna(), product_class_column_name] = df.loc[df[product_class_column_name].isna(), product_column_name]
    else:
        df[product_class_column_name].fillna('other', inplace=True)

    assert not df[product_class_column_name].isna().any()
    
    return df

def get_alignment_region_raw_read_alignment_args_by_cds_context_name_and_locus(
        cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args, 
        cds_context_name, 
        nuccore_accession, 
        longer_linked_repeat_cds_region,
):
    alignment_regions_raw_read_alignment_args = cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args[
        cds_context_name][nuccore_accession]
    
    for alignment_region_raw_read_alignment_args in alignment_regions_raw_read_alignment_args:
        if alignment_region_raw_read_alignment_args['longest_linked_repeat_cds_region_and_protein_id'][0] == longer_linked_repeat_cds_region:
            return alignment_region_raw_read_alignment_args
    assert False

            
def get_cds_df_with_product_class(cds_df_csv_file_path, alignment_region_raw_read_alignment_args):
    cds_df = pd.read_csv(cds_df_csv_file_path, sep='\t', low_memory=False)
    cds_df = add_product_class_column(cds_df, 'product', 'product_class')

    if 'cds_start_and_end_to_curated_product_class' in alignment_region_raw_read_alignment_args:
        cds_start_and_end_to_curated_product_class = alignment_region_raw_read_alignment_args['cds_start_and_end_to_curated_product_class']

        assert {'start_pos', 'end_pos'} <= set(cds_df)
        cds_df = cds_df.merge(pd.DataFrame([(start, end, product_class) for ((start, end), product_class) in cds_start_and_end_to_curated_product_class.items()], 
                     columns=['start_pos', 'end_pos', 'curated_product_class']), how='left')

        cds_with_replaced_product_class_df = cds_df[~cds_df['curated_product_class'].isna()]
        replaced_and_curated_products_are_as_expected = (
            (cds_with_replaced_product_class_df['product_class'] == 'other') |
            # (cds_with_replaced_product_class_df['product_class'] == 'hypothetical') |
            ((cds_with_replaced_product_class_df['product_class'] == 'helicase') & (cds_with_replaced_product_class_df['curated_product_class'] == 'HsdR'))
        ).all()
        if not replaced_and_curated_products_are_as_expected:
            raise RuntimeError(f'replaced_and_curated_products_are_as_expected == False. '
                               f'cds_with_replaced_product_class_df:\n{cds_with_replaced_product_class_df}')
        
        cds_df.loc[~cds_df['curated_product_class'].isna(), 'product_class'] = cds_df.loc[~cds_df['curated_product_class'].isna(), 'curated_product_class'] 
        
    
    return cds_df

def get_variant_ir_pairs_to_num_of_evidence_reads(read_name_to_possible_ir_pairs_used_to_reach_from_ref):
    variant_ir_pairs_to_num_of_evidence_reads = collections.defaultdict(int)
    for variant_ir_pairs in read_name_to_possible_ir_pairs_used_to_reach_from_ref.values():
        variant_ir_pairs_to_num_of_evidence_reads[frozenset(variant_ir_pairs)] += 1
    variant_ir_pairs_to_num_of_evidence_reads = dict(variant_ir_pairs_to_num_of_evidence_reads) # I don't want a defaultdict moving around.
    return variant_ir_pairs_to_num_of_evidence_reads

In [ ]:
ODDS_RATIO_DESCRIPTION_START = 'enrichment in predicted programmed inversion'
TARGET_PRODUCT_ODDS_RATIO_DESCRIPTION_END = 'targets'
NEIGHBOR_PRODUCT_ODDS_RATIO_DESCRIPTION_END = 'target neighbors'

TARGET_PRODUCT_ODDS_RATIO_DESCRIPTION = f'{ODDS_RATIO_DESCRIPTION_START} {TARGET_PRODUCT_ODDS_RATIO_DESCRIPTION_END} (odds ratio)'
NEIGHBOR_PRODUCT_ODDS_RATIO_DESCRIPTION = f'{ODDS_RATIO_DESCRIPTION_START} {NEIGHBOR_PRODUCT_ODDS_RATIO_DESCRIPTION_END} (odds ratio)'

TARGET_PRODUCT_ENRICHMENT_TABLE_ORDERED_COLUMN_NAME_AND_FINAL_NAME_AND_WIDTH = (
    ('product', 'GenBank CDS product annotation', 60),
    ('odds_ratio', TARGET_PRODUCT_ODDS_RATIO_DESCRIPTION, 16),
    ('corrected_pvalue', f'corrected p-value', 10),
)
NEIGHBOR_PRODUCT_ENRICHMENT_TABLE_ORDERED_COLUMN_NAME_AND_FINAL_NAME_AND_WIDTH = (
    ('product', 'GenBank CDS product annotation', 60),
    ('odds_ratio', NEIGHBOR_PRODUCT_ODDS_RATIO_DESCRIPTION, 16),
    ('corrected_pvalue', f'corrected p-value', 10),
)
PRODUCT_ENRICHMENT_COLUMN_NAME_TO_NUM_FORMAT_SPECIFIER = {'odds_ratio': '0.000', 'corrected_pvalue': 11}

# DEFAULT_REARRANGED_REGION_CMAP = 'jet'
# DEFAULT_REARRANGED_REGION_CMAP = 'gist_rainbow'
# DEFAULT_REARRANGED_REGION_CMAP = 'Greys'
# DEFAULT_REARRANGED_REGION_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['1', '0.2'])
# DEFAULT_REARRANGED_REGION_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', [(0, '1'), (0.4, '0.8'), (0.6, '0.45'), (1, '0.25')])
# DEFAULT_REARRANGED_REGION_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', [(0, '0.9'), (0.4, '0.7'), (0.6, '0.45'), (1, '0.25')])
# DEFAULT_REARRANGED_REGION_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['1', '0.1'])
DEFAULT_REARRANGED_REGION_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', [(0, '0.9'), (0.45, '0.7'), (0.55, '0.45'), (1, '0.25')])
# REDS_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['darkred', 'lightcoral'])
REDS_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['darkred', 'rosybrown'])
BLUES_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['darkblue', 'lightsteelblue'])
BLUE_PURPLE_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['blue', 'purple'])
BLUE_CYAN_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['blue', 'cyan'])
RED_GOLD_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['red', 'gold'])
RED_ORANGE_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['darkred', 'darkorange'])
PURPLE_ORANGE_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['purple', 'darkorange'])
RED_MAGENTA_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['darkred', 'magenta'])
GREEN_MAGENTA_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['lawngreen', 'magenta'])
RED_YELLOW_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['red', 'yellow'])
YELLOW_RED_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['yellow', 'red'])
YELLOW_DARK_RED_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list('', ['yellow', 'darkred'])

def get_matching_chunk(chunk, interval, matching_interval, matching_interval_strand):
    chunk_in_matching_interval = (
        [x - interval[0] for x in chunk] if (matching_interval_strand == 1) else 
        [interval[1] - x for x in chunk[::-1]]
    )
    matching_chunk = [x + matching_interval[0] for x in chunk_in_matching_interval]
    return matching_chunk

def draw_cds_invertion_on_ax(
    ax,
    region=None,
    cds_df=pd.DataFrame(),
    ir_pairs_df=pd.DataFrame(),
    interval_to_matching_interval_and_strand=dict(),
    region_with_gradient=None,
    draw_matching_region_cds=True,
    cds_arrowhead_width=None,
    region_cmap=DEFAULT_REARRANGED_REGION_CMAP,
    region_bottom_and_top=(0.6, 0.8),
    matching_region_bottom_and_top=(0.2, 0.4),
    cds_polygon_linewidth=1.7,
    cds_polygon_edge_color_with_alpha='black',
    genome_rectangle_linewidth=0.6, # seems like for a low linewidth, black becomes grey (i guess because boundaries are gradual?)
    genome_rectangle_alpha=1,
    product_class_to_fill_color_with_alpha=None,
    draw_region_rectangle=True,
    transform=None,
    clip_on=True,
    dont_show_product_name=True,
    draw_cds_polygon_again=False, # for overlapping CDSs
    ir_pairs_pointing_outward=False,
):
    repeat_arrow_props = dict(
        shrinkA=0, shrinkB=0, 
        # linewidth=0.5, 
        linewidth=1, 
        # arrowstyle="->", 
        # arrowstyle='->,head_width=5',
        arrowstyle='->,head_length=0.22',
        # mutation_scale=5,
    )
    
    cds_height = region_bottom_and_top[1] - region_bottom_and_top[0]
    region_middle_y = np.mean(region_bottom_and_top)
    matching_region_middle_y = np.mean(matching_region_bottom_and_top)
    height_truncate = (0.05 * cds_height) if draw_region_rectangle else 0
    cds_region_bottom_and_top = (region_bottom_and_top[0] + height_truncate, region_bottom_and_top[1] - height_truncate)
    cds_matching_region_bottom_and_top = (matching_region_bottom_and_top[0] + height_truncate, matching_region_bottom_and_top[1] - height_truncate)
    
    pair_infos = ir_pairs_df.to_dict('records')
    cds_infos = cds_df.to_dict('records')

    def show_product_name(top_left, top_right, bottom_left, cds_product):
        if not dont_show_product_name:
            ax.text(
                # np.mean((top_left[0],top_right[0])), 
                np.quantile((top_left[0],top_right[0]), 0.46), 
                # np.mean((top_left[1],bottom_left[1])), 
                np.quantile((bottom_left[1], top_left[1]), 0.44), 
                cds_product, va='center', ha='center', transform=transform, fontsize='x-small',
            )
        
    
    if transform is None:
        transform = ax.transData
    
    if region is not None:
        genome_height = region_bottom_and_top[1] - region_bottom_and_top[0]
        
        region_len = region[1] - region[0] + 1
        if draw_region_rectangle:
            ax.add_patch(matplotlib.patches.Rectangle((region[0], region_bottom_and_top[0]), region_len, genome_height,
                                                      color='black', 
                                                      # alpha=genome_rectangle_alpha, 
                                                      linewidth=genome_rectangle_linewidth, fill=False, transform=transform, clip_on=clip_on))
        
        if interval_to_matching_interval_and_strand:
            assert np.isclose(genome_height, matching_region_bottom_and_top[1] - matching_region_bottom_and_top[0])
            if draw_region_rectangle:
                ax.add_patch(matplotlib.patches.Rectangle((region[0], matching_region_bottom_and_top[0]), region_len, genome_height,
                                                          color='black', 
                                                          # alpha=genome_rectangle_alpha, 
                                                          linewidth=genome_rectangle_linewidth, fill=False, transform=transform, clip_on=clip_on))
            

            if region_with_gradient is None:
                interval_with_non_identical_matching_interval_to_matching_interval_and_strand = {
                    interval: (matching_interval, matching_interval_strand)
                    for interval, (matching_interval, matching_interval_strand) in interval_to_matching_interval_and_strand.items()
                    if (matching_interval_strand == -1) or (interval != matching_interval)
                }
                region_with_gradient = (min(x[0] for x in interval_with_non_identical_matching_interval_to_matching_interval_and_strand),
                                        max(x[1] for x in interval_with_non_identical_matching_interval_to_matching_interval_and_strand))
        


            region_gradient = list(range(region_with_gradient[0], region_with_gradient[1] + 2)) # why +2 and not +1?
            ax.imshow(
                [region_gradient],
                cmap=region_cmap,
                extent=(region_with_gradient[0], region_with_gradient[1] + 1, *region_bottom_and_top),
                aspect='auto',
                vmin=region_with_gradient[0],
                vmax=(region_with_gradient[1] + 1),
                transform=transform,
            )

            for interval, (matching_interval, matching_interval_strand) in interval_to_matching_interval_and_strand.items():
                if (interval[0] >= region_with_gradient[0]) and (interval[1] <= region_with_gradient[1]):
                    interval_gradient = list(range(interval[0], interval[1] + 2))
                    ax.imshow(
                        [interval_gradient if (matching_interval_strand == 1) else interval_gradient[::-1]],
                        cmap=region_cmap,
                        extent=(matching_interval[0], matching_interval[1] + 1, *matching_region_bottom_and_top),
                        aspect='auto',
                        vmin=region_with_gradient[0],
                        vmax=(region_with_gradient[1] + 1),
                        transform=transform,
                    )
    
    for pair_info in pair_infos:
        ir_color = pair_info['color']
        repeat_len = pair_info['right1'] - pair_info['left1'] + 1
        curr_arrow_props = {**repeat_arrow_props, 'color': ir_color}
        for repeat_num in (1,2):
            repeat_gradient = list(range(repeat_len + 1))
            if repeat_num == 1:
                repeat_gradient = repeat_gradient[::-1]
            left = pair_info[f'left{repeat_num}']
            right = pair_info[f'right{repeat_num}']
            repeat = (left, right)
            
            arrow_end, arrow_start = repeat
            if ir_pairs_pointing_outward:
                arrow_end, arrow_start = arrow_start, arrow_end
            if repeat_num == 2:
                arrow_end, arrow_start = arrow_start, arrow_end
            ax.annotate("", xy=(arrow_start, region_middle_y), 
                        xytext=(arrow_end, region_middle_y), arrowprops=curr_arrow_props)
            for interval, (matching_interval, matching_interval_strand) in interval_to_matching_interval_and_strand.items():
                repeat_chunk = generic_utils.get_intersection_of_2_intervals(repeat, interval)
                if repeat_chunk:
                    assert repeat == repeat_chunk
                    matching_repeat_chunk = get_matching_chunk(
                        chunk=repeat_chunk, 
                        interval=interval,
                        matching_interval=matching_interval,
                        matching_interval_strand=matching_interval_strand,
                    )
                    
                    arrow_end, arrow_start = matching_repeat_chunk
                    if (matching_interval_strand == 1) ^ (repeat_num == 1):
                        arrow_end, arrow_start = arrow_start, arrow_end
                    ax.annotate("", xy=(arrow_start, matching_region_middle_y), 
                                xytext=(arrow_end, matching_region_middle_y), arrowprops=curr_arrow_props)
    
    open_unfilled_polygon_common_kwargs = dict(closed=False, fill=False, edgecolor=cds_polygon_edge_color_with_alpha, 
                                               linewidth=cds_polygon_linewidth, transform=transform, clip_on=clip_on)
    for cds_info in cds_infos:
        start = cds_info['start_pos']
        end = cds_info['end_pos']
        cds_is_on_forward_strand = cds_info['strand'] == 1
        cds_region = (start, end)
        cds_product = cds_info['product'] if ('product' in cds_info) else None
        
        fill = False
        fill_color = None
        if product_class_to_fill_color_with_alpha:
            product_class = cds_info['product_class']
            if product_class in product_class_to_fill_color_with_alpha:
                fill = True
                fill_color = product_class_to_fill_color_with_alpha[product_class]
        
        closed_polygon_common_kwargs = dict(closed=True, fill=fill, facecolor=fill_color, edgecolor=cds_polygon_edge_color_with_alpha,
                                            linewidth=cds_polygon_linewidth, transform=transform, clip_on=clip_on)
        closed_unfilled_polygon_common_kwargs = dict(closed_polygon_common_kwargs)
        closed_unfilled_polygon_common_kwargs['fill'] = False
        closed_unfilled_polygon_common_kwargs['facecolor'] = None
        # 211112: I split to multiple calls to Polygon because currently there seems to be an annoying bug (in matplotlib, i guess) that makes the 
        # polygon position inaccurate when it includes a diagonal line. ugh. same for each polygon below that contains a diagonal line...
        if cds_is_on_forward_strand:
            top_left = [start, cds_region_bottom_and_top[1]]
            bottom_left = [start, cds_region_bottom_and_top[0]]
            top_right = [end - cds_arrowhead_width, cds_region_bottom_and_top[1]]
            bottom_right = [end - cds_arrowhead_width, cds_region_bottom_and_top[0]]
            polgyon_position_arg = [bottom_right, bottom_left, top_left, top_right, (end, np.mean(cds_region_bottom_and_top))]
            
        else:     
            top_left = [start + cds_arrowhead_width, cds_region_bottom_and_top[1]]
            bottom_left = [start + cds_arrowhead_width, cds_region_bottom_and_top[0]]
            top_right = [end, cds_region_bottom_and_top[1]]
            bottom_right = [end, cds_region_bottom_and_top[0]]
            polgyon_position_arg = [top_left, top_right, bottom_right, bottom_left, (start, np.mean(cds_region_bottom_and_top))]
        
        ax.add_patch(matplotlib.patches.Polygon(polgyon_position_arg, **closed_polygon_common_kwargs))
        if draw_cds_polygon_again:
            ax.add_patch(matplotlib.patches.Polygon(polgyon_position_arg, **closed_unfilled_polygon_common_kwargs, zorder=100))

        if cds_product:
            show_product_name(top_left, top_right, bottom_left, cds_product)
            

        if draw_matching_region_cds:
            for interval, (matching_interval, matching_interval_strand) in interval_to_matching_interval_and_strand.items():
                cds_chunk = generic_utils.get_intersection_of_2_intervals(cds_region, interval)
                if cds_chunk:
                    matching_cds_chunk = get_matching_chunk(
                        chunk=cds_chunk, 
                        interval=interval,
                        matching_interval=matching_interval,
                        matching_interval_strand=matching_interval_strand,
                    )
                    top_left = (matching_cds_chunk[0], cds_matching_region_bottom_and_top[1])
                    top_right = (matching_cds_chunk[1], cds_matching_region_bottom_and_top[1])
                    bottom_left = (matching_cds_chunk[0], cds_matching_region_bottom_and_top[0])
                    bottom_right = (matching_cds_chunk[1], cds_matching_region_bottom_and_top[0])
                    cds_chunk_contains_cds_start = cds_chunk[0] == cds_region[0]
                    cds_chunk_contains_cds_end = cds_chunk[1] == cds_region[1]
                    if not cds_is_on_forward_strand:
                        cds_chunk_contains_cds_start, cds_chunk_contains_cds_end = cds_chunk_contains_cds_end, cds_chunk_contains_cds_start
                    matching_interval_is_on_forward_strand = matching_interval_strand == 1
                    matching_interval_cds_is_on_forward_strand = not (cds_is_on_forward_strand ^ matching_interval_is_on_forward_strand)
                    
                    if not cds_chunk_contains_cds_end:
                        if not cds_chunk_contains_cds_start:
                            ax.add_patch(matplotlib.patches.Polygon([top_left, top_right], **open_unfilled_polygon_common_kwargs))
                            ax.add_patch(matplotlib.patches.Polygon([bottom_right, bottom_left], **open_unfilled_polygon_common_kwargs))

                        else:
                            assert cds_chunk_contains_cds_start
                            if matching_interval_cds_is_on_forward_strand:
                                ax.add_patch(matplotlib.patches.Polygon([bottom_right, bottom_left, top_left, top_right], **open_unfilled_polygon_common_kwargs))
                            else:
                                ax.add_patch(matplotlib.patches.Polygon([top_left, top_right, bottom_right, bottom_left], **open_unfilled_polygon_common_kwargs))
                            
                    else:
                        assert cds_chunk_contains_cds_end
                        if matching_interval_cds_is_on_forward_strand:
                            top_right = (top_right[0] - cds_arrowhead_width, top_right[1])
                            bottom_right = (bottom_right[0] - cds_arrowhead_width, bottom_right[1])
                        else:
                            top_left = (top_left[0] + cds_arrowhead_width, top_left[1])
                            bottom_left = (bottom_left[0] + cds_arrowhead_width, bottom_left[1])
                            
                        if cds_chunk_contains_cds_start:
                            if matching_interval_cds_is_on_forward_strand:
                                ax.add_patch(matplotlib.patches.Polygon([bottom_right, bottom_left, top_left, top_right], **open_unfilled_polygon_common_kwargs))
                                ax.add_patch(matplotlib.patches.Polygon([top_right, [matching_cds_chunk[1], np.mean(cds_matching_region_bottom_and_top)], 
                                                                         bottom_right], **open_unfilled_polygon_common_kwargs))
                            else:
                                ax.add_patch(matplotlib.patches.Polygon([top_left, top_right, bottom_right, bottom_left],
                                                                        **open_unfilled_polygon_common_kwargs))
                                ax.add_patch(matplotlib.patches.Polygon([bottom_left, [matching_cds_chunk[0], np.mean(cds_matching_region_bottom_and_top)], 
                                                                         top_left], **open_unfilled_polygon_common_kwargs))
                            if cds_product:
                                show_product_name(top_left, top_right, bottom_left, cds_product)
                                
                        else:
                            assert not cds_chunk_contains_cds_start
                            ax.add_patch(matplotlib.patches.Polygon([bottom_right, bottom_left], **open_unfilled_polygon_common_kwargs))
                            ax.add_patch(matplotlib.patches.Polygon([top_left, top_right], **open_unfilled_polygon_common_kwargs))
                            if not (cds_is_on_forward_strand ^ matching_interval_is_on_forward_strand):
                                ax.add_patch(matplotlib.patches.Polygon([top_right, [matching_cds_chunk[1], np.mean(matching_region_bottom_and_top)],
                                                                         bottom_right], **open_unfilled_polygon_common_kwargs))
                            if cds_is_on_forward_strand ^ matching_interval_is_on_forward_strand:
                                ax.add_patch(matplotlib.patches.Polygon([bottom_left, [matching_cds_chunk[0], np.mean(matching_region_bottom_and_top)], 
                                                                         top_left], **open_unfilled_polygon_common_kwargs))
    
        
    remove_all_ax_spines_an_x_and_y_axes(ax)
    
def set_x_lim_according_to_region(ax, region, margin_relative_size):
    region_len = region[1] - region[0]
    margin_size = margin_relative_size * region_len
    new_xlim = (region[0] - margin_size, region[1] + margin_size)
    ax.set_xlim(new_xlim)

In [ ]:
if 'fig1' in locals():
    plt.close(fig1)
# raise    
fig1 = plt.figure(
    figsize=(7,5), 
    # constrained_layout=True,
)
fig1_gridspec = fig1.add_gridspec(
    6, 16, 
    height_ratios=[2,0.65,1,1,1,1],
)

fig1.subplots_adjust(
    top=0.95,
    bottom=0.05,
    left=0.05,
    right=0.95,
    # hspace=0.2,
    # wspace=0.2
)
# fig1.tight_layout()

fig1a_ax = fig1.add_subplot(fig1_gridspec[0,:8])
fig1c_ax = fig1.add_subplot(fig1_gridspec[0,10:])

fig1b1_ax = fig1.add_subplot(fig1_gridspec[2:4,:3])
fig1b2_ax = fig1.add_subplot(fig1_gridspec[2:4,3:7])
fig1b3_ax = fig1.add_subplot(fig1_gridspec[2:4,7:11])
fig1b4_ax = fig1.add_subplot(fig1_gridspec[2:4,12:])
fig1b5_ax = fig1.add_subplot(fig1_gridspec[4:,11:])
fig1b6_ax = fig1.add_subplot(fig1_gridspec[4:,6:10])
fig1b7_ax = fig1.add_subplot(fig1_gridspec[4:,:5])


def draw_schematic_invertible_cdss_on_fig_and_ax(fig, ax, cds_height, ir_pair_color, no_ir_pairs=False, larger_region=False, 
                                                 nearby_recombinase=False, no_cds_pairs=False, very_long_repeats=False, 
                                                 cds_pairs_outside_invertible_region=False, draw_alignment_hourglass=False):
    region_1a_end = 11500
    fig1a_region = (1, region_1a_end)
    if larger_region:
        fig1a_region = (fig1a_region[0] - 2000, fig1a_region[1] + 2000)
    
    if no_cds_pairs:
        cds_1a_df = pd.DataFrame()
    elif cds_pairs_outside_invertible_region:
        cds_1a_df = pd.DataFrame([
            {
                'start_pos': -800,
                'end_pos': 1600,
                'strand': 1,
                'product': '',
            },
            {
                'start_pos': 9400,
                'end_pos': 12200,
                'strand': 1,
                'product': '',
            },
        ])
    else:
        cds_1a_df = pd.DataFrame([
            {
                'start_pos': 1700,
                'end_pos': 4500,
                'strand': 1,
                'product': '',
            },
            {
                'start_pos': 6500,
                'end_pos': 10500,
                'strand': -1,
                'product': '',
            },
            *([{
                'start_pos': -1400,
                'end_pos': 600,
                'strand': -1,
                'product': 'Rec',
            }] if nearby_recombinase else [])
        ])
    

    repeat_len = 2000 if very_long_repeats else 800 
    cds_arrowhead_width = 300
    
    right1_1a = 3100
    left2_1a = 7900
    
    left1_1a = right1_1a - repeat_len
    right2_1a = left2_1a + repeat_len
    pairs_1a_df = pd.DataFrame([
        {
            'left1': left1_1a,
            'right1': right1_1a,
            'left2': left2_1a,
            'right2': right2_1a,
            'color': ir_pair_color,
        },
    ])
    intervals_1a = (
        (fig1a_region[0], right1_1a),
        (right1_1a + 1, left2_1a - 1),
        (left2_1a, fig1a_region[1]),
    )
    interval_to_matching_interval_and_strand_1a = {
        intervals_1a[0]: (intervals_1a[0], 1),
        intervals_1a[1]: (intervals_1a[1], -1),
        intervals_1a[2]: (intervals_1a[2], 1),
    }

    region_top = 0.85
    matching_region_bottom = 0.15
    
    region_bottom = region_top - cds_height
    region_bottom_and_top = (region_bottom, region_top)
    matching_region_top = matching_region_bottom + cds_height
    
    
    draw_cds_invertion_on_ax(
        ax=ax,
        region=fig1a_region,
        cds_df=cds_1a_df,
        ir_pairs_df=(pd.DataFrame() if no_ir_pairs else pairs_1a_df),
        interval_to_matching_interval_and_strand=interval_to_matching_interval_and_strand_1a,
        cds_arrowhead_width=cds_arrowhead_width,
        region_bottom_and_top=region_bottom_and_top,
        matching_region_bottom_and_top=(matching_region_bottom, matching_region_top),
        dont_show_product_name=False,
    )
    
    if draw_alignment_hourglass:
        ax.add_patch(matplotlib.patches.Polygon([
            (intervals_1a[1][0], region_bottom), 
            (intervals_1a[1][1], region_bottom), 
            (intervals_1a[1][0], matching_region_top), 
            (intervals_1a[1][1], matching_region_top), 
        ], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
        linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))

    set_x_lim_according_to_region(ax, fig1a_region, margin_relative_size=0.1)
    ax.set_ylim(0, 1)




def draw_pipeline_on_fig_and_axes(fig, ax1, ax2, ax3, ax4, ax5, ax6, ax7, cds_height, dist_between_desc_and_cds,
                                  ir_pair1_color, ir_pair2_color, same_variant_in_other_genome_in_ax7=False):
    scaffold_break_linewidth = 1.3
    
    redX_alpha = 0.8
    redX_size = 100
    redX_linewidths = 3
    
    cluster_alpha = 0.4
    cluster_linewidth = 1.5
    
    question_mark_rectangle_linewidth = 1.5
    

    dist_between_desc_and_cds_with_cluster = dist_between_desc_and_cds + 0.02
    
    vertical_dist_between_region_in_ref_and_other_genome = 0.15
    
    remove_all_ax_spines_an_x_and_y_axes(ax1)
    
    if num_of_taxa >= 1e3:
        num_of_taxa_str_repr = generic_utils.get_num_rounded_to_thousands_str_repr(num_of_taxa, 1)
    else:
        num_of_taxa_str_repr = str(int(num_of_taxa))

    if num_of_all_cds >= 1e6:
        num_of_all_cds_str_repr = generic_utils.get_num_rounded_to_millions_str_repr(num_of_all_cds)
    else:
        num_of_all_cds_str_repr = generic_utils.get_num_rounded_to_thousands_str_repr(num_of_all_cds)

    if num_of_all_ir_pairs >= 1e6:
        num_of_all_ir_pairs_str_repr = generic_utils.get_num_rounded_to_millions_str_repr(num_of_all_ir_pairs, 1)
    else:
        num_of_all_ir_pairs_str_repr = generic_utils.get_num_rounded_to_thousands_str_repr(num_of_all_ir_pairs, 1)

    orig_num_of_cds_pairs = len(ir_pairs_linked_df[index_column_names.CDS_PAIR_INDEX_COLUMN_NAMES].drop_duplicates())
    orig_num_of_cds_pairs_str_repr = generic_utils.get_num_rounded_to_thousands_str_repr(
        orig_num_of_cds_pairs)

    print(f'num_of_taxa: '
          f'{num_of_taxa}')
    print(f'orig_num_of_cds_pairs: '
          f'{orig_num_of_cds_pairs}')

    num_of_cds_pairs_after_discarding_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num = len(
        pairs_after_discarding_ir_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_df[
            index_column_names.CDS_PAIR_INDEX_COLUMN_NAMES].drop_duplicates())
    num_of_cds_pairs_after_discarding_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_str_repr = (
        generic_utils.get_num_rounded_to_thousands_str_repr(
            num_of_cds_pairs_after_discarding_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num))

    print(f'num_of_cds_pairs_after_discarding_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num: '
          f'{num_of_cds_pairs_after_discarding_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num}')

    stage5_num_of_cds_pairs_with_any_homologous_locus_str_repr = generic_utils.get_num_rounded_to_thousands_str_repr(stage5_num_of_cds_pairs_with_any_homologous_locus)
    print(f'stage5_num_of_cds_pairs_with_any_homologous_locus: {stage5_num_of_cds_pairs_with_any_homologous_locus}')

    num_of_cds_pairs_with_high_confidence_ir_pairs = len(
        filtered_pairs_with_highest_confidence_bps_df[index_column_names.CDS_PAIR_INDEX_COLUMN_NAMES].drop_duplicates())
    num_of_cds_pairs_with_high_confidence_ir_pairs_str_repr = str(num_of_cds_pairs_with_high_confidence_ir_pairs)


    ax1.text(0.38, 0.5, f'annotated species\nrepresentative\ngenomes ({num_of_taxa_str_repr})', 
                   va='center', ha='center')

    arrow_edge_dist_from_y_middle_in_fig1b1 = 0.05
    arrow_edge_dist_from_y_middle_in_fig1b2 = 0.14
    
    for sign in (1, -1):
        fig.add_artist(matplotlib.patches.ConnectionPatch(
            (0.97, 0.5 + arrow_edge_dist_from_y_middle_in_fig1b1 * sign),
            (0.06, 0.5 + arrow_edge_dist_from_y_middle_in_fig1b2 * sign),
            coordsA=ax1.transAxes,
            coordsB=ax2.transAxes,
            color="black",
            arrowstyle="-|>",
        ))


    ax2_region = (200, 1800)
    ax2_region_middle = np.mean(ax2_region)
    ax2_cds_arrowhead_width = 110

    desc_bottom = 0.72
    ax2.text(ax2_region_middle, desc_bottom, f'coding sequences\n(CDSs, {num_of_all_cds_str_repr})', va='bottom', ha='center')
    region_top = desc_bottom - dist_between_desc_and_cds
    region_bottom = region_top - cds_height
    draw_cds_invertion_on_ax(
        ax=ax2,
        region=ax2_region,
        cds_df=pd.DataFrame([{'start_pos': 700,
                              'end_pos': 1300,
                              'strand': 1,
                              'product': '',}]),
        cds_arrowhead_width=ax2_cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
        # cds_polygon_linewidth=2.2,
    )

    desc_bottom = 0.2
    ax2.text(ax2_region_middle, desc_bottom, f'Inverted Repeats\n(IRs, {num_of_all_ir_pairs_str_repr})', va='bottom', ha='center')
    region_top = desc_bottom - dist_between_desc_and_cds
    region_bottom = region_top - cds_height
    draw_cds_invertion_on_ax(
        ax=ax2,
        region=ax2_region,
        ir_pairs_df=pd.DataFrame([
            {
                'left1': 500,
                'right1': 800,
                'left2': 1200,
                'right2': 1500,
                'color': ir_pair1_color,
            },
        ]),
        region_bottom_and_top=(region_bottom, region_top),
    )

    set_x_lim_according_to_region(ax2, ax2_region, margin_relative_size=0.4)
    ax2.set_ylim(0, 1)

    for sign in (1, -1):
        fig.add_artist(matplotlib.patches.ConnectionPatch(
            (0.95, 0.5 + arrow_edge_dist_from_y_middle_in_fig1b2 * sign),
            (0.1, 0.5 + arrow_edge_dist_from_y_middle_in_fig1b1 * sign),
            coordsA=ax2.transAxes,
            coordsB=ax3.transAxes,
            color="black",
            arrowstyle="-|>",
        ))



    ax3_region = (-300, 2300)
    ax3_region_middle = np.mean(ax3_region)
    ax3_cds_arrowhead_width = 160

    desc_bottom = 0.45
    ax3.text(ax3_region_middle, desc_bottom, 
                   f'CDS pairs with\n IRs ({orig_num_of_cds_pairs_str_repr})', va='bottom', ha='center')

    region_top = desc_bottom - dist_between_desc_and_cds
    region_bottom = region_top - cds_height

    cds1_start = -50
    cds2_end = 2050
    repeat_len = 280
    left1 = cds1_start + 250
    right2 = cds2_end - 130
    
    draw_cds_invertion_on_ax(
        ax=ax3,
        region=ax3_region,
        cds_df=pd.DataFrame([{'start_pos': cds1_start,
                              'end_pos': 950,
                              'strand': 1,
                              'product': '',},
                             {'start_pos': 1250,
                              'end_pos': cds2_end,
                              'strand': -1,
                              'product': '',}]),
        ir_pairs_df=pd.DataFrame([
            {
                'left1': left1,
                'right1': left1 + repeat_len ,
                'left2': right2 - repeat_len,
                'right2': right2,
                'color': ir_pair1_color,
            },
        ]),
        cds_arrowhead_width=ax3_cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
        # cds_polygon_linewidth=1,
    )
    set_x_lim_according_to_region(ax3, ax3_region, margin_relative_size=0.25)
    ax3.set_ylim(0, 1)


    fig.add_artist(matplotlib.patches.ConnectionPatch(
        (0.94, 0.5),
        (-0.18, 0.5),
        coordsA=ax3.transAxes,
        coordsB=ax4.transAxes,
        color="black",
        arrowstyle="-|>",
    ))



    ax4_cds_arrowhead_width = 170
    ax4_repeat1_len = 280
    repeat_leftmost_copy_left = -600
    repeat_rightmost_copy_left = 3050
    ax4_region = (repeat_leftmost_copy_left - 200, repeat_rightmost_copy_left + ax4_repeat1_len + 200)
    ax4_region_middle = np.mean(ax4_region)

    desc_bottom = 0.33
    ax4.text(ax4_region_middle, desc_bottom, 
             f'discard CDS pairs with\nrepetitive IRs, yielding\nprogrammed inversion\ncandidates (PICs, '
             f'{num_of_cds_pairs_after_discarding_pairs_linked_to_cds_containing_repeats_of_ir_pairs_with_high_repeat_estimated_copy_num_str_repr})', 
             va='bottom', ha='center')
    # alternatives to "yielding":
    # producing
    # giving rise to

    region_top = desc_bottom - dist_between_desc_and_cds
    region_bottom = region_top - cds_height

    draw_cds_invertion_on_ax(
        ax=ax4,
        region=ax4_region,
        cds_df=pd.DataFrame([{'start_pos': 220,
                              'end_pos': 1050,
                              'strand': -1,
                              'product': '',},
                             {'start_pos': 1300,
                              'end_pos': 2080,
                              'strand': 1,
                              'product': '',}]),
        ir_pairs_df=pd.DataFrame([
            {
                'left1': 600,
                'right1': 600 + ax4_repeat1_len,
                'left2': 1470,
                'right2': 1470 + ax4_repeat1_len,
                'color': ir_pair1_color,
            },
        ] + [
            {
                'left1': left1,
                'right1': left1 + ax4_repeat1_len,
                'left2': left2,
                'right2': left2 + ax4_repeat1_len,
                'color': ir_pair1_color,
            } for left1, left2 in (
                (-111111, repeat_leftmost_copy_left),
                (2600, 111111),
                (repeat_rightmost_copy_left, 111111),
            )
        ]),
        cds_arrowhead_width=ax4_cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
        # cds_polygon_linewidth=1,
    )

    region_top = region_bottom + cds_height
    break_height = cds_height * 1.2
    break_bottom = region_bottom - (break_height - cds_height) / 2
    break_top = region_top + (break_height - cds_height) / 2

    break_width = 300
    half_break_width = break_width / 2
    for break_x in (-50, 2350):
        break_bottom_left = (break_x - half_break_width, break_bottom)
        break_bottom_right = (break_x, break_bottom)
        break_top_left = (break_x, break_top)
        break_top_right = (break_x + half_break_width, break_top)

        ax4.plot([break_bottom_left[0], break_top_left[0]], 
                      [break_bottom_left[1], break_top_left[1]], color='black', linewidth=scaffold_break_linewidth)
        ax4.plot([break_bottom_right[0], break_top_right[0]], 
                      [break_bottom_right[1], break_top_right[1]], color='black', linewidth=scaffold_break_linewidth)
        ax4.add_patch(matplotlib.patches.Polygon([break_bottom_left, break_bottom_right, break_top_right, break_top_left], 
                                                  closed=True, fill=True, facecolor='white', edgecolor=None))

    ax4.scatter(ax4_region[0] - 400, region_bottom + cds_height / 2, color='red', marker='x', alpha=redX_alpha, s=redX_size, linewidths=redX_linewidths, clip_on=False)

    set_x_lim_according_to_region(ax4, ax4_region, margin_relative_size=0.03)
    ax4.set_ylim(0, 1)


    fig.add_artist(matplotlib.patches.ConnectionPatch(
        (0.47, 0.05),
        # (0.605, 0.85),
        (0.54, 0.8),
        coordsA=ax4.transAxes,
        coordsB=ax5.transAxes,
        color="black",
        arrowstyle="-|>",
    ))


    ax5_region = (200, 4700)
    ax5_region_middle = np.mean(ax5_region)
    ax5_cds_arrowhead_width = 150
    ax5_repeat1_len = 210

    desc_bottom = 0.45
    ax5.text(ax5_region_middle, desc_bottom, 'cluster\noverlapping PICs', va='bottom', ha='center')
    # forming PIC loci??
    # to form PIC loci??

    region_top = desc_bottom - dist_between_desc_and_cds_with_cluster
    region_bottom = region_top - cds_height

    right_cds_pair_start = 3000
    right_cds_pair_end = 4500
    right_cds_pair_len = 650
    right_cds_pair_ir_pair_pre_repeat_region_len = round(0.5 * right_cds_pair_len)
    right_cds_pair_ir_pair_repeat_len = round(0.25 * right_cds_pair_len)
    right_cds_pair_ir_pair_left1 = right_cds_pair_start + right_cds_pair_ir_pair_pre_repeat_region_len
    right_cds_pair_ir_pair_right2 = right_cds_pair_end - right_cds_pair_ir_pair_pre_repeat_region_len
    
    draw_cds_invertion_on_ax(
        ax=ax5,
        region=ax5_region,
        cds_df=pd.DataFrame([{'start_pos': 400,
                              'end_pos': 1050,
                              'strand': -1,
                              'product': '',},
                             {'start_pos': 1300,
                              'end_pos': 1900,
                              'strand': 1,
                              'product': '',},
                             {'start_pos': 2100,
                              'end_pos': 2600,
                              'strand': 1,
                              'product': '',},
                             {'start_pos': right_cds_pair_start,
                              'end_pos': right_cds_pair_start + right_cds_pair_len,
                              'strand': -1,
                              'product': '',},
                             {'start_pos': right_cds_pair_end - right_cds_pair_len,
                              'end_pos': right_cds_pair_end,
                              'strand': 1,
                              'product': '',},
                            ]),
        ir_pairs_df=pd.DataFrame([
            {
                'left1': 700,
                'right1': 700 + ax5_repeat1_len,
                'left2': 1420,
                'right2': 1420 + ax5_repeat1_len,
                'color': ir_pair2_color,
            },
            {
                'left1': 111111,
                'right1': 111111 + ax5_repeat1_len,
                'left2': 2110,
                'right2': 2110 + ax5_repeat1_len,
                'color': ir_pair2_color,
            },
            {
                'left1': right_cds_pair_ir_pair_left1,
                'right1': right_cds_pair_ir_pair_left1 + right_cds_pair_ir_pair_repeat_len,
                'left2': right_cds_pair_ir_pair_right2 - right_cds_pair_ir_pair_repeat_len,
                'right2': right_cds_pair_ir_pair_right2,
                'color': ir_pair1_color,
            },
        ]),
        cds_arrowhead_width=ax5_cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
        # cds_polygon_linewidth=1,
    )

    cluster_height = cds_height * 1.5
    cluster_top = region_top + (cluster_height - cds_height) / 2
    cluster_bottom = region_bottom - (cluster_height - cds_height) / 2

    for cluster_start, cluster_end in (
        (400, 2600),
        (right_cds_pair_start, right_cds_pair_end),
    ):
        ax5.add_patch(matplotlib.patches.Polygon([(cluster_start, cluster_bottom), 
                                                  (cluster_start, cluster_top), 
                                                  (cluster_end, cluster_top), 
                                                  (cluster_end, cluster_bottom)], 
                                                 closed=True, fill=False, alpha=cluster_alpha, linewidth=cluster_linewidth, linestyle=(0, (1,1))))

    set_x_lim_according_to_region(ax5, ax5_region, margin_relative_size=0.03)
    ax5.set_ylim(0, 1)


    
    
    fig.add_artist(matplotlib.patches.ConnectionPatch(
        (-0.06, 0.5),
        (1.03, 0.5),
        coordsA=ax5.transAxes,
        coordsB=ax6.transAxes,
        color="black",
        arrowstyle="-|>",
    ))




    ax6_region = (200, 2100)
    ax6_region_middle = np.mean(ax6_region)
    ax6_cds_arrowhead_width = 100
    ax6_repeat1_len = 110

    desc_bottom = 0.53
    ax6.text(ax6_region_middle, desc_bottom, 
                   f'PICs with homologous\nloci in same-species\ngenomes ({stage5_num_of_cds_pairs_with_any_homologous_locus_str_repr})', va='bottom', ha='center')

    region_top = desc_bottom - dist_between_desc_and_cds_with_cluster
    region_bottom = region_top - cds_height

    cds1_start = 750
    cds1_end = 1200
    cds2_start = 1350
    cds2_end = 1800
    left1 = cds1_start + 240
    right2 = cds2_end - 240
    draw_cds_invertion_on_ax(
        ax=ax6,
        region=ax6_region,
        cds_df=pd.DataFrame([{'start_pos': cds1_start,
                              'end_pos': cds1_end,
                              'strand': -1,
                              'product': '',},
                             {'start_pos': cds2_start,
                              'end_pos': cds2_end,
                              'strand': 1,
                              'product': '',},
                            ]),
        ir_pairs_df=pd.DataFrame([
            {
                'left1': left1,
                'right1': left1 + ax6_repeat1_len,
                'left2': right2 - ax6_repeat1_len,
                'right2': right2,
                'color': ir_pair1_color,
            },
        ]),
        cds_arrowhead_width=ax6_cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
        # cds_polygon_linewidth=1,
    )

    
    other_nuccore_region_top = region_bottom - vertical_dist_between_region_in_ref_and_other_genome
    other_nuccore_region_bottom = other_nuccore_region_top - cds_height
    draw_cds_invertion_on_ax(
        ax=ax6,
        region=ax6_region,
        region_bottom_and_top=(other_nuccore_region_bottom, other_nuccore_region_top),
        # cds_polygon_linewidth=1,
    )
    
    cluster_top = region_top + (cluster_height - cds_height) / 2
    cluster_bottom = region_bottom - (cluster_height - cds_height) / 2
    ax6.add_patch(matplotlib.patches.Polygon([(cds1_start, cluster_bottom), 
                                              (cds1_start, cluster_top), 
                                              (cds2_end, cluster_top), 
                                              (cds2_end, cluster_bottom)], 
                                             closed=True, fill=False, alpha=cluster_alpha, linewidth=cluster_linewidth, linestyle=(0, (1,1))))

    position_offset = -150
    rectangle_height = cds_height * 1.2
    rectangle_top = other_nuccore_region_top + (rectangle_height - cds_height) / 2
    rectangle_bottom = other_nuccore_region_bottom - (rectangle_height - cds_height) / 2
    rectangle_start = cds1_start + position_offset
    rectangle_end = cds2_end + position_offset
    ax6.add_patch(matplotlib.patches.Polygon([(rectangle_start, rectangle_bottom), 
                                              (rectangle_start, rectangle_top), 
                                              (rectangle_end, rectangle_top), 
                                              (rectangle_end, rectangle_bottom)], 
                                             closed=True, fill=True, facecolor='white', edgecolor='darkslategrey', alpha=1, 
                                             linewidth=question_mark_rectangle_linewidth))
    ax6.text(np.mean([rectangle_start, rectangle_end]), np.mean([rectangle_bottom, rectangle_top]) - 0.009, 
                  '?', va='center', ha='center', fontsize=11)

    margin_size = 200

    ax6.add_patch(matplotlib.patches.Polygon([
        (rectangle_start - margin_size, other_nuccore_region_top), 
        (rectangle_start, other_nuccore_region_top), 
        (cds1_start, region_bottom), 
        (cds1_start - margin_size, region_bottom), 
    ], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
        linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))

    ax6.add_patch(matplotlib.patches.Polygon([
        (rectangle_end, other_nuccore_region_top), 
        (rectangle_end + margin_size, other_nuccore_region_top), 
        (cds2_end + margin_size, region_bottom), 
        (cds2_end, region_bottom), 
    ], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
        linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))

    set_x_lim_according_to_region(ax6, ax6_region, margin_relative_size=0.15)
    ax6.set_ylim(0, 1)



    fig.add_artist(matplotlib.patches.ConnectionPatch(
        (-0.04, 0.5),
        (1.07, 0.5),
        coordsA=ax6.transAxes,
        coordsB=ax7.transAxes,
        color="black",
        arrowstyle="-|>",
    ))




    ax7_region = (270, 2400)
    ax7_region_middle = np.mean(ax7_region)
    ax7_cds_arrowhead_width = 70
    ax7_repeat1_len = 110

    desc_bottom = 0.6
    ax7.text(ax7_region_middle, desc_bottom, 
                   # f'CDS pairs with\nrearrangements ({num_of_cds_pairs_with_high_confidence_ir_pairs_str_repr})', 
                   f'PICs with\nvariation ({num_of_cds_pairs_with_high_confidence_ir_pairs_str_repr})', 
                   # f'variable\nCDS pairs ({num_of_cds_pairs_with_high_confidence_ir_pairs_str_repr})', 
             va='bottom', ha='center')

    region_top = desc_bottom - dist_between_desc_and_cds_with_cluster
    region_bottom = region_top - cds_height

    cds1_start = 650
    cds1_end = 1150
    cds2_start = 1650
    cds2_end = 2150
    left1 = cds1_end - ax7_repeat1_len - 200
    right2 = cds2_start + ax7_repeat1_len + 150

    right1 = left1 + ax7_repeat1_len
    left2 = right2 - ax7_repeat1_len

    spacer_start = left1 + ax7_repeat1_len + 1
    spacer_end = right2 - ax7_repeat1_len - 1
    spacer_size = spacer_end - spacer_start

    intervals = (
        (ax7_region[0], spacer_start - 1),
        (spacer_start, spacer_end),
        (spacer_end + 1, ax7_region[1]),
    )

    position_offset = -110

    left1_in_other_nuccore = left1 + position_offset
    right2_in_other_nuccore = right2 + position_offset

    other_nuccore_region_top = region_bottom - vertical_dist_between_region_in_ref_and_other_genome
    other_nuccore_region_bottom = other_nuccore_region_top - cds_height

    other_nuccore_cds1_start = cds1_start + position_offset
    other_nuccore_cds2_end = cds2_end + position_offset
    other_nuccore_right1 = other_nuccore_cds1_start + (right1 - cds1_start)
    other_nuccore_left2 = other_nuccore_cds2_end - (cds2_end - left2)

    matching_intervals = [(x[0] + position_offset, x[1] + position_offset) for x in intervals]
    
    
    right_inversion_interval_to_matching_interval_and_strand = {
        intervals[0]: (matching_intervals[0], 1),
        intervals[1]: (matching_intervals[1], (1 if same_variant_in_other_genome_in_ax7 else -1)),
        intervals[2]: (matching_intervals[2], 1),
    }

    
    ir_pairs_df = pd.DataFrame([
        {
            'left1': left1,
            'right1': right1,
            'left2': left2,
            'right2': right2,
            'color': ir_pair1_color,
        },
    ])

    
    draw_cds_invertion_on_ax(
        ax=ax7,
        region=ax7_region,
        cds_df=pd.DataFrame([{'start_pos': cds1_start,
                              'end_pos': cds1_end,
                              'strand': -1,
                              'product': '',},
                             {'start_pos': cds2_start,
                              'end_pos': cds2_end,
                              'strand': 1,
                              'product': '',},
                            ]),
        ir_pairs_df=ir_pairs_df,
        region_with_gradient=(spacer_start, spacer_end),
        interval_to_matching_interval_and_strand=right_inversion_interval_to_matching_interval_and_strand,
        cds_arrowhead_width=ax7_cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
        matching_region_bottom_and_top=(other_nuccore_region_bottom, other_nuccore_region_top),
        # draw_matching_region_cds=False,
        # cds_polygon_linewidth=1,
    )
    
    cluster_top = region_top + (cluster_height - cds_height) / 2
    cluster_bottom = region_bottom - (cluster_height - cds_height) / 2
    ax7.add_patch(matplotlib.patches.Polygon([(cds1_start, cluster_bottom), 
                                              (cds1_start, cluster_top), 
                                              (cds2_end, cluster_top), 
                                              (cds2_end, cluster_bottom)], 
                                             closed=True, fill=False, alpha=cluster_alpha, linewidth=cluster_linewidth, linestyle=(0, (1,1))))

    margin_size = 200
    ax7.add_patch(matplotlib.patches.Polygon([
        (other_nuccore_cds1_start - margin_size, other_nuccore_region_top), 
        (other_nuccore_cds1_start, other_nuccore_region_top), 
        (cds1_start, region_bottom), 
        (cds1_start - margin_size, region_bottom), 
    ], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
        linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))

    ax7.add_patch(matplotlib.patches.Polygon([
        (other_nuccore_cds2_end, other_nuccore_region_top), 
        (other_nuccore_cds2_end + margin_size, other_nuccore_region_top), 
        (cds2_end + margin_size, region_bottom), 
        (cds2_end, region_bottom), 
    ], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
        linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))
    
    ax7.add_patch(matplotlib.patches.Polygon([
        (other_nuccore_right1, other_nuccore_region_top), 
        (other_nuccore_left2, other_nuccore_region_top), 
        (right1, region_bottom), 
        (left2, region_bottom), 
    ], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
        linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))


    set_x_lim_according_to_region(ax7, ax7_region, margin_relative_size=0.03)
    ax7.set_ylim(0, 1)


def draw_snp_distance_between_rearranged_loci_histogram(fig, ax):
    distance_column = (1 - filtered_pairs_with_highest_confidence_bps_df.groupby(index_column_names.MERGED_CDS_PAIR_REGION_INDEX_COLUMN_NAMES)[
        'mauve_total_match_proportion_of_region_in_other_nuccore_with_highest_confidence_bps'].max()) * 100
    max_distance = distance_column.max()
    # print(distance_column.describe())

    # bins = list(np.linspace(0, max_distance, 13))
    # bin_size = 0.002 # for fraction
    bin_size = 0.2 # for percentage
    bins = np.arange(0, max_distance + bin_size, bin_size)

    assert distance_column.min() >= bins[0]
    assert distance_column.max() <= bins[-1]
    
    hist_color = (*matplotlib.colors.to_rgb('grey'), 1)
    hist_color = (0.5,0.5,0.5)
    hist_color = 'grey'
    hist_color = GREY_HIST_COLOR
    ax.hist(
        distance_column,
        bins=bins,
        color=hist_color,
        # color='grey',
        # edgecolor=(*matplotlib.colors.to_rgb('grey'), 0.1),
        edgecolor=hist_color,
        # histtype='step', 
        # linewidth=8,
        # alpha=0.75, # ugh. just use the annoying grey and that's it.
    )
    ax.set_yscale('log')

    ax.set_ylabel(
        # '# rearranged loci',
        # '# loci with variation',
        # '# variable loci',
        '# PIC loci',
        # fontsize=9,
    )
    ax.set_xlabel(
        # 'SNP distance between\nrearranged loci (%)', 
        'SNP distance between\nlocus variants (%)', 
        # fontsize=9,
    )
    ax.set_xlim(0, bins[-1])

    ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))

# ir_pair1_color = BLUES_CMAP
# ir_pair1_color = BLUE_PURPLE_CMAP
# ir_pair2_color = 'Reds_r'
# ir_pair2_color = REDS_CMAP
# ir_pair2_color = RED_GOLD_CMAP
# ir_pair2_color = RED_ORANGE_CMAP
# ir_pair2_color = RED_MAGENTA_CMAP

ir_pair1_color = COLORBREWER_SET1_WITHOUT_GREY_AND_YELLOW[1]
ir_pair2_color = COLORBREWER_SET1_WITHOUT_GREY_AND_YELLOW[2]
    
draw_schematic_invertible_cdss_on_fig_and_ax(
    fig1, fig1a_ax, cds_height=0.14, ir_pair_color=ir_pair1_color, 
    draw_alignment_hourglass=True,
)
draw_pipeline_on_fig_and_axes(
    fig1, fig1b1_ax, fig1b2_ax, fig1b3_ax, fig1b4_ax, fig1b5_ax, fig1b6_ax, fig1b7_ax, cds_height=0.12, dist_between_desc_and_cds=0.04,
    ir_pair1_color=ir_pair1_color, ir_pair2_color=ir_pair2_color,
    # same_variant_in_other_genome_in_ax7=True,
)
draw_snp_distance_between_rearranged_loci_histogram(fig1, fig1c_ax)

fig1a_ax.text(0.02, 0.98, 'A', va='center', ha='left', transform=matplotlib.transforms.blended_transform_factory(fig1.transFigure, fig1a_ax.transAxes), fontweight='bold')
fig1b1_ax.text(0.02, 0.98, 'B', va='center', ha='left', transform=matplotlib.transforms.blended_transform_factory(fig1.transFigure, fig1b1_ax.transAxes), fontweight='bold')
fig1c_ax.text(-0.35, 0.98, 'C', va='center', ha='left', transform=fig1c_ax.transAxes, fontweight='bold')

print(f'len(extended_merged_cds_pair_region_df): {len(extended_merged_cds_pair_region_df)}')

if 1:
    fig1.savefig(os.path.join(paper_figs_and_tables_dir_path, 'fig1.png'))
    fig1.savefig(os.path.join(paper_figs_and_tables_dir_path, 'fig1.pdf'))

In [ ]:
column_name_to_critical_val_and_passed_threshold_column_name

In [ ]:
if 'fig2' in locals():
    plt.close(fig2)
# raise    
fig2 = plt.figure(
    figsize=(5.9,8.8), 
    # constrained_layout=True,
)
# ugh. if the height of the histograms is too low, the minor ticks won't be showed 
# (i guess because they are too close to each other, which means forcing matplotlib to show them makes no sense)...
fig2_gridspec = fig2.add_gridspec(
    7, 1, 
    height_ratios=[0.6,0.4,1,0.65,1,0.6,0.8], 
    wspace=0, hspace=0,
)

fig2.subplots_adjust(
    top=0.94,
    bottom=0.06,
    left=0,
    right=0.92,
    # hspace=0.2,
    # wspace=0.2
)

fig2_upper_sub_gridspec = fig2_gridspec[0].subgridspec(1, 3, width_ratios=[0.02,1,0], wspace=0,hspace=0)
fig2_upper_middle_sub_gridspec = fig2_gridspec[2].subgridspec(1, 5, width_ratios=[0.42,1,0.25,1,0.09], wspace=0,hspace=0)
fig2_lower_middle_sub_gridspec = fig2_gridspec[4].subgridspec(1, 5, width_ratios=[0.42,1,0.25,1,0.09], wspace=0,hspace=0)
fig2_lower_sub_gridspec = fig2_gridspec[-1].subgridspec(1, 5, width_ratios=[0.42,1,0.25,1,0.09], wspace=0,hspace=0)

fig2a_ax = fig2.add_subplot(fig2_upper_sub_gridspec[1])
fig2b_ax = fig2.add_subplot(fig2_upper_middle_sub_gridspec[1])
fig2c_ax = fig2.add_subplot(fig2_upper_middle_sub_gridspec[3])
fig2d_ax = fig2.add_subplot(fig2_lower_middle_sub_gridspec[1])
fig2e_ax = fig2.add_subplot(fig2_lower_middle_sub_gridspec[3])
fig2f_ax = fig2.add_subplot(fig2_lower_sub_gridspec[1:-1])


def get_draw_attr_arrow(ax, upper_attr_line_y, upper_attr_text_y, lower_attr_line_y, lower_attr_text_y, attr_arrow_props):
    def draw_attr_arrow(line_xs, text, is_upper):
        if is_upper:
            line_y = upper_attr_line_y
            text_y = upper_attr_text_y
            va = 'bottom'
        else:
            line_y = lower_attr_line_y
            text_y = lower_attr_text_y
            va = 'top'
        
        ax.annotate("", xy=(line_xs[0], line_y), xytext=(line_xs[1], line_y), arrowprops=attr_arrow_props)
        ax.text(np.mean(line_xs), text_y, text, va=va, ha='center')
    return draw_attr_arrow
    
ATTR_ARROW_PROPS = dict(arrowstyle="<->", shrinkA=0, shrinkB=0, linewidth=0.5)

def draw_schematic_invertible_cds_attributes_on_fig_and_ax(fig, ax, cds_height, ir_pair1_color, ir_pair2_color):
    region_top = 0.8
    # dist_between_attr_line_and_cds = 0.1
    dist_between_attr_line_and_cds = cds_height / 0.22 * 0.1
    # dist_between_attr_line_and_text = 0.05
    dist_between_attr_line_and_text = cds_height / 0.22 * 0.05
    dist_between_lower_attr_text_and_formulas = 0.43
    region_left_margin_len_as_fraction_of_region_len = 0.02
    region_right_margin_len_as_fraction_of_region_len = 0.85
    dist_between_region_and_formulas_as_fraction_of_region_len = 0.05
    
    
    
    cds_arrowhead_width = 100
    r1 = 290
    r2 = 350
    
    cds1_len = 1700
    cds2_len = 1500
    cds_distance = 330
    cds1_start = 0
    
    o1 = 550
    o2 = 300
    i1 = 330
    i2 = 350
    
    cds1_end = cds1_start + cds1_len
    
    cds2_start = cds1_end + cds_distance
    cds2_end = cds2_start + cds2_len
    
    region = (cds1_start - 250, cds2_end + 250)
    
    cds1 = {
        'start_pos': cds1_start,
        'end_pos': cds1_end,
        'strand': 1,
        'product': '',
    }
    cds2 = {
        'start_pos': cds2_start,
        'end_pos': cds2_end,
        'strand': -1,
        'product': '',
    }
    
    cds_df = pd.DataFrame([cds1, cds2])

    ir_pair1_left1 = cds1['start_pos'] + o1
    ir_pair1_right2 = cds2['end_pos'] - o2
    ir_pair1 = {
        'left1': ir_pair1_left1,
        'right1': ir_pair1_left1 + r1,
        'left2': ir_pair1_right2 - r1,
        'right2': ir_pair1_right2,
        'color': ir_pair1_color,
    }
    ir_pair2_left2 = cds1['end_pos'] - i1
    ir_pair2_right1 = cds2['start_pos'] + i2
    ir_pair2 = {
        'left1': ir_pair2_left2 - r2,
        'right1': ir_pair2_left2,
        'left2': ir_pair2_right1,
        'right2': ir_pair2_right1 + r2,
        'color': ir_pair2_color,
    }
    
    
    pairs_df = pd.DataFrame([ir_pair1, ir_pair2])

    region_bottom = region_top - cds_height
    draw_cds_invertion_on_ax(
        ax=ax,
        region=region,
        cds_df=cds_df,
        ir_pairs_df=pairs_df,
        cds_arrowhead_width=cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
    )
    
    
    upper_attr_line_y = region_top + dist_between_attr_line_and_cds
    lower_attr_line_y = region_bottom - dist_between_attr_line_and_cds
    upper_attr_text_y = upper_attr_line_y + dist_between_attr_line_and_text
    lower_attr_text_y = lower_attr_line_y - dist_between_attr_line_and_text
    
    
    draw_attr_arrow = get_draw_attr_arrow(ax, upper_attr_line_y, upper_attr_text_y, lower_attr_line_y, lower_attr_text_y, ATTR_ARROW_PROPS)
        
    
    line_xs = [ir_pair1['left1'], ir_pair1['right1']]
    draw_attr_arrow(line_xs, r'$r_1$', is_upper=True)
    
    line_xs = [ir_pair2['left1'], ir_pair2['right1']]
    draw_attr_arrow(line_xs, r'$r_2$', is_upper=True)
    
    
    line_xs = [cds1_start, ir_pair1['left1']]
    draw_attr_arrow(line_xs, r'$u_L$', is_upper=True)
    draw_attr_arrow(line_xs, r'$o_L$', is_upper=False)
    
    line_xs = [ir_pair1['right2'], cds2_end]
    draw_attr_arrow(line_xs, r'$u_R$', is_upper=True)
    draw_attr_arrow(line_xs, r'$o_R$', is_upper=False)
    
    
    
    line_xs = [ir_pair2['right1'], cds1_end]
    draw_attr_arrow(line_xs, r'$i_L$', is_upper=False)
    
    line_xs = [cds2_start, ir_pair2['left2']]
    draw_attr_arrow(line_xs, r'$i_R$', is_upper=False)
    
    line_xs = [cds1_end, cds2_start]
    draw_attr_arrow(line_xs, '$d$', is_upper=True)

    formulas_y = np.mean([region_top, region_bottom])
    
    region_len = region[1] - region[0]
    left_x = region[0] - region_left_margin_len_as_fraction_of_region_len * region_len
    right_x = region[1] + region_right_margin_len_as_fraction_of_region_len * region_len
    formulas_left_x = region[1] + dist_between_region_and_formulas_as_fraction_of_region_len * region_len
    
    formula_vertical_dist_from_region_middle = 0.25
    
    
    ax.text(
        formulas_left_x, 
        formulas_y + 2.2 * formula_vertical_dist_from_region_middle, 
        r'repeat length$=\max(r_1,r_2)$',
        va='center', ha='left',
    )
    
    ax.text(
        formulas_left_x, 
        formulas_y + 0.7 * formula_vertical_dist_from_region_middle, 
        r'CDS distance$=d$', 
        va='center', ha='left',
    )
    
    ax.text(
        formulas_left_x, 
        formulas_y - 3 * formula_vertical_dist_from_region_middle, 
        r'asymmetry$=1-\dfrac{\min(u_L,u_R)}{\max(u_L,u_R)}$', 
        va='center', ha='left',
    )
    
    
    
    ax.text(
        formulas_left_x,
        formulas_y - formula_vertical_dist_from_region_middle, 
        r'$\dfrac{\mathrm{orientation}}{\mathrm{matching}}=\dfrac{o_L+d+o_R}{(o_L+d+o_R)+(i_L+d+i_R)}$',
        va='center', ha='left',
    )
    # an ugly hack to get rid of the fraction line - we just overwrite it with a white line. ugh.
    ax.plot([formulas_left_x, formulas_left_x + region_len * 0.27], 
            [formulas_y - formula_vertical_dist_from_region_middle] * 2 , 
            color=COLOR_OF_HACKY_LINE_TO_OVERWRITE_FRACTION_LINE_OF_ORIENTATION_MATCHING, 
            linewidth=1.8, # 1.8 is fine with 300 dpi.
            zorder=100)
    
    # set_x_lim_according_to_region(ax, region, margin_relative_size=0.03)
    ax.set_xlim(left_x, right_x)
    ax.set_ylim(0, 1)
    
def draw_schematic_orientation_matching_examples_on_fig_and_ax(fig, ax, cds_height, ir_pair1_color, ir_pair2_color, cds_and_repeat_info):
    region_top = 0.88
    # dist_between_attr_line_and_cds = 0.1
    # dist_between_attr_line_and_text = 0.05
    dist_between_attr_line_and_cds = cds_height / 0.22 * 0.1
    dist_between_attr_line_and_text = cds_height / 0.22 * 0.05
    dist_between_lower_attr_text_and_formulas = 0.43
    region_left_margin_len_as_fraction_of_region_len = 0.02
    region_right_margin_len_as_fraction_of_region_len = 0.35
    dist_between_region_and_formulas_as_fraction_of_region_len = 0.05
    
    dist_between_region_bottom_and_opposite_orientation_region_top = 0.5
    
    
    cds_arrowhead_width = cds_and_repeat_info['cds_arrowhead_width']
    r1 = cds_and_repeat_info['r1']
    r2 = cds_and_repeat_info['r2']
    cds1_strand = cds_and_repeat_info['cds1_strand']
    cds1_len = cds_and_repeat_info['cds1_len']
    cds2_len = cds_and_repeat_info['cds2_len']
    cds_distance = cds_and_repeat_info['cds_distance']
    cds1_start = cds_and_repeat_info['cds1_start']
    o1 = cds_and_repeat_info['o1']
    o2 = cds_and_repeat_info['o2']
    i1 = cds_and_repeat_info['i1']
    i2 = cds_and_repeat_info['i2']
    d = cds_distance
    
    cds1_end = cds1_start + cds1_len
    
    cds2_start = cds1_end + cds_distance
    cds2_end = cds2_start + cds2_len
    
    region = (cds1_start - 250, cds2_end + 250)
    
    cds1 = {
        'start_pos': cds1_start,
        'end_pos': cds1_end,
        'strand': cds1_strand,
        'product': '',
    }
    cds2 = {
        'start_pos': cds2_start,
        'end_pos': cds2_end,
        'strand': -cds1_strand,
        'product': '',
    }
    
    cds_df = pd.DataFrame([cds1, cds2])

    ir_pair1_left1 = cds1['start_pos'] + o1
    ir_pair1_right2 = cds2['end_pos'] - o2
    ir_pair1 = {
        'left1': ir_pair1_left1,
        'right1': ir_pair1_left1 + r1,
        'left2': ir_pair1_right2 - r1,
        'right2': ir_pair1_right2,
        'color': ir_pair1_color,
    }
    ir_pair2_left2 = cds1['end_pos'] - i1
    ir_pair2_right1 = cds2['start_pos'] + i2
    ir_pair2 = {
        'left1': ir_pair2_left2 - r2,
        'right1': ir_pair2_left2,
        'left2': ir_pair2_right1,
        'right2': ir_pair2_right1 + r2,
        'color': ir_pair2_color,
    }
    
    
    pairs_df = pd.DataFrame([ir_pair1, ir_pair2])

    region_bottom = region_top - cds_height
    draw_cds_invertion_on_ax(
        ax=ax,
        region=region,
        cds_df=cds_df,
        ir_pairs_df=pairs_df,
        cds_arrowhead_width=cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
    )
    
    opposite_orientation_cds_df = cds_df.copy()
    opposite_orientation_cds_df['strand'] = -opposite_orientation_cds_df['strand']
    opposite_orientation_region_top = region_bottom - dist_between_region_bottom_and_opposite_orientation_region_top
    opposite_orientation_region_bottom = opposite_orientation_region_top - cds_height
    opposite_orientation_pairs_df = pairs_df.copy()
    opposite_orientation_pairs_df['left1'] = cds1_start + cds1_end - pairs_df['right1']
    opposite_orientation_pairs_df['right1'] = cds1_start + cds1_end - pairs_df['left1']
    opposite_orientation_pairs_df['left2'] = cds2_start + cds2_end - pairs_df['right2']
    opposite_orientation_pairs_df['right2'] = cds2_start + cds2_end - pairs_df['left2']
        
    draw_cds_invertion_on_ax(
        ax=ax,
        region=region,
        cds_df=opposite_orientation_cds_df,
        ir_pairs_df=opposite_orientation_pairs_df,
        cds_arrowhead_width=cds_arrowhead_width,
        region_bottom_and_top=(opposite_orientation_region_top, opposite_orientation_region_bottom),
        ir_pairs_pointing_outward=True,
    )
    
    ax.add_patch(matplotlib.patches.Polygon([
        (cds1_start, region_bottom), 
        (cds1_end, region_bottom), 
        (cds1_start, opposite_orientation_region_top), 
        (cds1_end, opposite_orientation_region_top), 
    ], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
        linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))
    ax.add_patch(matplotlib.patches.Polygon([
        (cds2_start, region_bottom), 
        (cds2_end, region_bottom), 
        (cds2_start, opposite_orientation_region_top), 
        (cds2_end, opposite_orientation_region_top), 
    ], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
        linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))
    
    upper_attr_line_y = region_top + dist_between_attr_line_and_cds
    lower_attr_line_y = region_bottom - dist_between_attr_line_and_cds
    upper_attr_text_y = upper_attr_line_y + dist_between_attr_line_and_text
    lower_attr_text_y = lower_attr_line_y - dist_between_attr_line_and_text
    
    
    draw_attr_arrow = get_draw_attr_arrow(ax, upper_attr_line_y, upper_attr_text_y, lower_attr_line_y, lower_attr_text_y, ATTR_ARROW_PROPS)
        
    line_xs = [cds1_end, cds2_start]
    draw_attr_arrow(line_xs, '$d$', is_upper=True)
    
    line_xs = [cds1_start, ir_pair1['left1']]
    draw_attr_arrow(line_xs, r'$o_L$', is_upper=True)
    
    line_xs = [ir_pair1['right2'], cds2_end]
    draw_attr_arrow(line_xs, r'$o_R$', is_upper=True)
    
    
    
    line_xs = [ir_pair2['right1'], cds1_end]
    draw_attr_arrow(line_xs, r'$i_L$', is_upper=True)
    
    line_xs = [cds2_start, ir_pair2['left2']]
    draw_attr_arrow(line_xs, r'$i_R$', is_upper=True)

    formulas_y = np.mean([region_top, region_bottom])
    
    region_len = region[1] - region[0]
    left_x = region[0] - region_left_margin_len_as_fraction_of_region_len * region_len
    right_x = region[1] + region_right_margin_len_as_fraction_of_region_len * region_len
    formulas_left_x = region[1] + dist_between_region_and_formulas_as_fraction_of_region_len * region_len
    
    formula_vertical_dist_from_region_middle = 0.55
    
    orientation_matching = (o1+d+o2) / (o1+d+o2+i1+d+i2)
    ax.text(
        formulas_left_x,
        formulas_y, 
        r'$\dfrac{\mathrm{orientation}}{\mathrm{matching}}=' + f'{orientation_matching:.1f}$',
        va='center', ha='left',
    )
    # an ugly hack to get rid of the fraction line - we just overwrite it with a white line. ugh.
    ax.plot([formulas_left_x, formulas_left_x + region_len * 0.26], 
            [formulas_y] * 2 , 
            color=COLOR_OF_HACKY_LINE_TO_OVERWRITE_FRACTION_LINE_OF_ORIENTATION_MATCHING, 
            linewidth=2.4, # 2.4 is fine with 300 dpi. (lower was ok here but bad in another viewer. ugh)
            zorder=100)
    
    ax.text(
        region[0] - region_len * 0.02,
        formulas_y, 
        'observed\nlocus',
        va='center', ha='right',
    )
    ax.text(
        region[0] - region_len * 0.02,
        np.mean([opposite_orientation_region_top, opposite_orientation_region_bottom]), 
        'hypothetical\nopposite\norientation locus',
        va='center', ha='right',
    )
    # set_x_lim_according_to_region(ax, region, margin_relative_size=0.03)
    ax.set_xlim(left_x, right_x)
    ax.set_ylim(0, 1)
    
def draw_schematic_asymmetry_examples_on_fig_and_ax(fig, ax, cds_height, ir_pair1_color, ir_pair2_color, cds_and_repeat_info):
    region_top = 0.8
    # dist_between_attr_line_and_cds = 0.1
    # dist_between_attr_line_and_text = 0.05
    dist_between_attr_line_and_cds = cds_height / 0.22 * 0.1
    dist_between_attr_line_and_text = cds_height / 0.22 * 0.05
    dist_between_lower_attr_text_and_formulas = 0.43
    region_left_margin_len_as_fraction_of_region_len = 0.02
    region_right_margin_len_as_fraction_of_region_len = 0.35
    dist_between_region_and_formulas_as_fraction_of_region_len = 0.05
    
    dist_between_region_bottom_and_opposite_orientation_region_top = 0.5
    
    
    cds_arrowhead_width = cds_and_repeat_info['cds_arrowhead_width']
    r1 = cds_and_repeat_info['r1']
    r2 = cds_and_repeat_info['r2']
    cds1_strand = cds_and_repeat_info['cds1_strand']
    cds1_len = cds_and_repeat_info['cds1_len']
    cds2_len = cds_and_repeat_info['cds2_len']
    cds_distance = cds_and_repeat_info['cds_distance']
    cds1_start = cds_and_repeat_info['cds1_start']
    o1 = cds_and_repeat_info['o1']
    o2 = cds_and_repeat_info['o2']
    i1 = cds_and_repeat_info['i1']
    i2 = cds_and_repeat_info['i2']
    
    if cds1_strand == 1:
        u1, u2 = o1, o2
    else:
        u1, u2 = i1, i2
    
    cds1_end = cds1_start + cds1_len
    
    cds2_start = cds1_end + cds_distance
    cds2_end = cds2_start + cds2_len
    
    region = (cds1_start - 250, cds2_end + 250)
    
    cds1 = {
        'start_pos': cds1_start,
        'end_pos': cds1_end,
        'strand': cds1_strand,
        'product': '',
    }
    cds2 = {
        'start_pos': cds2_start,
        'end_pos': cds2_end,
        'strand': -cds1_strand,
        'product': '',
    }
    
    cds_df = pd.DataFrame([cds1, cds2])

    ir_pair1_left1 = cds1['start_pos'] + o1
    ir_pair1_right2 = cds2['end_pos'] - o2
    ir_pair1 = {
        'left1': ir_pair1_left1,
        'right1': ir_pair1_left1 + r1,
        'left2': ir_pair1_right2 - r1,
        'right2': ir_pair1_right2,
        'color': ir_pair1_color,
    }
    ir_pair2_left2 = cds1['end_pos'] - i1
    ir_pair2_right1 = cds2['start_pos'] + i2
    ir_pair2 = {
        'left1': ir_pair2_left2 - r2,
        'right1': ir_pair2_left2,
        'left2': ir_pair2_right1,
        'right2': ir_pair2_right1 + r2,
        'color': ir_pair2_color,
    }
    
    
    pairs_df = pd.DataFrame([ir_pair1, ir_pair2])

    region_bottom = region_top - cds_height
    draw_cds_invertion_on_ax(
        ax=ax,
        region=region,
        cds_df=cds_df,
        ir_pairs_df=pairs_df,
        cds_arrowhead_width=cds_arrowhead_width,
        region_bottom_and_top=(region_bottom, region_top),
    )
    
    upper_attr_line_y = region_top + dist_between_attr_line_and_cds
    lower_attr_line_y = region_bottom - dist_between_attr_line_and_cds
    upper_attr_text_y = upper_attr_line_y + dist_between_attr_line_and_text
    lower_attr_text_y = lower_attr_line_y - dist_between_attr_line_and_text
    
    
    draw_attr_arrow = get_draw_attr_arrow(ax, upper_attr_line_y, upper_attr_text_y, lower_attr_line_y, lower_attr_text_y, ATTR_ARROW_PROPS)
        
    if cds1_strand == 1:
        line_xs = [cds1_start, ir_pair1['left1']]
    else:
        line_xs = [ir_pair2['right1'], cds1_end]
    draw_attr_arrow(line_xs, r'$u_L$', is_upper=False)
    
    if cds1_strand == 1:
        line_xs = [ir_pair1['right2'], cds2_end]
    else:
        line_xs = [cds2_start, ir_pair2['left2']]
    draw_attr_arrow(line_xs, r'$u_R$', is_upper=False)
    
    
    formulas_y = np.mean([region_top, region_bottom])
    
    region_len = region[1] - region[0]
    left_x = region[0] - region_left_margin_len_as_fraction_of_region_len * region_len
    right_x = region[1] + region_right_margin_len_as_fraction_of_region_len * region_len
    formulas_left_x = region[1] + dist_between_region_and_formulas_as_fraction_of_region_len * region_len
    
    formula_vertical_dist_from_region_middle = 0.55
    
    asymmetry = 1 - min(u1, u2) / max(u1, u2)
    ax.text(
        formulas_left_x,
        formulas_y, 
        r'$\mathrm{asymmetry}=' + f'{asymmetry:.2f}$',
        va='center', ha='left',
    )
    
    # set_x_lim_according_to_region(ax, region, margin_relative_size=0.03)
    ax.set_xlim(left_x, right_x)
    ax.set_ylim(0, 1)

def plot_cum_hist_on_ax(ax, curr_df, column_name, bool_column_name, expected_critical_x, 
                        outermost_bin_edges=None, pvalue_on_top_left=False):
    assert not curr_df[column_name].isna().any()
    
    column_min = curr_df[column_name].min()
    column_max = curr_df[column_name].max()

    
    bins = list(np.sort(curr_df[column_name].unique()))
    
    if outermost_bin_edges:
        # this allows hiding the annoying vertical lines ax.hist draws, and also showing the ticks on the edges (due to ax.set_xlim later)
        assert outermost_bin_edges[0] <= column_min
        assert outermost_bin_edges[1] >= column_max
        if outermost_bin_edges[0] < bins[0]:
            bins = [outermost_bin_edges[0]] + bins
        if outermost_bin_edges[1] > bins[-1]:
            bins.append(outermost_bin_edges[1])
    
    hist_ys = []
    print(curr_df[bool_column_name].value_counts())
    curr_df_grouped = curr_df.groupby(bool_column_name)
    for high_confidence, group_df in curr_df_grouped:
        hist_res = ax.hist(
            group_df[column_name],
            bins=bins,
            density=True,
            cumulative=True,
            histtype='step', 
            # linewidth=8,
            # alpha=0.9,
            label=ATTR_NAME_TO_VAL_TO_LABEL[bool_column_name][high_confidence],
            color=ATTR_NAME_TO_VAL_TO_COLOR[bool_column_name][high_confidence],
        )
        hist_ys.append(hist_res[0])
        
    hist1_ys, hist2_ys = hist_ys
    x_with_highest_hist_dif = bins[np.argmax(np.abs(hist1_ys - hist2_ys))]
    # assert np.isclose(x_with_highest_hist_dif, expected_critical_x)
    assert x_with_highest_hist_dif == expected_critical_x
    
    _, pvalue = scipy.stats.ks_2samp(
        *[x[column_name] for _, x in curr_df_grouped],
        alternative='two-sided',
        
        mode='asymp', 
        # mode='exact', 
    )
    print(f'KS pvalue: {pvalue}')
    
    u_statistic, pvalue = scipy.stats.mannwhitneyu(
        *[x[column_name] for _, x in curr_df_grouped],
        alternative='two-sided',
    )
    assert (~curr_df[bool_column_name]).sum() == len(curr_df) - curr_df[bool_column_name].sum()
    u_statistic_for_identical_distributions = curr_df[bool_column_name].sum() * (~curr_df[bool_column_name]).sum() / 2
    print(f'MW U statistic: {u_statistic} (u_statistic_for_identical_distributions: {u_statistic_for_identical_distributions})')
    
    
    if pvalue == 0:
        pvalue_repr = '0'
    else:
        log10_pval = np.log10(pvalue)
        if log10_pval < -10:
            assert int(log10_pval) != log10_pval
            log10_pval_ceil = np.ceil(log10_pval)
            pvalue_repr = get_pvalue_repr(10 ** log10_pval_ceil, verbose=False)
            pvalue_repr = f'P<{pvalue_repr}'
        else:
            pvalue_repr = get_pvalue_repr(pvalue, verbose=False)
            pvalue_repr = f'P={pvalue_repr}'
    if pvalue_on_top_left:
        ax.text(0.025, 0.97, f'{pvalue_repr}', va='top', ha='left', transform=ax.transAxes)
    else:
        ax.text(0.972, 0.01, f'{pvalue_repr}', va='bottom', ha='right', transform=ax.transAxes)
    
    ax.set_ylabel(
        # 'CDS pair\ncumulative fraction', 
        'PIC cumulative fraction', 
        # labelpad=5,
    )

    ylim = (0, 1.011)
    ax.set_xlim(bins[0], bins[-1])

    ax.plot([x_with_highest_hist_dif] * 2, ylim,
            linestyle='--',color='red',
            alpha=THRESHOLD_DASHED_LINE_ALPHA)
    # ax.text(x_with_highest_hist_dif, 1.01, critical_x_name_repr, va='bottom', ha='center', clip_on=False)
    ax.set_ylim(*ylim)
    ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
    ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
    
def plot_cds_spacer_len_cum_histograms_on_ax(fig, ax, df, bool_column_name):
    # curr_df = cds_pairs_df.copy()
    curr_df = df.copy()

    column_name = 'operon_spacer_len'
    curr_df[column_name] = curr_df[column_name] / 1000.0
    
    plot_cum_hist_on_ax(ax, curr_df, column_name, bool_column_name, 
                        expected_critical_x=column_name_to_critical_val_and_passed_threshold_column_name[column_name][0] / 1000,
                        outermost_bin_edges=(curr_df[column_name].min(),15.02))
    
    ax.set_xlabel(
        'CDS distance (kbp)', 
        # labelpad=7, # ugh. seems like this makes things better only for 100 dpi???
    )
    

    
def plot_max_repeat_len_cum_histograms_on_ax(fig, ax, df, bool_column_name):
    curr_df = df.copy()
    
    column_name = 'max_repeat_len'
    
    
    
    plot_cum_hist_on_ax(ax, curr_df, column_name, bool_column_name,
                        expected_critical_x=column_name_to_critical_val_and_passed_threshold_column_name[column_name][0])
    
    ax.set_xscale('log')
    ax.set_xlabel(
        'repeat length (bp)', 
        # labelpad=7,
    )
    
    orig_xlim = ax.get_xlim()
    ax.set_xlim(orig_xlim[0] - 1, orig_xlim[-1])
    

def plot_asymmetry_cum_histograms_on_ax(fig, ax, df, bool_column_name):
    curr_df = df.copy()

    column_name = 'operon_asymmetry'


    plot_cum_hist_on_ax(ax, curr_df, column_name, bool_column_name, 
                        expected_critical_x=column_name_to_critical_val_and_passed_threshold_column_name[column_name][0],
                       outermost_bin_edges=(0,1.009), pvalue_on_top_left=True)
    
    ax.set_xlabel(
        'asymmetry', 
        # labelpad=7, # ugh. seems like this makes things better only for 100 dpi???
    )

    
def plot_repeat_orientation_matching_cum_histograms_on_ax(fig, ax, df, bool_column_name):
    curr_df = df.copy()

    column_name = 'operon_closest_repeat_position_orientation_matching'


    plot_cum_hist_on_ax(ax, curr_df, column_name, bool_column_name, 
                        expected_critical_x=column_name_to_critical_val_and_passed_threshold_column_name[column_name][0],
                       outermost_bin_edges=(0,1.009), pvalue_on_top_left=True)
    
    ax.set_xlabel(
        'orientation matching', 
        # labelpad=7, # ugh. seems like this makes things better only for 100 dpi???
    )


def draw_logistic_regression_coefficients(fig, ax, logistic_regression_fit_result_df):
    curr_df = logistic_regression_fit_result_df.copy()
    curr_df['adjusted_coef_minus_its_std_err'] = curr_df['adjusted_coef'] - curr_df['adjusted_std err']
    
    bars_alpha = 0.85
    error_bar_alpha = 0.85
    error_bar_capsize = 2.5
    
    curr_df.sort_values(['adjusted_coef_minus_its_std_err'], inplace=True, ascending=False)
    
    num_of_predictors = len(curr_df)

    xs = np.arange(1, num_of_predictors + 1)
    width = 0.3  # the width of the bars
    adjusted_bars = ax.bar(xs - width/2, curr_df['adjusted_coef'], width, yerr=curr_df['adjusted_std err'], 
                           label='adjusted', 
                           color='grey',
                           alpha=bars_alpha,
                           capsize=error_bar_capsize, error_kw={'alpha': error_bar_alpha})
    unadjusted_bars = ax.bar(xs + width/2, curr_df['coef'], width, yerr=curr_df['std err'], 
                             label='unadjusted', 
                             color='grey', 
                             # edgecolor='black', 
                             # hatch='..', 
                             alpha=(bars_alpha - 0.4),
                             capsize=error_bar_capsize, error_kw={'alpha': error_bar_alpha})
    
    
    ax.set_ylabel(
        'logistic regression\ncoefficient',
        # fontsize='small',
    )
    ax.set_xticks(xs)
    ax.set_xticklabels(
        [LOGISTIC_REGRESSION_COEFFICIENT_BOOL_COLUMN_NAME_TO_LABEL[x] if x in LOGISTIC_REGRESSION_COEFFICIENT_BOOL_COLUMN_NAME_TO_LABEL else x for x in curr_df['bool_column_name']],
        # rotation=45, ha='right', rotation_mode='anchor', 
        # fontsize='small',
    )

    legend = ax.legend(fontsize='small')
    
if logistic_regression_fit_result_df is not None:
    draw_logistic_regression_coefficients(
        fig2, fig2f_ax, 
        logistic_regression_fit_result_df, 
    )
    
ir_pair1_color = COLORBREWER_SET1_WITHOUT_GREY_AND_YELLOW[1]
ir_pair2_color = COLORBREWER_SET1_WITHOUT_GREY_AND_YELLOW[2]
bool_column_name = 'any_high_confidence_ir_pair_linked_to_cds_pair'
df_for_cum_hists = extended1_cds_pairs_for_enrichment_analysis_and_logistic_regression_df
draw_schematic_invertible_cds_attributes_on_fig_and_ax(fig2, fig2a_ax, cds_height=0.22, ir_pair1_color=ir_pair1_color, 
                                                       ir_pair2_color=ir_pair2_color)
plot_max_repeat_len_cum_histograms_on_ax(fig2, fig2b_ax, df_for_cum_hists, bool_column_name)
plot_cds_spacer_len_cum_histograms_on_ax(fig2, fig2c_ax, df_for_cum_hists, bool_column_name)
plot_repeat_orientation_matching_cum_histograms_on_ax(fig2, fig2d_ax, df_for_cum_hists, bool_column_name)
plot_asymmetry_cum_histograms_on_ax(fig2, fig2e_ax, df_for_cum_hists, bool_column_name)

legend_elements = [
    matplotlib.lines.Line2D([0], [0], color=ATTR_NAME_TO_VAL_TO_COLOR['any_high_confidence_ir_pair_linked_to_cds_pair'][high_confidence], 
           label=ATTR_NAME_TO_VAL_TO_LABEL['any_high_confidence_ir_pair_linked_to_cds_pair'][high_confidence]) 
    for high_confidence in (True, False)
]
fig2b_ax.legend(
    handles=legend_elements,
    fontsize='small', 
    bbox_transform=fig2b_ax.transAxes,
    # bbox_transform=fig2e_ax.transAxes,
    
    # bbox_to_anchor=(0, 1.2),
    # loc='lower left',
    
    bbox_to_anchor=(
        1.04, 
        # 1.04,
        1,
    ),
    loc='lower right',
)

for ax in (fig2c_ax, fig2e_ax):
    ax.axes.get_yaxis().get_label().set_visible(False)
    # ax.set_yticks([])
    ax.set_yticklabels([])
    
    
if 1:
    fig2a_ax.text(0.03, 0.975, 'A', va='center', ha='center', transform=fig2.transFigure, fontweight='bold')
    fig2b_ax.text(0.03, 1.15, 'B', va='center', ha='center', fontweight='bold',
                      transform=matplotlib.transforms.blended_transform_factory(fig2.transFigure, fig2b_ax.transAxes))
    fig2c_ax.text(-0.07, 1.15, 'C', va='center', ha='center', fontweight='bold', transform=fig2c_ax.transAxes)
                      # transform=matplotlib.transforms.blended_transform_factory(fig2c_ax.transAxes, fig2.transFigure))
    fig2d_ax.text(0.03, 1.15, 'D', va='center', ha='center', fontweight='bold',
                      transform=matplotlib.transforms.blended_transform_factory(fig2.transFigure, fig2d_ax.transAxes))
    fig2e_ax.text(-0.07, 1.15, 'E', va='center', ha='center', fontweight='bold', transform=fig2e_ax.transAxes)
    fig2f_ax.text(0.03, 1.15, 'F', va='center', ha='center', fontweight='bold',
                      transform=matplotlib.transforms.blended_transform_factory(fig2.transFigure, fig2f_ax.transAxes))

        
if 1:
    if 'fig_s6' in locals():
        plt.close(fig_s6)
    fig_s6 = plt.figure(
        figsize=(6,4.5), 
    )
    fig_s6_gridspec = fig_s6.add_gridspec(
        3, 1, 
        height_ratios=[1,0.4,1], 
        wspace=0, hspace=0,
    )

    fig_s6.subplots_adjust(
        top=0.94,
        bottom=0.03,
        left=0.23,
        right=0.91,
        # hspace=0.2,
        # wspace=0.2
    )

    fig_s6a_ax = fig_s6.add_subplot(fig_s6_gridspec[0])
    fig_s6b_ax = fig_s6.add_subplot(fig_s6_gridspec[2])

    cds_and_repeat_info = {
        'cds_arrowhead_width': 100,
        'r1': 200,
        'r2': 200,
        'cds1_len': 1700,
        'cds2_len': 1500,
        'cds1_strand': -1,
        'cds_distance': 250,
        'cds1_start': 0,
        'o1': 900,
        'o2': 750,
        'i1': 285,
        'i2': 270,
    }
    cds_height = 0.14
    
    draw_schematic_orientation_matching_examples_on_fig_and_ax(
        fig_s6, fig_s6a_ax, 
        cds_height=cds_height, 
        ir_pair1_color=ir_pair1_color, ir_pair2_color=ir_pair2_color,
        cds_and_repeat_info=cds_and_repeat_info,
    )
    
    cds_and_repeat_info['cds1_strand'] = -cds_and_repeat_info['cds1_strand']
    draw_schematic_orientation_matching_examples_on_fig_and_ax(
        fig_s6, fig_s6b_ax, 
        cds_height=cds_height, 
        ir_pair1_color=ir_pair1_color, ir_pair2_color=ir_pair2_color,
        cds_and_repeat_info=cds_and_repeat_info,
    )
    
    fig_s6a_ax.text(-0.28, 1.07, 'A', va='center', ha='center', transform=fig_s6a_ax.transAxes, fontweight='bold')
    fig_s6b_ax.text(-0.28, 1.07, 'B', va='center', ha='center', transform=fig_s6b_ax.transAxes, fontweight='bold')

if 1:
    if 'fig_s7' in locals():
        plt.close(fig_s7)
    fig_s7 = plt.figure(
        figsize=(6,2.5), 
    )
    fig_s7_gridspec = fig_s7.add_gridspec(
        3, 1, 
        height_ratios=[1,0.25,1], 
        wspace=0, hspace=0,
    )

    fig_s7.subplots_adjust(
        top=0.99,
        bottom=0,
        left=0.02,
        right=0.97,
        # hspace=0.2,
        # wspace=0.2
    )

    fig_s7a_ax = fig_s7.add_subplot(fig_s7_gridspec[0])
    fig_s7b_ax = fig_s7.add_subplot(fig_s7_gridspec[2])

    cds_and_repeat_info = {
        'cds_arrowhead_width': 100,
        'r1': 200,
        'r2': 200,
        'cds1_len': 1700,
        'cds2_len': 1050,
        'cds1_strand': -1,
        'cds_distance': 350,
        'cds1_start': 0,
        'o1': 290,
        'o2': 330,
        'i1': 900,
        'i2': 200,
    }
    cds_height = 0.225
    
    draw_schematic_asymmetry_examples_on_fig_and_ax(
        fig_s7, fig_s7a_ax, 
        cds_height=cds_height, 
        ir_pair1_color=ir_pair1_color, ir_pair2_color=ir_pair2_color,
        cds_and_repeat_info=cds_and_repeat_info,
    )
    
    cds_and_repeat_info['cds1_strand'] = -cds_and_repeat_info['cds1_strand']
    draw_schematic_asymmetry_examples_on_fig_and_ax(
        fig_s7, fig_s7b_ax, 
        cds_height=cds_height, 
        ir_pair1_color=ir_pair1_color, ir_pair2_color=ir_pair2_color,
        cds_and_repeat_info=cds_and_repeat_info,
    )
    
    fig_s7a_ax.text(-0.03, 1.03, 'A', va='center', ha='center', transform=fig_s7a_ax.transAxes, fontweight='bold')
    fig_s7b_ax.text(-0.03, 1.03, 'B', va='center', ha='center', transform=fig_s7b_ax.transAxes, fontweight='bold')
    
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html says: 
# The ‘two-sided’ ‘exact’ computation computes the complementary probability and then subtracts from 1. 
# As such, the minimum probability it can return is about 1e-16. (in the end i only used asymp, to avoid pvalue=0)

# according to https://stackoverflow.com/questions/37647396/statsmodels-logistic-regression-odds-ratio/47740828#47740828 (and also 
# according to https://en.wikipedia.org/wiki/Logistic_regression#Logistic_function,_odds,_odds_ratio,_and_logit, IIUC, it seems that 
# the odds ratio == exp(coefficient)

if 1:
    fig2.savefig(os.path.join(paper_figs_and_tables_dir_path, 'fig2.png'))
    fig2.savefig(os.path.join(paper_figs_and_tables_dir_path, 'fig2.pdf'))
    fig_s6.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s6.png'))
    # fig_s6.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s6.pdf'))
    fig_s7.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s7.png'))
    # fig_s7.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s7.pdf'))


In [ ]:
if 'fig3' in locals():
    plt.close(fig3)
# raise    
fig3 = plt.figure(
    figsize=(6.5,9), 
    # constrained_layout=True,
)
fig3_gridspec = fig3.add_gridspec(
    3, 1, 
    height_ratios=[1,0.3,1], 
    wspace=0, hspace=0,
)

fig3.subplots_adjust(
    top=0.97,
    bottom=0.08,
    left=0.125,
    # right=0.99,
    right=0.805,
    # hspace=0.2,
    # wspace=0.2
)

fig3_uppper_sub_gridspec = fig3_gridspec[0].subgridspec(1, 3, width_ratios=[0.0002, 1,0.05], wspace=0, hspace=0)
fig3_lower_sub_gridspec = fig3_gridspec[2].subgridspec(1, 3, width_ratios=[0.0002, 1,0.05], wspace=0, hspace=0)

fig3a_ax = fig3.add_subplot(fig3_uppper_sub_gridspec[1])
fig3b_ax = fig3.add_subplot(fig3_lower_sub_gridspec[1])

def draw_volcano_plot_on_ax(fig, ax, test_results_df, enrichment_desc_for_xlabel, 
                            jitter_sd_as_fraction_of_ax_limit_range=0, corrected_pvalue_threshold=0):
    curr_df = test_results_df.copy()
    
    any_corrected_pvalue_below_threshold = curr_df['corrected_pvalue'].min() <= corrected_pvalue_threshold
    curr_df.loc[curr_df['corrected_pvalue'] <= corrected_pvalue_threshold, 'corrected_pvalue'] = corrected_pvalue_threshold
    
    
    curr_df = add_product_class_column(curr_df, 'product', 'product_class')
    
    any_zero_pval = curr_df['corrected_pvalue'].min() == 0
    if any_zero_pval:
        min_non_zero_pval = generic_utils.get_min_strictly_positive(curr_df['corrected_pvalue'])
        pval_fake_min = min_non_zero_pval / 100
        curr_df.loc[curr_df['corrected_pvalue'] == 0, 'corrected_pvalue'] = pval_fake_min
        threshold_for_pval_fake_min = (min_non_zero_pval + pval_fake_min) / 2

    any_inf_odds_ratio = np.isposinf(curr_df['odds_ratio']).any()
    if any_inf_odds_ratio:
        max_finite_odds_ratio = curr_df[np.isfinite(curr_df['odds_ratio'])]['odds_ratio'].max()
        fake_inf_odds_ratio = max_finite_odds_ratio * 1.2
        curr_df.loc[np.isposinf(curr_df['odds_ratio']), 'odds_ratio'] = fake_inf_odds_ratio
    curr_df = curr_df[curr_df['odds_ratio'] > 0]
    
    curr_df['log_odds_ratio'] = np.log2(curr_df['odds_ratio'])
    assert not (curr_df['corrected_pvalue'] == 0).any()
    curr_df['minus_log_corrected_pvalue'] = -np.log10(curr_df['corrected_pvalue'])

    xlim = ax.get_xlim()
    x_jitter_sd = jitter_sd_as_fraction_of_ax_limit_range * (xlim[1] - xlim[0])
    ylim = ax.get_ylim()
    y_jitter_sd = jitter_sd_as_fraction_of_ax_limit_range * (ylim[1] - ylim[0])
    
    num_of_data_points = len(curr_df)
    curr_df['log_odds_ratio'] += np.random.normal(scale=x_jitter_sd, size=num_of_data_points)
    curr_df['minus_log_corrected_pvalue'] += np.random.normal(scale=y_jitter_sd, size=num_of_data_points)
    
    curr_df_grouped = curr_df.groupby('product_class')
    product_classes = set(curr_df['product_class'])
    product_class_to_scatter_result = {}
    assert set(curr_df['product_class']) <= set(PRODUCT_CLASSES_ORDERED)
    for product_class in PRODUCT_CLASSES_ORDERED:
        if product_class in product_classes:
            group_df = curr_df_grouped.get_group(product_class)
            product_class_to_scatter_result[product_class] = ax.scatter(
                group_df['log_odds_ratio'],
                group_df['minus_log_corrected_pvalue'],
                # alpha=HISTOGRAM_WITH_OVERLAP_ALPHA,
                # alpha=0.5,
                # edgecolor='black',
                edgecolor=VOLCANO_PLOT_EDGE_COLOR,
                linewidth=VOLCANO_PLOT_LINEWIDTH,
                label=product_class,
                # color=PRODUCT_CLASS_TO_COLOR[product_class],
                color=('none' if (PRODUCT_CLASS_TO_COLOR[product_class] == 'white') else PRODUCT_CLASS_TO_COLOR[product_class]),
            )
    

    # ax.set_xlabel('enrichment of annotation near rearranged CDSs ($\log_2$(odds ratio))')
    ax.set_xlabel(f'{ODDS_RATIO_DESCRIPTION_START} {enrichment_desc_for_xlabel}\n($\log_2$(odds ratio))')
    ax.set_ylabel('$-\log_{10}$(corrected p-value)')

    if any_inf_odds_ratio:
        log_fake_inf_odds_ratio = np.log2(fake_inf_odds_ratio)
        non_inf_log_odds_ratios = curr_df[curr_df['log_odds_ratio'] < log_fake_inf_odds_ratio]['log_odds_ratio']
        dotted_line_log_odds_ratio = log_fake_inf_odds_ratio - 0.11
        assert (non_inf_log_odds_ratios < dotted_line_log_odds_ratio).all()
        orig_ylim = ax.get_ylim()
        ax.plot([dotted_line_log_odds_ratio] * 2, orig_ylim,
                linestyle=FAKE_VAL_THRESHOLD_LINE_STYLE,
                color=FAKE_VAL_THRESHOLD_LINE_COLOR,
                # label=f'x=0',
                alpha=FAKE_VAL_THRESHOLD_LINE_ALPHA)
        ax.set_ylim(orig_ylim)
        
    
    if any_corrected_pvalue_below_threshold:
        minus_log10_corrected_pvalue_threshold = -np.log10(corrected_pvalue_threshold)
        assert minus_log10_corrected_pvalue_threshold == int(minus_log10_corrected_pvalue_threshold)
        minus_log10_corrected_pvalue_threshold = int(minus_log10_corrected_pvalue_threshold)
        
        below_thresh_minus_log10_corrected_pvalues = curr_df[curr_df['minus_log_corrected_pvalue'] < minus_log10_corrected_pvalue_threshold]['minus_log_corrected_pvalue']
        dotted_line_minus_log10_corrected_pvalue = minus_log10_corrected_pvalue_threshold - 0.6
        assert (below_thresh_minus_log10_corrected_pvalues < dotted_line_minus_log10_corrected_pvalue).all()
        
        orig_xlim = ax.get_xlim()
        # ax.plot([0.075] * 2, orig_xlim,
        ax.plot(orig_xlim, [dotted_line_minus_log10_corrected_pvalue] * 2,
                linestyle=FAKE_VAL_THRESHOLD_LINE_STYLE,
                color=FAKE_VAL_THRESHOLD_LINE_COLOR,
                # label=f'x=0',
                alpha=FAKE_VAL_THRESHOLD_LINE_ALPHA)
        ax.set_xlim(orig_xlim)
        
        
    if any_corrected_pvalue_below_threshold:
        orig_y_tick_positions = ax.get_yticks()

        minus_log10_corrected_pvalue_threshold_tick_label = f'$\geqslant${minus_log10_corrected_pvalue_threshold}'
        new_y_tick_positions = [y for y in orig_y_tick_positions if y < minus_log10_corrected_pvalue_threshold] + [minus_log10_corrected_pvalue_threshold]
        ax.set_yticks(new_y_tick_positions)
        
        new_y_tick_labels = [remove_trailing_zeros(str(x)) for x in new_y_tick_positions[:-1]] + [minus_log10_corrected_pvalue_threshold_tick_label]
        
        ax.set_yticklabels(new_y_tick_labels)
        
    if any_inf_odds_ratio:
        orig_x_tick_positions = ax.get_xticks()

        inf_tick_label = r'$\inf$' # r'$\infty$' doesnt look good...
        new_x_tick_positions = [x for x in orig_x_tick_positions if x < log_fake_inf_odds_ratio] + [log_fake_inf_odds_ratio]
        ax.set_xticks(new_x_tick_positions)
        
        new_x_tick_labels = [remove_trailing_zeros(str(x)) for x in new_x_tick_positions[:-1]] + [inf_tick_label]
        
        ax.set_xticklabels(new_x_tick_labels)
        
    margin_size = 0.16
    ax.set_xlim(curr_df['log_odds_ratio'].min() - margin_size, curr_df['log_odds_ratio'].max() + margin_size)
            
            
    new_bottom_ylim = -5
    assert curr_df['minus_log_corrected_pvalue'].min() >= new_bottom_ylim
    ax.set_ylim(new_bottom_ylim, ax.get_ylim()[1])
            
    return product_class_to_scatter_result
    

corrected_pvalue_threshold = 1e-30
target_product_class_to_scatter_result = draw_volcano_plot_on_ax(
    fig3, fig3a_ax, 
    test_results_df=repeat_cds_product_fisher_or_g_result_df,
    enrichment_desc_for_xlabel=TARGET_PRODUCT_ODDS_RATIO_DESCRIPTION_END,
    # jitter_sd_as_fraction_of_ax_limit_range=0.08,
    jitter_sd_as_fraction_of_ax_limit_range=0,
    corrected_pvalue_threshold=corrected_pvalue_threshold,
)

neighbor_product_class_to_scatter_result = draw_volcano_plot_on_ax(
    fig3, fig3b_ax, 
    test_results_df=adjacent_repeat_cds_product_fisher_or_g_result_df,
    enrichment_desc_for_xlabel=NEIGHBOR_PRODUCT_ODDS_RATIO_DESCRIPTION_END,
    # jitter_sd_as_fraction_of_ax_limit_range=0.08,
    jitter_sd_as_fraction_of_ax_limit_range=0,
    corrected_pvalue_threshold=corrected_pvalue_threshold,
)

relevant_product_classes_and_scatter_results_ordered = []
for product_class in PRODUCT_CLASSES_ORDERED:
    if product_class in target_product_class_to_scatter_result:
        relevant_product_classes_and_scatter_results_ordered.append((product_class, target_product_class_to_scatter_result[product_class]))
    elif product_class in neighbor_product_class_to_scatter_result:
        relevant_product_classes_and_scatter_results_ordered.append((product_class, neighbor_product_class_to_scatter_result[product_class]))
        
fig3a_ax.legend(
    [x[1] for x in relevant_product_classes_and_scatter_results_ordered], 
    [x[0] for x in relevant_product_classes_and_scatter_results_ordered], 
    # bbox_transform=matplotlib.transforms.blended_transform_factory(fig3.transFigure, fig3a_ax.transAxes),
    bbox_transform=fig3a_ax.transAxes,
    bbox_to_anchor=(0.995, 1.03),
    # bbox_to_anchor=(1, 0.9),
    loc='upper left',
    # loc='upper right',
    # fontsize='x-small', 
    fontsize='small', 
)

if 1:
    fig3a_ax.text(0.03, 1.02, 'A', va='center', ha='center', fontweight='bold',
                  transform=matplotlib.transforms.blended_transform_factory(fig3.transFigure, fig3a_ax.transAxes))
    fig3b_ax.text(0.03, 1.02, 'B', va='center', ha='center', fontweight='bold',
                  transform=matplotlib.transforms.blended_transform_factory(fig3.transFigure, fig3b_ax.transAxes))

if 1:
    fig3.savefig(os.path.join(paper_figs_and_tables_dir_path, 'fig3.png'))
    fig3.savefig(os.path.join(paper_figs_and_tables_dir_path, 'fig3.pdf'))


In [ ]:
if 'fig4' in locals():
    plt.close(fig4)
# raise



MTASE_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED = [
    ('MTase: two upstream helicases', 'NZ_CP068294.1', (3122130, 3125861)),
    ('MTase: two upstream helicases', 'NZ_UGYW01000002.1', (1269931, 1273395)),
    ('MTase: two upstream helicases', 'NZ_UFVQ01000003.1', (2778297, 2784881)),
    ('MTase: Class 1 DISARM', 'NZ_CP010519.1', (3712964, 3717292)),
    ('MTase: Class 1 DISARM-like', 'NZ_CP061344.1', (2385445, 2390031)),

    ('MTase: upstream PLD&SNF2 helicase', 'CP083813.1', (5324991, 5328359)),
    ('MTase: downstream PLD&SNF2 helicase', 'CP044495.1', (1257875, 1261063)),
    ('MTase: upstream DUF1016', 'CP033760.1', (4019614, 4023351)), 
    ('MTase: solitary', 'NZ_CP082886.1', (2883305, 2887030)),
    
    ('MTase: BREX type 1', 'NC_013198.1', (2161973, 2165527)),
    ('MTase: BREX type 1, downstream extra short PglX', 'NZ_CP068173.1', (2193315, 2195903)),
]
OTHER_PRESUMABLY_NOVEL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED = [
    ('DUF4965', 'CP065872.1', (6177248, 6179761)),
    ('PilV and phage tail collar', 'CP084655.1', (816997, 818571)),
]


NON_NOVEL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED = [
    ('RM specificity: R-M-S', 'NZ_CP022464.2', (6453804, 6455096)),
    ('RM specificity: R-M-DUF1016-S', 'CP046428.1', (3459162, 3460715)),
    ('RM specificity: R-M-RhuM-S', 'CP081899.1', (2601241, 2602485)),
    ('RM specificity: R-M-Fic/DOC-S', 'NZ_CP082886.1', (4369547, 4371175)),
    ('RM specificity: M-invertibleSs-R', 'NZ_CP059830.1', (15345, 16556)),
    
    ('phage tail: upstream phage tail protein I and downstream tail fiber assembly', 'CP056267.1', (5092989, 5094518)),
    ('phage tail: downstream transporter and endonuclease', 'CP076386.1', (93499, 94659)),
    ('phage tail: upstream DUF2313 and downstream tail fiber assembly', 'CP066032.1', (4023024, 4024490)),

    ('OM receptor: downstream SusD/RagB', 'NZ_CP012938.1', (2847275, 2850391)),
]

FIG4_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED = (
    MTASE_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED
    + 
    OTHER_PRESUMABLY_NOVEL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED
    # + 
    # NON_NOVEL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED
)

ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED = (
    FIG4_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED + 
    NON_NOVEL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED
)
    
FIG4_CDS_POLYGON_FILL_ALPHA = 1
FIG4_CDS_POLYGON_EDGE_ALPHA = 1

FIG4_PRODUCT_CLASS_TO_FILL_COLOR_WITH_ALPHA = {
    **{
        # product_class: (*matplotlib.colors.to_rgb(color), FIG4_CDS_POLYGON_FILL_ALPHA)
        product_class: color
        for product_class, color in PRODUCT_CLASS_TO_COLOR.items()
        if product_class not in ('other', 'hypothetical')
    },
    'other': 'white',
}
FIG4_CDS_POLYGON_EDGE_COLOR_WITH_ALPHA = (*matplotlib.colors.to_rgb('black'), FIG4_CDS_POLYGON_EDGE_ALPHA)
    
    
num_of_cds_contexts = len(FIG4_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED)


fig4 = plt.figure(
    figsize=(
        12, 
        10.5,
        # 1.5 + 0.5 * num_of_cds_contexts,
    ), 
    # constrained_layout=True,
)
num_of_rows = 2 + num_of_cds_contexts * 2 - 1
fig4_gridspec = fig4.add_gridspec(
    num_of_rows, 7, 
    width_ratios=[2.3,10,0.1,1.2,0.8,1.4,0.15], 
    wspace=0.0001,
    hspace=0.0001,
    
    # height_ratios=[2.2, 0.5] + ([1, 0.26] * num_of_cds_contexts)[:-1], 
    height_ratios=([2.2, 0.5] + ([1, 0.26] * (num_of_cds_contexts - 1))[:-1] + [1.3, 1]), 
)

fig4.subplots_adjust(
    top=0.98,
    bottom=0.08,
    left=0.03,
    right=0.99,
    # hspace=0.2,
    # wspace=0.2
)

long_read_alignment_sub_gridspec = fig4_gridspec[0,1].subgridspec(1, 9, width_ratios=[0,1,0.08,1,0.08,1,0.08,1,0.8], wspace=0.00001)

fig4_long_read_alignment_axes = [fig4.add_subplot(long_read_alignment_sub_gridspec[i]) for i in range(1,8,2)]
fig4_cds_context_axes = [fig4.add_subplot(fig4_gridspec[x,1]) for x in range(2, num_of_rows, 2)]
fig4_pie_axes = [fig4.add_subplot(fig4_gridspec[x,3]) for x in range(2, num_of_rows, 2)]
fig4_taxon_dist_axes = [fig4.add_subplot(fig4_gridspec[x,5]) for x in range(2, num_of_rows, 2)]
sub_gridspec_for_colorbar = fig4_gridspec[0,-2].subgridspec(2, 1, height_ratios=[1, 0.3], hspace=0)
fig4_taxon_dist_colorbar_ax = fig4.add_subplot(sub_gridspec_for_colorbar[0])

remove_all_ax_spines_an_x_and_y_axes(fig4_taxon_dist_colorbar_ax)

    
    
def draw_taxon_distribution_heatmap(fig, axes, colorbar_ax, cds_context_names_and_longer_linked_cds_loci_ordered, 
                                    taxon_column_name_for_counting, counted_obj_desc, heatmap_colormap):
    curr_df = all_homologs_extended_df.copy()
    
    curr_df = curr_df.merge(pd.DataFrame([(x[:2] + x[2]) for x in cds_context_names_and_longer_linked_cds_loci_ordered], columns=[
        'cds_context_name', 'longer_linked_repeat_cds_nuccore_accession', 
        'longer_linked_repeat_cds_start', 'longer_linked_repeat_cds_end',
    ]))
    curr_df = curr_df[['cds_context_name', 'taxon_phylum', taxon_column_name_for_counting, 'is_homolog_potentially_modified_by_pi']].drop_duplicates()
    
    count_df = curr_df.groupby(['cds_context_name', 'taxon_phylum'])[taxon_column_name_for_counting].nunique().reset_index(name='total_count').merge(
        curr_df[curr_df['is_homolog_potentially_modified_by_pi']].groupby(
            ['cds_context_name', 'taxon_phylum'])[taxon_column_name_for_counting].nunique().reset_index(
            name='potentially_modified_by_pi_count'), how='left')
    count_df['potentially_modified_by_pi_count'].fillna(0, inplace=True)
    
    
    count_df['phylum_proportion'] = count_df['potentially_modified_by_pi_count'] / count_df['total_count']
    assert (count_df['phylum_proportion'] >= 0).all()
    assert (count_df['phylum_proportion'] <= 1).all()
    assert len(count_df[['cds_context_name', 'taxon_phylum']].drop_duplicates()) == len(count_df)
    cds_context_name_to_phylum_to_proportion_and_total_count = {
        cds_context_name: {
            df_row['taxon_phylum']: (df_row['phylum_proportion'], df_row['total_count'])
            for _, df_row in context_df.iterrows()
        }
        for cds_context_name, context_df in count_df.groupby('cds_context_name')
    }
    
    phylum_names_sorted = sorted(count_df['taxon_phylum'].unique())
    num_of_phyla = len(phylum_names_sorted)

    for i, (ax, cds_context_name_and_longer_linked_cds_locus) in enumerate(zip(axes, cds_context_names_and_longer_linked_cds_loci_ordered)):
        cds_context_name = cds_context_name_and_longer_linked_cds_locus[0]
        
        cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args = search_for_pis_args['stage6'][
            'cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args']
        alignment_region_raw_read_alignment_args = get_alignment_region_raw_read_alignment_args_by_cds_context_name_and_locus(
            cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args, 
            *cds_context_name_and_longer_linked_cds_locus,
        )
        longer_linked_cds_phylum = alignment_region_raw_read_alignment_args['phylum']
        
        no_homologs = cds_context_name not in cds_context_name_to_phylum_to_proportion_and_total_count
        
        # print(f'cds_context_name: {cds_context_name}')
        if not no_homologs:
            phylum_to_proportion_and_total_count = cds_context_name_to_phylum_to_proportion_and_total_count[cds_context_name]

            # make sure the phylum of the locus with long read evidence does not contradict the heatmap.
            assert phylum_to_proportion_and_total_count[longer_linked_cds_phylum][0] > 0
        else:
            phylum_to_proportion_and_total_count = {}
            

        ordered_proportion_and_total_count = [phylum_to_proportion_and_total_count[phylum] if phylum in phylum_to_proportion_and_total_count else (np.nan, np.nan) 
                                              for phylum in phylum_names_sorted]
        
        im = ax.imshow(
            [[x[0] for x in ordered_proportion_and_total_count]],
            cmap=heatmap_colormap,
            vmin=0, vmax=1, 
            # norm=matplotlib.colors.LogNorm(vmin=min_non_zero_value_in_heatmap, vmax=1),
        )
        # ys = np.arange(len(cds_context_names_and_longer_linked_cds_loci_ordered))
        # ax.set_yticks(ys)
        # ax.set_yticklabels(heatmap_df.iloc[i:i+1].index)
        ax.set_yticks([])
        ax.set_xticks([])
        
        xlim = ax.get_xlim()
        assert (xlim[1] - xlim[0]) == num_of_phyla
        
        # ax.grid(which="minor", color="black", linestyle='-', linewidth=1) # doesn't work for some reason...
        ylim = ax.get_ylim()
        
        ax.vlines(x=np.arange(xlim[0] + 1, xlim[1] - 0.5), ymin=ylim[0], ymax=ylim[1], color="black", linestyle='-', linewidth=0.5, alpha=0.4)
        for i, proportion_and_total_count in enumerate(ordered_proportion_and_total_count):
            total_count = proportion_and_total_count[1]
            if pd.isna(total_count):
                for ys in (ylim, ylim[::-1]):
                    ax.plot([xlim[0] + i, xlim[0] + i + 1], ys, color="black", linestyle='-', linewidth=0.5, alpha=0.4)
            else:
                ax.text(xlim[0] + i + 0.5, 1, str(total_count), 
                    va='bottom', ha='center', 
                    transform=matplotlib.transforms.blended_transform_factory(ax.transData, ax.transAxes),
                    clip_on=False,
                    # fontsize=9,
                    fontsize='small',
                )
                    
        

    last_ax = axes[-1]
    xs = np.arange(num_of_phyla)
    # last_ax.set_xticks([])
    last_ax.set_xticks(xs)
    last_ax.set_xticklabels(phylum_names_sorted, rotation=45, ha="right", rotation_mode="anchor", fontsize='x-small')

    cbar = fig.colorbar(
        im, 
        # ax=last_ax, 
        ax=colorbar_ax, 
        # pad=0.15,
        # https://stackoverflow.com/questions/18195758/set-matplotlib-colorbar-size-to-match-graph
        # fraction=fraction=0.06,
        fraction=0.09,
        orientation='horizontal',
    )
    cbar.ax.set_title(f'{counted_obj_desc} fraction', fontsize='small')
    cbar.ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
    
def draw_variant_representation(fig, ax, y_pos, ir_pairs, y_height, color, variant_regions_and_types=None, transform=None):
    if transform is None:
        transform = ax.transData
        
    hatch = '\\\\\\\\\\\\\\\\'
    
    repeat_arrow_props = dict(
        shrinkA=0, shrinkB=0, 
        linewidth=0.5, 
        # arrowstyle="->", 
        # arrowstyle='->,head_width=5',
        # arrowstyle='->,head_length=0.22',
        arrowstyle='->,head_length=0.22,head_width=0.1',
        # mutation_scale=5,
        # color='black',
    )
    repeat_arrow_annotation_kwargs = dict(
        arrowprops=repeat_arrow_props, 
        # clip_on=False, 
        # zorder=100,
        annotation_clip=False,
        # transform=transform,
        xycoords=transform,
    )
    
    
    if variant_regions_and_types is None:
        if len(ir_pairs) == 1:
            ir_pair = next(iter(ir_pairs))
            variant_regions_and_types = ((ir_pair[1:3], {'inverted'}), )
        else:
            assert len(ir_pairs) == 2
            ir_pair1, ir_pair2 = list(ir_pairs)
            if ir_pair2[0] < ir_pair1[0]:
                ir_pair1, ir_pair2 = ir_pair2, ir_pair1
            
            assert ir_pair1[1] < ir_pair2[0]
            assert ir_pair2[3] < ir_pair1[2]
            
            variant_regions_and_types = (
                ((ir_pair1[1], ir_pair2[1]), {'moved', 'inverted'}),
                ((ir_pair2[1], ir_pair2[2]), set()),
                ((ir_pair2[2], ir_pair1[2]), {'moved', 'inverted'}),
            )
    for region, region_type in variant_regions_and_types:
        region_bottom_left = (region[0], y_pos - y_height / 2)
        region_width = region[1] - region[0]
        basic_rectangle_kwargs = dict(
            xy=region_bottom_left, width=region_width, height=y_height,
            facecolor='white', edgecolor=color, fill=False,
            linewidth=0.3, 
            clip_on=False,
            transform=transform,
            # zorder=100,
        )
        
        if region_type:
            if region_type == {'inverted'}:
                rectangle_kwargs = dict(basic_rectangle_kwargs)
                rectangle_kwargs['fill'] = False
                rectangle_kwargs['hatch'] = hatch
                ax.add_patch(matplotlib.patches.Rectangle(**rectangle_kwargs))
            elif region_type == {'moved'}:
                rectangle_kwargs = dict(basic_rectangle_kwargs)
                rectangle_kwargs['fill'] = True
                rectangle_kwargs['facecolor'] = color
                ax.add_patch(matplotlib.patches.Rectangle(**rectangle_kwargs))
            elif region_type == {'moved', 'inverted'}:
                rectangle_kwargs = dict(basic_rectangle_kwargs)
                rectangle_kwargs['fill'] = True
                rectangle_kwargs['facecolor'] = color
                rectangle_kwargs['edgecolor'] = 'white'
                rectangle_kwargs['hatch'] = hatch
                ax.add_patch(matplotlib.patches.Rectangle(**rectangle_kwargs))
            else:
                # print(f'region_type: {region_type}')
                assert False
        
        ax.add_patch(matplotlib.patches.Rectangle(**basic_rectangle_kwargs))
    
    
    for ir_pair in ir_pairs:
        ax.annotate("", xy=(ir_pair[1], y_pos), xytext=(ir_pair[0], y_pos), **repeat_arrow_annotation_kwargs)
        ax.annotate("", xy=(ir_pair[2], y_pos), xytext=(ir_pair[3], y_pos), **repeat_arrow_annotation_kwargs)
    

def get_position_tuple_mirror_image(positions):
    return tuple(-x for x in positions[::-1])
    
def get_cds_df_or_mirror_image_cds_df(cds_df, longer_linked_repeat_cds_strand):
    if longer_linked_repeat_cds_strand == -1:
        cds_df = cds_df.copy()
        cds_df.rename(columns={'start_pos': 'end_pos', 'end_pos': 'start_pos'}, inplace=True)
        cds_df['start_pos'] = -cds_df['start_pos']
        cds_df['end_pos'] = -cds_df['end_pos']
        assert (cds_df['strand'].abs() == 1).all()
        cds_df['strand'] = -cds_df['strand']
    return cds_df

def get_ir_pair_region_with_margins_or_its_mirror_image(ir_pair_region_with_margins, longer_linked_repeat_cds_strand):
    return (
        ir_pair_region_with_margins if (longer_linked_repeat_cds_strand == 1)
        else get_position_tuple_mirror_image(ir_pair_region_with_margins)
    )

def get_variant_ir_pairs_or_their_mirror_images(variant_ir_pairs, longer_linked_repeat_cds_strand):
    return (
        variant_ir_pairs if (longer_linked_repeat_cds_strand == 1)
        else frozenset({get_position_tuple_mirror_image(ir_pair) for ir_pair in variant_ir_pairs})
    )

def get_variant_regions_and_types_or_their_mirror_images(variant_regions_and_types, longer_linked_repeat_cds_strand):
    return (
        variant_regions_and_types if (longer_linked_repeat_cds_strand == 1)
        else tuple((get_position_tuple_mirror_image(x), y) for x, y in variant_regions_and_types)
    )
    
def draw_cds_context_representatives(fig, context_axes, pie_axes, cds_context_names_and_longer_linked_cds_loci_ordered, non_ref_variant_colors,
                                     product_class_to_fill_color_with_alpha, cds_polygon_edge_color_with_alpha, 
                                     legend_kwargs, draw_variant_repr=True, move_right_for_non_ref_pie=False):
    cds_height = 0.4
    dist_between_cds_regions = 0.03
    cds_region_margin_size_as_fraction_of_cds_region = 0.0002
    dist_between_cds_region_bottom_and_arrow_ends = 0.23
    vertical_dist_between_different_variant_arrow_heads = 0.2
    vertical_dist_between_same_variant_arrows = 0.08
    # cds_arrowhead_width_as_fraction_of_max_presumably_relevant_cds_region_len = 0.011
    # cds_arrowhead_width_as_fraction_of_max_presumably_relevant_cds_region_len = 0.008
    cds_arrowhead_width_as_fraction_of_max_presumably_relevant_cds_region_len = 0.006
    
    max_num_of_non_ref_variants = len(non_ref_variant_colors)
    cds_context_classes_ordered = [x[0].partition(':')[0] for x in cds_context_names_and_longer_linked_cds_loci_ordered]
    cds_context_classes = set(cds_context_classes_ordered)
    num_of_cds_contexts = len(cds_context_names_and_longer_linked_cds_loci_ordered)
    
    cds_context_class_to_context_interval = {
        x: (cds_context_classes_ordered.index(x), num_of_cds_contexts - cds_context_classes_ordered[::-1].index(x) - 1)
        for x in cds_context_classes
    }
    assert len(generic_utils.get_merged_intervals(cds_context_class_to_context_interval.values())) == len(cds_context_class_to_context_interval)
    
    
    curr_df = all_raw_read_alignment_results_df.copy()
    curr_df = curr_df[curr_df['describe_in_the_paper']]
    
    curr_df = curr_df.merge(pd.DataFrame([(x[:2] + x[2]) for x in cds_context_names_and_longer_linked_cds_loci_ordered],
                                         columns=['cds_context_name', 'nuccore_accession', 
                                                  'longer_linked_repeat_cds_start', 'longer_linked_repeat_cds_end']))
    max_presumably_relevant_cds_region_len = (curr_df['presumably_relevant_cds_region_end'] - 
                                              curr_df['presumably_relevant_cds_region_start'] + 1).max()
    
    cds_arrowhead_width = round(cds_arrowhead_width_as_fraction_of_max_presumably_relevant_cds_region_len * max_presumably_relevant_cds_region_len)
    
    curr_df_grouped = curr_df.groupby(['cds_context_name', 'nuccore_accession', 'longer_linked_repeat_cds_start', 'longer_linked_repeat_cds_end'])
    cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args = search_for_pis_args['stage6'][
        'cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args']
    name_to_cds_context_info = search_for_pis_args['enrichment_analysis']['name_to_cds_context_info']
    all_product_classes_in_fig = set()
    for (cds_context_name, nuccore_accession, longer_linked_repeat_cds_region), context_ax, pie_ax in zip(cds_context_names_and_longer_linked_cds_loci_ordered, context_axes, pie_axes):
        # print(cds_context_name)
        
        alignment_region_raw_read_alignment_args = get_alignment_region_raw_read_alignment_args_by_cds_context_name_and_locus(
            cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args, 
            cds_context_name, nuccore_accession, longer_linked_repeat_cds_region,
        )
        locus_definition = alignment_region_raw_read_alignment_args['locus_definition']
        phylum = alignment_region_raw_read_alignment_args['phylum']
        longer_linked_repeat_cds_protein_id = alignment_region_raw_read_alignment_args['longest_linked_repeat_cds_region_and_protein_id'][1]
        
        longer_linked_repeat_cds_region_middle = np.mean(longer_linked_repeat_cds_region)
        
        
        curr_context_df = curr_df_grouped.get_group((cds_context_name, nuccore_accession, *longer_linked_repeat_cds_region))
        assert len(curr_context_df[[
            'ir_pairs_df_csv_file_path',
            'nuccore_gb_file_path',
            'filtered_extended_cds_df_csv_file_path',
            'num_of_ref_variant_evidence_reads',
            'num_of_non_ref_variant_evidence_reads',
            'presumably_relevant_cds_region_start',
            'presumably_relevant_cds_region_end',
            'nuccore_accession',
            'longer_linked_repeat_cds_strand',
        ]].drop_duplicates()) == 1
        
        
        ir_pairs_df_csv_file_path = curr_context_df['ir_pairs_df_csv_file_path'].iloc[0]
        nuccore_accession = curr_context_df['nuccore_accession'].iloc[0]
        ir_pair_region_with_margins = alignment_region_raw_read_alignment_args['ir_pair_region_with_margins']
        alignment_region = alignment_region_raw_read_alignment_args['alignment_region']
        cds_region = alignment_region_raw_read_alignment_args['presumably_relevant_cds_region']
        nuccore_gb_file_path = curr_context_df['nuccore_gb_file_path'].iloc[0]
        filtered_extended_cds_df_csv_file_path = curr_context_df['filtered_extended_cds_df_csv_file_path'].iloc[0]
        num_of_ref_variant_evidence_reads = curr_context_df['num_of_ref_variant_evidence_reads'].iloc[0]
        num_of_non_ref_variant_evidence_reads = curr_context_df['num_of_non_ref_variant_evidence_reads'].iloc[0]
        longer_linked_repeat_cds_strand = curr_context_df['longer_linked_repeat_cds_strand'].iloc[0]
        total_num_of_evidence_reads = num_of_ref_variant_evidence_reads + num_of_non_ref_variant_evidence_reads
        
        sra_accession_to_variants_and_reads_info = alignment_region_raw_read_alignment_args[
            'sra_accession_to_variants_and_reads_info']
        assert len(sra_accession_to_variants_and_reads_info) == 1
        reads_info = next(iter(
            sra_accession_to_variants_and_reads_info.values()))
        non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref = reads_info[
            'non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref']
        
        

        
        cds_df = get_cds_df_with_product_class(filtered_extended_cds_df_csv_file_path, alignment_region_raw_read_alignment_args)
            
        all_product_classes_in_fig |= set(cds_df['product_class'])
        
        cds_region_len = cds_region[1] - cds_region[0] + 1
        # region_top = 0.97
        region_top = 0.5 + cds_height / 2
        
        region_bottom = region_top - cds_height
        
        cds_region_margin_size = round(cds_region_margin_size_as_fraction_of_cds_region * cds_region_len)
        
        
        draw_cds_invertion_on_ax(
            ax=context_ax,
            # region=region,
            # region=get_ir_pair_region_with_margins_or_its_mirror_image(ir_pair_region_with_margins, longer_linked_repeat_cds_strand),
            cds_df=get_cds_df_or_mirror_image_cds_df(cds_df, longer_linked_repeat_cds_strand),
            cds_arrowhead_width=cds_arrowhead_width,
            region_bottom_and_top=(region_bottom, region_top),
            cds_polygon_edge_color_with_alpha=cds_polygon_edge_color_with_alpha,
            product_class_to_fill_color_with_alpha=product_class_to_fill_color_with_alpha,
            draw_region_rectangle=False,
            clip_on=False,
            draw_cds_polygon_again=True,
            # cds_polygon_edge_color=None,
            # cds_polygon_edge_color='white',
        )
        text_left_to_context = f'{locus_definition}\n{nuccore_accession}\n{cds_region[0]}-{cds_region[1]}'
        
        if longer_linked_repeat_cds_strand == -1:
            text_left_to_context += ', complementary'
        
        context_ax.text(-0.01, 0.5, text_left_to_context, 
                        va='center', ha='right', transform=context_ax.transAxes, clip_on=False, fontsize='x-small')
        
        
        non_ref_variant_ir_pairs_to_num_of_evidence_reads = get_variant_ir_pairs_to_num_of_evidence_reads(
            non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref)
        
        
        
        # Actually, this isn't guaranteed, because a read can be discarded due to a better alignment to somewhere else in the genome.
        # For example, SRR11812841.1.24913.1 (though see detailed explanation in massive_screening_configuration.py).
        # assert sum(non_ref_variant_ir_pairs_to_num_of_evidence_reads.values()) == num_of_non_ref_variant_evidence_reads
        
        
        sorted_non_ref_variant_ir_pairs_and_num_of_evidence_reads = sorted(
            non_ref_variant_ir_pairs_to_num_of_evidence_reads.items(), key=(lambda x: x[1]), reverse=True)
        
        if max_num_of_non_ref_variants < len(non_ref_variant_ir_pairs_to_num_of_evidence_reads):
            raise RuntimeError(f'too many non ref variants ({len(non_ref_variant_ir_pairs_to_num_of_evidence_reads)} variants here)!')
            
        # assert len(non_ref_variant_colors) >= len(non_ref_variant_ir_pairs_to_num_of_evidence_reads)
        
        non_ref_variant_ir_pairs_to_color = {x: y for (x, _), y in zip(sorted_non_ref_variant_ir_pairs_and_num_of_evidence_reads, non_ref_variant_colors)}
        
        if draw_variant_repr:
            variant_height = vertical_dist_between_different_variant_arrow_heads / 2
            
            for i, (variant_ir_pairs, color) in enumerate(non_ref_variant_ir_pairs_to_color.items()):
                if i % 2 == 0:
                    variant_repr_y = (region_bottom - dist_between_cds_region_bottom_and_arrow_ends - 
                                      (i // 2) * vertical_dist_between_different_variant_arrow_heads)
                else:
                    variant_repr_y = (region_top + dist_between_cds_region_bottom_and_arrow_ends + 
                                      (i // 2) * vertical_dist_between_different_variant_arrow_heads)
                
                try:
                    variant_regions_and_types = reads_info[
                        'complex_variant_ir_pairs_to_variant_regions_and_types'][variant_ir_pairs]
                    variant_regions_and_types_or_their_mirror_images = get_variant_regions_and_types_or_their_mirror_images(
                        variant_regions_and_types, longer_linked_repeat_cds_strand)
                except KeyError:
                    variant_regions_and_types_or_their_mirror_images = None
                    
                except KeyError:
                    variant_regions_and_types = None
                draw_variant_representation(
                    fig=fig, 
                    ax=context_ax, 
                    y_pos=variant_repr_y, 
                    ir_pairs=get_variant_ir_pairs_or_their_mirror_images(variant_ir_pairs, longer_linked_repeat_cds_strand),
                    y_height=variant_height, 
                    color=color, 
                    variant_regions_and_types=variant_regions_and_types_or_their_mirror_images,
                )
                
        
        if longer_linked_repeat_cds_strand == 1:
            xlim_left = cds_region[0]
            xlim_right = cds_region[0] + max_presumably_relevant_cds_region_len
            assert xlim_right >= cds_region[1]
        else:
            xlim_left = -cds_region[1]
            xlim_right = xlim_left + max_presumably_relevant_cds_region_len
            assert xlim_right >= -cds_region[0]
            
        x_lims = (xlim_left, xlim_right)
        context_ax.set_xlim(x_lims)
        context_ax.set_ylim(-0.25, 1.25)
        
        num_of_evidence_reads_per_variant_sorted = [x for (_, x) in sorted_non_ref_variant_ir_pairs_and_num_of_evidence_reads]
        variant_colors_sorted = [non_ref_variant_ir_pairs_to_color[x] for x, _ in sorted_non_ref_variant_ir_pairs_and_num_of_evidence_reads]
        total_num_of_non_ref_reads = sum(num_of_evidence_reads_per_variant_sorted)
        ref_start_angle = 180 + total_num_of_non_ref_reads / (num_of_ref_variant_evidence_reads + total_num_of_non_ref_reads) * 180
        
        num_of_non_ref_variants = len(num_of_evidence_reads_per_variant_sorted)
        draw_non_ref_pie = (num_of_non_ref_variants >= 4) and (num_of_non_ref_variants <= 10)
        pie_radius = 1
        
        if draw_non_ref_pie and move_right_for_non_ref_pie:
            ref_pie_center = (2.7, 0)
            non_ref_pie_center = (0, 0)
        else:
            ref_pie_center = (0, 0)
            non_ref_pie_center = (-2.7, 0)
        
        pie_ax.pie([num_of_ref_variant_evidence_reads] + num_of_evidence_reads_per_variant_sorted, 
                   colors=(['black'] + variant_colors_sorted), 
                   radius=pie_radius, 
                   # center=(cds_region[1] + 400, 0),
                   center=ref_pie_center,
                   startangle=ref_start_angle,
             # counterclock=True, startangle=0,
             )
        pie_frame_circle_kwargs = dict(
            color='black', fill=False, clip_on=False,
            linewidth=0.4,
        )
        # pie_ax.add_patch(plt.Circle(ref_pie_center, radius=pie_radius, **pie_frame_circle_kwargs))
        
        if draw_non_ref_pie:
            ref_start_angle_in_radians = ref_start_angle * np.pi / 180
            ref_start_x = ref_pie_center[0] + np.cos(ref_start_angle_in_radians)
            ref_start_y = ref_pie_center[1] + np.sin(ref_start_angle_in_radians)
            
            non_ref_pie_center_x = non_ref_pie_center[0]
            non_ref_pie_radius = 1
            
            
            # I want each of the lines that connect the two pies to be tangent to the new pie. I didn't manage to find an analytic solution
            # fast, so I just solve it numerically (quick and dirty, but it seems to work).

            # use the law of cosines
            c_squared = (ref_start_x - non_ref_pie_center_x) ** 2 + ref_start_y ** 2
            a = non_ref_pie_radius
            a_squared = a ** 2
            
            def get_wanted_point_x(radian_angle_in_non_ref_pie):
                return non_ref_pie_center_x + np.cos(radian_angle_in_non_ref_pie) * non_ref_pie_radius
            
            def get_wanted_point_y(radian_angle_in_non_ref_pie):
                return np.sin(radian_angle_in_non_ref_pie) * non_ref_pie_radius
                
            def get_abs_dif_between_angle_and_90_in_radians(radian_angle_in_non_ref_pie):
                x = get_wanted_point_x(radian_angle_in_non_ref_pie)
                y = get_wanted_point_y(radian_angle_in_non_ref_pie)
                b_squared = (ref_start_x - x) ** 2 + (-ref_start_y - y) ** 2
                b = np.sqrt(b_squared)
                return abs(np.arccos((a_squared + b_squared - c_squared) / (2 * a * b)) - np.pi / 2)
            
            angle_for_wanted_point = min([angle for angle in np.linspace(0, np.pi / 2, 100)], key=get_abs_dif_between_angle_and_90_in_radians)
            wanted_point_x = get_wanted_point_x(angle_for_wanted_point)
            wanted_upper_point_y = get_wanted_point_y(angle_for_wanted_point)
            
            pie_ax.add_patch(matplotlib.patches.Polygon(
                [
                    (wanted_point_x, wanted_upper_point_y),
                    (wanted_point_x, -wanted_upper_point_y),
                    (ref_start_x, ref_start_y),
                    (ref_start_x, -ref_start_y),
                ], 
                closed=True, fill=True, facecolor=(*matplotlib.colors.to_rgb('black'), 0.1), edgecolor=(*matplotlib.colors.to_rgb('black'), 1),
                linewidth=0.3, 
                clip_on=False,
            )).set_zorder(0)
            
            non_ref_pie_center = (non_ref_pie_center_x, 0)
            non_ref_pie_patches, _ = pie_ax.pie(
                num_of_evidence_reads_per_variant_sorted, 
                colors=variant_colors_sorted,
                radius=non_ref_pie_radius, 
                # center=(cds_region[1] + 400, 0),
                center=non_ref_pie_center,
                startangle=0,
                # counterclock=True, startangle=0,
            )
            [x.set_zorder(100) for x in non_ref_pie_patches]
            
            # pie_ax.add_patch(plt.Circle(non_ref_pie_center, radius=non_ref_pie_radius, **pie_frame_circle_kwargs))
            

        pie_ax.set_ylim(-1.01, 1.01)
        pie_ax.set_xlim(-1.01, 1.01)
        pie_ax.text(
            ref_pie_center[0] + 1.13, 0, str(total_num_of_evidence_reads), va='center', ha='left', 
            # ref_pie_center[0] + 1.03, 0.5, str(total_num_of_evidence_reads), va='center', ha='left', 
            # transform=(matplotlib.transforms.blended_transform_factory(fig4_pie_axes[0].transAxes, pie_ax.transAxes)),
        )
        
        
        # ax.set_xlim(cds_region[0] - 300, cds_region[1] + )
        
    legend_elements = [
        # ax.add_patch(matplotlib.patches.Polygon([(0,0), (0,0)], closed=True, fill=True, facecolor=product_class_to_fill_color_with_alpha[product_class], 
        #                                         edgecolor=cds_polygon_edge_color_with_alpha, label=product_class)) 
        matplotlib.patches.Patch(facecolor=product_class_to_fill_color_with_alpha[product_class], edgecolor=cds_polygon_edge_color_with_alpha,
                         label=product_class)
        for product_class in PRODUCT_CLASSES_ORDERED
        if (product_class not in ('other', 'hypothetical')) and (product_class in all_product_classes_in_fig)
    ]

    
    
    fig.legend(
        handles=legend_elements,
        **legend_kwargs,
    )

def draw_long_read_alignments(fig, axes, cds_context_name, nuccore_accession, longer_linked_repeat_cds_region, 
                              product_class_to_fill_color_with_alpha, cds_polygon_edge_color_with_alpha, non_ref_variant_colors,
                             num_of_axes_in_line, cds_arrowhead_width_as_fraction_of_ir_pair_region_with_margins=0.05):
    
    region_top = 0.12
    region_bottom = 0.04
    dist_between_arbitrary_lowest_pos_in_read_of_different_variants_as_fraction_of_ir_pair_region_with_margins = 0.25
    
    cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args = search_for_pis_args['stage6'][
        'cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args']
    alignment_region_raw_read_alignment_args = (
        get_alignment_region_raw_read_alignment_args_by_cds_context_name_and_locus(
            cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args, 
            cds_context_name, 
            nuccore_accession, 
            longer_linked_repeat_cds_region,
        )
    )
    ir_pair_region_with_margins = alignment_region_raw_read_alignment_args['ir_pair_region_with_margins']
    
    ir_pair_region_with_margins_len = ir_pair_region_with_margins[1] - ir_pair_region_with_margins[0] + 1
    
    curr_df = all_raw_read_alignment_results_df.copy()
    curr_df = curr_df[curr_df['cds_context_name'] == cds_context_name]
    curr_df = curr_df[curr_df['nuccore_accession'] == nuccore_accession]
    curr_df = curr_df[curr_df['ir_pair_region_with_margins_start'] == ir_pair_region_with_margins[0]]
    curr_df = curr_df[curr_df['ir_pair_region_with_margins_end'] == ir_pair_region_with_margins[1]]
    curr_df = curr_df[curr_df['is_best_evidence_read_for_variant']]

    longer_linked_repeat_cds_strand = set(curr_df['longer_linked_repeat_cds_strand'])
    assert len(longer_linked_repeat_cds_strand) == 1
    longer_linked_repeat_cds_strand = next(iter(longer_linked_repeat_cds_strand))
    
    ref_read_name = curr_df[curr_df['read_evidence_type'] == 'ref_variant']['sseqid'].iloc[0]
    non_ref_read_names = set(curr_df[curr_df['read_evidence_type'] == 'non_ref_variant']['sseqid'])
    num_of_variants = len(curr_df)
    
    
    assert curr_df['filtered_extended_cds_df_csv_file_path'].nunique() == 1
    filtered_extended_cds_df_csv_file_path = curr_df['filtered_extended_cds_df_csv_file_path'].iloc[0]
    assert curr_df['best_evidence_read_name_to_info_pickle_file_path'].nunique() == 1
    best_evidence_read_name_to_info_pickle_file_path = curr_df['best_evidence_read_name_to_info_pickle_file_path'].iloc[0]
    
    with open(best_evidence_read_name_to_info_pickle_file_path, 'rb') as f:
        best_evidence_read_name_to_info = pickle.load(f)
    
    
    sra_accession_to_variants_and_reads_info = alignment_region_raw_read_alignment_args[
        'sra_accession_to_variants_and_reads_info']
    assert len(sra_accession_to_variants_and_reads_info) == 1
    reads_info = next(iter(
        sra_accession_to_variants_and_reads_info.values()))
    non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref = reads_info['non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref']
    non_ref_variant_ir_pairs_to_num_of_evidence_reads = get_variant_ir_pairs_to_num_of_evidence_reads(non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref)
    
    non_ref_variant_ir_pairs_ordered = sorted(non_ref_variant_ir_pairs_to_num_of_evidence_reads, reverse=True, 
                                              key=lambda x: non_ref_variant_ir_pairs_to_num_of_evidence_reads[x])
    non_ref_read_names_sorted = [
        [read_name for read_name in non_ref_read_names
         if (non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref[read_name] == variant_ir_pairs)][0]
        for variant_ir_pairs in non_ref_variant_ir_pairs_ordered
    ]
    ref_and_non_ref_read_names_sorted = [ref_read_name] + non_ref_read_names_sorted
    
    
    cds_df = get_cds_df_with_product_class(filtered_extended_cds_df_csv_file_path, alignment_region_raw_read_alignment_args)

    assert len(non_ref_read_names_sorted) <= len(non_ref_variant_colors)
    
    num_of_reads_to_plot = len(ref_and_non_ref_read_names_sorted)
    for i in range(num_of_reads_to_plot, len(axes)):
        remove_all_ax_spines_an_x_and_y_axes(axes[i])
        
    max_ys = []
    for ax, read_name, variant_color, variant_ir_pairs in zip(axes, ref_and_non_ref_read_names_sorted, 
                                                              ['black'] + non_ref_variant_colors, [None] + non_ref_variant_ir_pairs_ordered):
        # print(read_name)
        transform_for_plotting_cds = matplotlib.transforms.blended_transform_factory(ax.transData, ax.transAxes)
        cds_arrowhead_width = round(cds_arrowhead_width_as_fraction_of_ir_pair_region_with_margins * ir_pair_region_with_margins_len)
        

        draw_cds_invertion_on_ax(
            ax=ax,
            region=get_ir_pair_region_with_margins_or_its_mirror_image(ir_pair_region_with_margins, longer_linked_repeat_cds_strand), # could actually remove this line?
            cds_df=get_cds_df_or_mirror_image_cds_df(cds_df, longer_linked_repeat_cds_strand),
            # ir_pairs_df=ir_pairs_df,
            cds_arrowhead_width=cds_arrowhead_width,
            region_bottom_and_top=(region_bottom, region_top),
            cds_polygon_edge_color_with_alpha=cds_polygon_edge_color_with_alpha,
            product_class_to_fill_color_with_alpha=product_class_to_fill_color_with_alpha,
            draw_region_rectangle=False,
            transform=transform_for_plotting_cds,
            # clip_on=False,
        )
        
        if variant_ir_pairs:
            try:
                variant_regions_and_types = reads_info[
                    'complex_variant_ir_pairs_to_variant_regions_and_types'][variant_ir_pairs]
                variant_regions_and_types_or_their_mirror_images = get_variant_regions_and_types_or_their_mirror_images(
                    variant_regions_and_types, longer_linked_repeat_cds_strand)
            except KeyError:
                variant_regions_and_types_or_their_mirror_images = None
            draw_variant_representation(
                fig=fig, 
                ax=ax, 
                y_pos=-0.04, 
                ir_pairs=get_variant_ir_pairs_or_their_mirror_images(variant_ir_pairs, longer_linked_repeat_cds_strand),
                y_height=0.06, 
                color=variant_color, 
                variant_regions_and_types=variant_regions_and_types_or_their_mirror_images,
                transform=transform_for_plotting_cds,
            )
        
        
        mauve_xmfa_file_path = best_evidence_read_name_to_info[read_name]['mauve_alignment_results_xmfa_file_path']
        # print(mauve_xmfa_file_path)
        list_of_matching_positions_df_per_pairwise_alignment = mauve_interface_and_utils.get_list_of_matching_positions_df_for_each_pairwise_alignment_in_xmfa(
            mauve_xmfa_file_path,
        )


        lowest_pos_in_read = min(x['seq0_position'].min() for x in list_of_matching_positions_df_per_pairwise_alignment)
        max_pos_in_read = max(x['seq0_position'].max() for x in list_of_matching_positions_df_per_pairwise_alignment)
        
        for matching_positions_df in list_of_matching_positions_df_per_pairwise_alignment:
            seq1_positions = matching_positions_df['seq1_position'] + ir_pair_region_with_margins[0] - 1
            
            ax.scatter(
                seq1_positions if (longer_linked_repeat_cds_strand == 1) else -seq1_positions,
                matching_positions_df['seq0_position'] - lowest_pos_in_read + 1,
                s=0.1,
                marker='o',
                # marker='.',
                # color=variant_color,
                facecolors=variant_color,
                edgecolors=variant_color,
                alpha=0.7,
            )
        
        curr_max_y = max_pos_in_read - lowest_pos_in_read + 1
        max_ys.append(curr_max_y)
        
    max_y = max(max_ys)
    for ax in axes[:num_of_reads_to_plot]:
        min_y = -100
        assert region_top > 0
        new_ylim_bottom = (min_y - region_top * max_y) / (1 - region_top)
        ax.set_ylim(new_ylim_bottom, max_y)
        
        # set_x_lim_according_to_region(ax, ir_pair_region_with_margins, margin_relative_size=0.001)
        # ax.set_xlim(ir_pair_region_with_margins)
        ax.set_xlim(get_ir_pair_region_with_margins_or_its_mirror_image(ir_pair_region_with_margins, longer_linked_repeat_cds_strand))
        # ax.spines['left'].set_visible(True)
        ax.get_yaxis().set_visible(True)
        ax.plot([(ir_pair_region_with_margins[0] if (longer_linked_repeat_cds_strand == 1) else -ir_pair_region_with_margins[1])] * 2, 
                [region_top, 1], color='black', clip_on=True, 
                transform=matplotlib.transforms.blended_transform_factory(ax.transData, ax.transAxes))
        ax.set_aspect('equal')
        yticks = ax.get_yticks().tolist()
        new_yticks = [x for x in yticks if ((x % 1000 == 0) and x >= 0)]
        ax.yaxis.set_major_locator(matplotlib.ticker.FixedLocator(new_yticks))
        ax.set_yticklabels([remove_trailing_zeros(str(x / 1000)) for x in new_yticks])
        # ax.get_xaxis().set_visible(True)
    
    for i in range(0, num_of_reads_to_plot, num_of_axes_in_line):
        axes[i].set_ylabel(
            # 'position in\nread (kbp)', 
            'relative position\nin read (kbp)', 
            fontsize='small',
        )
    
    
        
    


taxon_heatmap_colormap = matplotlib.colors.LinearSegmentedColormap.from_list('', ['0.8', '0'])
taxon_heatmap_colormap = matplotlib.colors.LinearSegmentedColormap.from_list('', ['0.85', '0'])
taxon_heatmap_colormap = matplotlib.colors.LinearSegmentedColormap.from_list('', ['0.9', '0'])
taxon_heatmap_colormap = matplotlib.colors.LinearSegmentedColormap.from_list('', ['1', '0'])
taxon_heatmap_colormap.set_bad(color='white')

legend_kwargs = dict(
    bbox_transform=fig4_long_read_alignment_axes[-1].transAxes,
    bbox_to_anchor=(1.3, 0.5),
    loc='center left',
    # fontsize='x-small', 
    fontsize='small', 
)
    
draw_cds_context_representatives(
    fig4, context_axes=fig4_cds_context_axes, pie_axes=fig4_pie_axes,
    cds_context_names_and_longer_linked_cds_loci_ordered=FIG4_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED, 
    non_ref_variant_colors=NON_REF_VARIANT_COLORS,
    product_class_to_fill_color_with_alpha=FIG4_PRODUCT_CLASS_TO_FILL_COLOR_WITH_ALPHA,
    cds_polygon_edge_color_with_alpha=FIG4_CDS_POLYGON_EDGE_COLOR_WITH_ALPHA,
    legend_kwargs=legend_kwargs,
    # draw_variant_repr=False,
)

draw_taxon_distribution_heatmap(
    fig4, axes=fig4_taxon_dist_axes, colorbar_ax=fig4_taxon_dist_colorbar_ax,
    cds_context_names_and_longer_linked_cds_loci_ordered=FIG4_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED, 
    # taxon_column_name_for_counting='taxon_uid', 
    taxon_column_name_for_counting='taxon_species', 
    counted_obj_desc='species',
    heatmap_colormap=taxon_heatmap_colormap,
)

draw_long_read_alignments(
    fig4, fig4_long_read_alignment_axes, 
    cds_context_name=FIG4_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED[0][0], 
    nuccore_accession=FIG4_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED[0][1], 
    longer_linked_repeat_cds_region=FIG4_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED[0][2], 
    cds_polygon_edge_color_with_alpha=FIG4_CDS_POLYGON_EDGE_COLOR_WITH_ALPHA,
    product_class_to_fill_color_with_alpha=FIG4_PRODUCT_CLASS_TO_FILL_COLOR_WITH_ALPHA,
    non_ref_variant_colors=NON_REF_VARIANT_COLORS,
    num_of_axes_in_line=4,
    cds_arrowhead_width_as_fraction_of_ir_pair_region_with_margins=0.05,
)

arbitrary_ax = fig4_long_read_alignment_axes[0]
arbitrary_ax.text(0.015, 0.985, 'A', va='center', ha='center', transform=fig4.transFigure, fontweight='bold')
arbitrary_ax.text(0.015, 1.08, 'B', va='center', ha='center', fontweight='bold',
                  transform=matplotlib.transforms.blended_transform_factory(fig4.transFigure, fig4_cds_context_axes[0].transAxes))
arbitrary_ax.text(-0.09, 1.08, 'C', va='center', ha='center', fontweight='bold',
                  transform=matplotlib.transforms.blended_transform_factory(fig4_pie_axes[0].transAxes, fig4_cds_context_axes[0].transAxes))
arbitrary_ax.text(-0.07, 1.08, 'D', va='center', ha='center',fontweight='bold',
                  transform=matplotlib.transforms.blended_transform_factory(fig4_taxon_dist_axes[0].transAxes, fig4_cds_context_axes[0].transAxes))

if 1:
    fig4.savefig(os.path.join(paper_figs_and_tables_dir_path, 'fig4.png'))
    fig4.savefig(os.path.join(paper_figs_and_tables_dir_path, 'fig4.pdf'))

In [ ]:
if 1:
    plt.close('all')
# raise

sup_fig_start_num = 9
num_of_long_read_alignments_per_line = 4
for i, cds_context_name_and_locus in enumerate(ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED[1:]):
    print(f'cds_context_name_and_locus: {cds_context_name_and_locus}')
    cds_context_name, nuccore_accession, longer_linked_repeat_cds_region = cds_context_name_and_locus
    
    cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args = search_for_pis_args['stage6'][
        'cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args']
    alignment_region_raw_read_alignment_args = get_alignment_region_raw_read_alignment_args_by_cds_context_name_and_locus(
        cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args, 
        cds_context_name, nuccore_accession, longer_linked_repeat_cds_region,
    )
    sra_accession_to_variants_and_reads_info = alignment_region_raw_read_alignment_args[
        'sra_accession_to_variants_and_reads_info']
    assert len(sra_accession_to_variants_and_reads_info) == 1
    reads_info = next(iter(
        sra_accession_to_variants_and_reads_info.values()))
    non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref = reads_info[
        'non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref']
    non_ref_variant_ir_pairs_to_num_of_evidence_reads = get_variant_ir_pairs_to_num_of_evidence_reads(
        non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref)
    num_of_non_ref_variants = len(non_ref_variant_ir_pairs_to_num_of_evidence_reads)
    num_of_variants = 1 + num_of_non_ref_variants
    # num_of_long_read_rows = math.ceil(num_of_variants / 3)
    num_of_long_read_rows = 2 if (num_of_variants <= 8) else 7
    num_of_long_read_axes = num_of_long_read_rows * num_of_long_read_alignments_per_line
    
    y_size = 1.6 + 1.8 * num_of_long_read_rows
    height_ratios = [0.7, 0.4] + ([2.2, 0.4] * num_of_long_read_rows)[:-1]
    if num_of_variants > 15:
        height_ratios = [0.76, 1.2] + ([2.2, 0.4] * num_of_long_read_rows)[:-1]
    
    fig = plt.figure(
        figsize=(
            10, 
            y_size,
        ), 
    )
    fig_gridspec = fig.add_gridspec(
        1 + num_of_long_read_rows * 2, 4, 
        wspace=0.0001,
        hspace=0.0001,
        width_ratios=[3,10,0.43,1.2], 
        height_ratios=height_ratios, 
    )

    fig.subplots_adjust(
        # top=0.96 if (num_of_long_read_rows == 2) else 0.945,
        top=0.945,
        bottom=0.06,
        left=0.02,
        right=0.85,
        # hspace=0.2,
        # wspace=0.2
    )
    # fig1.tight_layout()

    long_read_alignment_sub_gridspecs = [
        fig_gridspec[x,1].subgridspec(1, 9, width_ratios=[0,1,0.33,1,0.33,1,0.33,1,0], wspace=0.00001)
        for x in range(2, 2 + 2 * num_of_long_read_rows, 2)
    ]
    
    long_read_alignment_axes = []
    for long_read_alignment_sub_gridspec in long_read_alignment_sub_gridspecs:
        long_read_alignment_axes.extend([fig.add_subplot(long_read_alignment_sub_gridspec[i]) for i in (1,3,5,7)])
    
    cds_context_ax = fig.add_subplot(fig_gridspec[0,1])
    pie_ax = fig.add_subplot(fig_gridspec[0,3])

    
    legend_kwargs = dict(
        # bbox_transform=cds_context_ax.transAxes,
        # bbox_to_anchor=(1.027, -0.13),
        # loc='upper left',
        # bbox_transform=long_read_alignment_axes[2].transAxes,
        bbox_transform=matplotlib.transforms.blended_transform_factory(pie_ax.transAxes, long_read_alignment_axes[0].transAxes),
        bbox_to_anchor=(-0.14, 1.06),
        loc='upper left',
        fontsize='small', 
    )

    draw_cds_context_representatives(
        fig, context_axes=[cds_context_ax], pie_axes=[pie_ax],
        cds_context_names_and_longer_linked_cds_loci_ordered=[cds_context_name_and_locus], 
        non_ref_variant_colors=NON_REF_VARIANT_COLORS,
        product_class_to_fill_color_with_alpha=FIG4_PRODUCT_CLASS_TO_FILL_COLOR_WITH_ALPHA,
        cds_polygon_edge_color_with_alpha=FIG4_CDS_POLYGON_EDGE_COLOR_WITH_ALPHA,
        legend_kwargs=legend_kwargs,
        move_right_for_non_ref_pie=True,
        # show_variant_ir_pairs=False,
    )

    draw_long_read_alignments(
        fig, long_read_alignment_axes, 
        cds_context_name=cds_context_name, 
        nuccore_accession=nuccore_accession, longer_linked_repeat_cds_region=longer_linked_repeat_cds_region, 
        cds_polygon_edge_color_with_alpha=FIG4_CDS_POLYGON_EDGE_COLOR_WITH_ALPHA,
        product_class_to_fill_color_with_alpha=FIG4_PRODUCT_CLASS_TO_FILL_COLOR_WITH_ALPHA,
        non_ref_variant_colors=NON_REF_VARIANT_COLORS,
        num_of_axes_in_line=num_of_long_read_alignments_per_line,
        cds_arrowhead_width_as_fraction_of_ir_pair_region_with_margins=0.03,
    )
    if 1:
        fig.savefig(os.path.join(paper_figs_and_tables_dir_path, f'sup_fig_s{sup_fig_start_num + i}.png'))
        # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, f'sup_fig_s{sup_fig_start_num + i}.pdf'))

In [ ]:
for curr_df, excel_file_name, ordered_column_name_and_final_name_and_width, sheet_name in (
    (repeat_cds_product_fisher_or_g_result_df, 'table1.xlsx', TARGET_PRODUCT_ENRICHMENT_TABLE_ORDERED_COLUMN_NAME_AND_FINAL_NAME_AND_WIDTH, 'Table 1'),
    (adjacent_repeat_cds_product_fisher_or_g_result_df, 'table2.xlsx', NEIGHBOR_PRODUCT_ENRICHMENT_TABLE_ORDERED_COLUMN_NAME_AND_FINAL_NAME_AND_WIDTH, 'Table 2'),
):
    curr_df = curr_df[curr_df['corrected_pvalue'] <= search_for_pis_args[
        'enrichment_analysis']['max_corrected_pvalue_for_product_to_be_considered_significantly_enriched']].copy()
    curr_df.sort_values('odds_ratio', ascending=False, inplace=True)

    table_file_path = os.path.join(paper_figs_and_tables_dir_path, excel_file_name)

    write_xlsx_for_df(table_file_path, sheet_name, curr_df, ordered_column_name_and_final_name_and_width, 
                      column_name_to_num_format_specifier=PRODUCT_ENRICHMENT_COLUMN_NAME_TO_NUM_FORMAT_SPECIFIER)



In [ ]:
def add_location_str_repr_columns(df):
    if 'longer_linked_repeat_cds_start_pos' in df:
        df['longest target CDS location'] = (
            df['longer_linked_repeat_cds_start_pos'].astype(str) + '-' + 
            df['longer_linked_repeat_cds_end_pos'].astype(str)
        )
    
    if 'repeat1_cds_start_pos' in df:
        df['left target CDS location'] = (
            df['repeat1_cds_start_pos'].astype(str) + '-' + 
            df['repeat1_cds_end_pos'].astype(str)
        )
        df['right target CDS location'] = (
            df['repeat2_cds_start_pos'].astype(str) + '-' + 
            df['repeat2_cds_end_pos'].astype(str)
        )
    
    if 'repeat1_cds_operon_start' in df:
        df['left target CDS operon'] = (
            df['repeat1_cds_operon_start'].astype(int).astype(str) + '-' + 
            df['repeat1_cds_operon_end'].astype(int).astype(str)
        )
        df['right target CDS operon'] = (
            df['repeat2_cds_operon_start'].astype(int).astype(str) + '-' + 
            df['repeat2_cds_operon_end'].astype(int).astype(str)
        )
        
    if 'merged_cds_pair_region_start' in df:
        curr_df['PIC locus location'] = (
            curr_df['merged_cds_pair_region_start'].astype(str) + '-' + 
            curr_df['merged_cds_pair_region_end'].astype(str)
        )
        
    if 'region_in_other_nuccore_start' in df:
        curr_df['homologous locus location'] = (
            curr_df['region_in_other_nuccore_start'].astype(str) + '-' + 
            curr_df['region_in_other_nuccore_end'].astype(str)
        )
        
    if 'start_pos' in df:
        curr_df['CDS location'] = (
            curr_df['start_pos'].astype(str) + '-' + 
            curr_df['end_pos'].astype(str)
        )
    
    return df



ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF = pd.DataFrame(
    [x[1:2] + x[2] for x in ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED],
    columns=['nuccore_accession', 'longer_linked_repeat_cds_start_pos', 'longer_linked_repeat_cds_end_pos'],
)


def verify_sra_accession_and_short_read_name_matches_read_name(reads_df):
    for _, row in reads_df.iterrows():
        short_read_name = row['short_read_name']
        read_name = row['read_name']
        sra_accession = row['sra_accession']
        
        is_valid = (
            (read_name == f'{sra_accession}.1.{short_read_name}.1') or
            (read_name == f'{sra_accession}.1.{short_read_name}.3') or
            (read_name == f'{sra_accession}.1.{short_read_name}.5') or
            (read_name == f'{sra_accession}.1.{short_read_name}.7') or
            (read_name == f'{sra_accession}.1.{short_read_name}.9') or
            (read_name == f'{sra_accession}.3.{short_read_name}.1') or
            (read_name == f'{sra_accession}.3.{short_read_name}.3') or
            (read_name == f'{sra_accession}.3.{short_read_name}.5') or
            (read_name == f'{sra_accession}.3.{short_read_name}.7') or
            (read_name == f'{sra_accession}.3.{short_read_name}.9') or
            (read_name == f'{sra_accession}.1.{short_read_name}') or
            (read_name == f'{sra_accession}.man.{short_read_name}')
        )
        if not is_valid:
            print(read_name, sra_accession, short_read_name)
        assert is_valid
    

In [ ]:
curr_df = putative_pi_locus_descriptions_df.copy()

curr_df = ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF.merge(curr_df)
curr_df = add_location_str_repr_columns(curr_df)

table_file_path = os.path.join(paper_figs_and_tables_dir_path, 'table3.xlsx')

    
ordered_column_name_and_final_name_and_width = (
    ('nuccore_accession', 'NCBI Nucleotide accession', 22),
    ('longest target CDS location', 'longest target CDS location', 22),
    # ('longer_linked_repeat_cds_protein_id', 'longest target CDS NCBI Protein accession', 23),
    # ('num_of_identified_variants_including_ref', 'number of identified variants', 11),
    ('target_gene_product_description', 'longest target CDS product description', 60),
    ('locus_description_for_table_3', 'locus description', 80),
)

write_xlsx_for_df(table_file_path, 'Table 3', curr_df, ordered_column_name_and_final_name_and_width)

In [ ]:
# WRITE_BIG_TABLES = False
WRITE_BIG_TABLES = True
table_file_path = os.path.join(paper_figs_and_tables_dir_path, 'supplementary_tables.xlsx')
excel_writer = pd.ExcelWriter(table_file_path, engine='xlsxwriter')

curr_df = pd.DataFrame([
    ('Table S1', 'Systematically scanned species and their corresponding representative genome NCBI Assembly accessions'),
    ('Table S2', 'Homologous loci - loci homologous to programmed inversion candidate (PIC) loci found in same-species genomes'),
    ('Table S3', 'PICs for which intra-species variation was identified'),
    ('Table S4', 'All PICs and their genomic sequence attribute measures and predicted probability'),
    ('Table S5', 'Genbank coding sequence (CDS) product annotations tested for enrichment in predicted programmed '
                 'inversion targets'),
    ('Table S6', 'Genbank CDS product annotations tested for enrichment in predicted programmed '
                 'inversion target neighbors'),
    ('Table S7', 'Programmed inversion loci and their corresponding inverted repeats (presumably) promoting programmed inversions'),
    ('Table S8', 'DNA-seq Long-reads discarded due to anomalies identified during manual examination'),
    ('Table S9', 'DNA-seq Long-reads matching programmed inversion loci'),
    ('Table S10', 'GenBank CDS product annotations and their manually assigned gene families'),
    ('Table S11', 'Homologs of programmed inversion target CDSs'),
    ('Table S12', 'Previously described shufflon proteins (which were aligned to the shufflon and phage tail fusion-protein we identified)'),
    ('Table S13', 'Aggregate summary of RNA-seq reads uniquely matching the reference genome or the reference genome with an in-silico inverted region (flanked by inverted repeats)'),
    ('Table S14', 'RNA-seq reads uniquely matching the reference genome or the reference genome with an in-silico inverted region (flanked by inverted repeats)'),
], columns=['name', 'description'])

ordered_column_name_and_final_name_and_width = (
        ('name', 'name', 20),
        ('description', 'description', 80),
    )

write_xlsx_for_df(None, 'Legend', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)

if WRITE_BIG_TABLES:
    ordered_column_name_and_final_name_and_width = (
        ('taxon_species', 'species', 40),
        ('primary_assembly_accession', 'NCBI Assembly accession', 18),
    )

    write_xlsx_for_df(None, 'Supplementary Table S1', taxa_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)

if WRITE_BIG_TABLES:
    assert not region_in_other_nuccore_df['satisfied_mauve_total_match_proportion_and_min_sub_alignment_min_match_proportion_thresholds'].isna().any()
    curr_df = region_in_other_nuccore_df[region_in_other_nuccore_df['satisfied_mauve_total_match_proportion_and_min_sub_alignment_min_match_proportion_thresholds']].copy()
    curr_df['minus_mauve_total_match_proportion'] = -curr_df['mauve_total_match_proportion']
    curr_df.sort_values([
        'nuccore_accession', 
        'merged_cds_pair_region_start',
        'minus_mauve_total_match_proportion',
        'other_nuccore_accession',
        'region_in_other_nuccore_start',
    ], ascending=True, inplace=True)
    
    curr_df = add_location_str_repr_columns(curr_df)
    
    ordered_column_name_and_final_name_and_width = (
        ('nuccore_accession', 'PIC locus NCBI Nucleotide accession', 24),
        ('PIC locus location', 'PIC locus location', 22),
        ('other_nuccore_accession', 'homologous locus NCBI Nucleotide accession', 24),
        ('homologous locus location', 'homologous locus location', 22),
        ('mauve_total_match_proportion', 'match proportion of alignment between PIC locus and homologous locus', 15),
    )

    write_xlsx_for_df(None, 'Supplementary Table S2', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer, column_name_to_num_format_specifier={
        'mauve_total_match_proportion': '0.000',
    })

curr_df = filtered_minimal_ir_pairs_linked_to_breakpoint_pairs_df[[
    'nuccore_accession', 'index_in_nuccore_ir_pairs_df_csv_file', 'other_nuccore_accession',
    'region_in_other_nuccore_start', 'region_in_other_nuccore_end', 
]].merge(pairs_with_highest_confidence_bps_df[[
    'nuccore_accession', 'index_in_nuccore_ir_pairs_df_csv_file', 
    'repeat1_cds_start_pos',
    'repeat1_cds_end_pos',
    'repeat1_cds_product',
    'repeat2_cds_start_pos',
    'repeat2_cds_end_pos',
    'repeat2_cds_product',
    'high_confidence_bp_for_both_repeats',
]])
assert curr_df['high_confidence_bp_for_both_repeats'].all()

curr_df.sort_values([
    'nuccore_accession', 
    'repeat1_cds_start_pos',
    'other_nuccore_accession',
    'region_in_other_nuccore_start',
], ascending=True, inplace=True)

curr_df = add_location_str_repr_columns(curr_df)

ordered_column_name_and_final_name_and_width = (
    ('nuccore_accession', 'PIC NCBI Nucleotide accession', 24),
    ('left target CDS location', 'PIC left target CDS location', 22),
    ('right target CDS location', 'PIC right target CDS location', 22),
    ('repeat1_cds_product', 'PIC left target CDS product annotation', 25),
    ('repeat2_cds_product', 'PIC right target CDS product annotation', 25),
    ('other_nuccore_accession', 'homologous locus NCBI Nucleotide accession', 24),
    ('homologous locus location', 'homologous locus location', 22),
)

write_xlsx_for_df(None, 'Supplementary Table S3', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)
    
    
if WRITE_BIG_TABLES:
    curr_df = extended3_all_cds_pairs_df.copy()
    curr_df['minus_predicted_any_high_confidence_ir_pair_linked_to_cds_pair_probability'] = -curr_df[
        'predicted_any_high_confidence_ir_pair_linked_to_cds_pair_probability']
    
    curr_df.sort_values([
        'minus_predicted_any_high_confidence_ir_pair_linked_to_cds_pair_probability', 
        'repeat1_cds_product', 
        'repeat2_cds_product', 
        'nuccore_accession', 
        'repeat1_cds_start_pos',
    ], ascending=True, inplace=True)
    
    curr_df = add_location_str_repr_columns(curr_df)
    
    
    ordered_column_name_and_final_name_and_width = (
        ('nuccore_accession', 'NCBI Nucleotide accession', 22),
        ('left target CDS location', 'left target CDS location', 22),
        ('right target CDS location', 'right target CDS location', 22),
        ('predicted_any_high_confidence_ir_pair_linked_to_cds_pair_probability', 'predicted programmed inversion probability', 13),
        ('predicted_any_high_confidence_ir_pair_linked_to_cds_pair', 'predicted programmed inversion', 12),
        
        ('repeat1_cds_product', 'left target CDS product annotation', 25),
        ('repeat2_cds_product', 'right target CDS product annotation', 25),
        
        ('left target CDS operon', 'left target CDS operon', 22),
        ('right target CDS operon', 'right target CDS operon', 22),
        
        ('operon_spacer_len', 'CDS distance (bp)', 10),
        ('low_operon_spacer_len', 'short CDS distance', 10),
        ('max_repeat_len', 'repeat length (bp)', 8),
        ('high_max_repeat_len', 'long repeats', 8),
        ('operon_asymmetry', 'asymmetry', 11),
        ('high_operon_asymmetry', 'high asymmetry', 11),
        ('operon_closest_repeat_position_orientation_matching', 'orientation matching', 12),
        ('high_operon_closest_repeat_position_orientation_matching', 'high orientation matching', 12),
    )

    write_xlsx_for_df(None, 'Supplementary Table S4', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer, column_name_to_num_format_specifier={
        'predicted_any_high_confidence_ir_pair_linked_to_cds_pair_probability': '0.000',
        'operon_asymmetry': '0.000',
        'operon_closest_repeat_position_orientation_matching': '0.000',
    })
    

for curr_df, ordered_column_name_and_final_name_and_width, sheet_name in (
    (repeat_cds_product_fisher_or_g_result_df, TARGET_PRODUCT_ENRICHMENT_TABLE_ORDERED_COLUMN_NAME_AND_FINAL_NAME_AND_WIDTH, 'Supplementary Table S5'),
    (adjacent_repeat_cds_product_fisher_or_g_result_df, NEIGHBOR_PRODUCT_ENRICHMENT_TABLE_ORDERED_COLUMN_NAME_AND_FINAL_NAME_AND_WIDTH, 'Supplementary Table S6'),
):
    curr_df = curr_df.copy()
    curr_df['minus_odds_ratio'] = -curr_df['odds_ratio']
    curr_df.sort_values(['corrected_pvalue', 'minus_odds_ratio'], ascending=True, inplace=True)

    write_xlsx_for_df(None, sheet_name, curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer,
                      column_name_to_num_format_specifier=PRODUCT_ENRICHMENT_COLUMN_NAME_TO_NUM_FORMAT_SPECIFIER)


sra_accession_to_type_and_sra_file_name = search_for_pis_args['stage6']['sra_accession_to_type_and_sra_file_name']

read_ends_with_expected_suffix = lambda x: x.rsplit('.')[-1] in (str(x) for x in (1,3,5,7,9))

variant_ir_pair_flat_dicts = []
anomalous_read_flat_dicts = []
non_ref_read_flat_dicts = []
for nuccore_accession_to_alignment_regions_raw_read_alignment_args in search_for_pis_args['stage6'][
    'cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args'
].values():
    for nuccore_accession, alignment_regions_raw_read_alignment_args in nuccore_accession_to_alignment_regions_raw_read_alignment_args.items():
        for alignment_region_raw_read_alignment_args in alignment_regions_raw_read_alignment_args:
            sra_accession_to_variants_and_reads_info = alignment_region_raw_read_alignment_args[
                'sra_accession_to_variants_and_reads_info']
            assert len(sra_accession_to_variants_and_reads_info) == 1
            sra_accession = next(iter(
                sra_accession_to_variants_and_reads_info.keys()))
            reads_info = next(iter(
                sra_accession_to_variants_and_reads_info.values()))
            sra_file_name = sra_accession_to_type_and_sra_file_name[sra_accession][1]
            read_name_prefix_len = len(sra_file_name) + 1
            # print(f'sra_file_name: {sra_file_name}')
            # print(f'read_name_prefix_len: {read_name_prefix_len}')
            
            non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref = reads_info[
                'non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref']
            non_ref_variant_ir_pairs_to_num_of_evidence_reads = get_variant_ir_pairs_to_num_of_evidence_reads(
                non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref)
            longer_linked_repeat_cds_region = alignment_region_raw_read_alignment_args['longest_linked_repeat_cds_region_and_protein_id'][0]
            sorted_non_ref_variant_ir_pairs_and_num_of_evidence_reads = sorted(non_ref_variant_ir_pairs_to_num_of_evidence_reads.items(), 
                                                                               reverse=True, key=lambda x: x[1])
            for non_ref_variant_ir_pairs, num_of_evidence_reads in sorted_non_ref_variant_ir_pairs_and_num_of_evidence_reads:
                variant_ir_pair_repr = massive_screening_configuration.get_variant_ir_pair_str_repr(non_ref_variant_ir_pairs)
                variant_ir_pair_flat_dicts.append({
                    'nuccore_accession': nuccore_accession,
                    'longer_linked_repeat_cds_start_pos': longer_linked_repeat_cds_region[0],
                    'longer_linked_repeat_cds_end_pos': longer_linked_repeat_cds_region[1],
                    'variant_ir_pair_repr': variant_ir_pair_repr,
                    'number of reads matching this variant': num_of_evidence_reads,
                })
                
                sorted_evidence_read_names = sorted(x for x in non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref
                                                    if non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref[x] == non_ref_variant_ir_pairs)
                assert len(sorted_evidence_read_names) == num_of_evidence_reads
                # print([x for x in sorted_evidence_read_names if not read_ends_with_expected_suffix(x)])
                assert all(read_ends_with_expected_suffix(x) for x in sorted_evidence_read_names)
                sorted_evidence_short_read_names = [x[read_name_prefix_len:-2] for x in sorted_evidence_read_names]
                
                non_ref_read_flat_dicts.extend([
                    {
                        'nuccore_accession': nuccore_accession,
                        'longer_linked_repeat_cds_start_pos': longer_linked_repeat_cds_region[0],
                        'longer_linked_repeat_cds_end_pos': longer_linked_repeat_cds_region[1],
                        'variant_ir_pair_repr': variant_ir_pair_repr, 
                        'read_name': read_name, 
                        'sra_accession': sra_accession, 
                        'short_read_name': short_read_name, 
                    }
                    for read_name, short_read_name in zip(sorted_evidence_read_names, sorted_evidence_short_read_names)
                ])
                
            if 'not_non_ref_variant_read_name_to_anomaly_description' in reads_info:
                not_non_ref_variant_read_name_to_anomaly_description = reads_info['not_non_ref_variant_read_name_to_anomaly_description']

                sorted_not_non_ref_variant_read_names_and_anomaly_descs = sorted(not_non_ref_variant_read_name_to_anomaly_description.items())
                assert all(x.endswith('.1') or x.endswith('.3') for x, _ in sorted_not_non_ref_variant_read_names_and_anomaly_descs)
                # print(read_name_prefix_len)
                sorted_not_non_ref_short_read_names = [x[read_name_prefix_len:-2] for x, _ in sorted_not_non_ref_variant_read_names_and_anomaly_descs]
                
                anomalous_read_flat_dicts.extend([
                    {
                        'nuccore_accession': nuccore_accession,
                        'longer_linked_repeat_cds_start_pos': longer_linked_repeat_cds_region[0],
                        'longer_linked_repeat_cds_end_pos': longer_linked_repeat_cds_region[1],
                        'read_name': read_name, 
                        'sra_accession': sra_accession, 
                        'short_read_name': short_read_name, 
                        'anomaly_description': anomaly_description,
                    }
                    for (read_name, anomaly_description), short_read_name in zip(sorted_not_non_ref_variant_read_names_and_anomaly_descs, 
                                                                                 sorted_not_non_ref_short_read_names)
                ])


curr_df = ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF.merge(pd.DataFrame(variant_ir_pair_flat_dicts))
assert (len(curr_df[list(ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF)].drop_duplicates()) == 
        len(ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF))
curr_df = add_location_str_repr_columns(curr_df)
ordered_column_name_and_final_name_and_width = (
    ('nuccore_accession', 'NCBI Nucleotide accession', 22),
    ('longest target CDS location', 'longest target CDS location', 22),
    ('variant_ir_pair_repr', 'inverted repeats (presumably) promoting programmed inversions allowing switching from the reference variant to this variant', 50),
    ('number of reads matching this variant', 'number of reads matching this variant', 14),
)
num_of_identified_pilv_phage_tail_collar_variants = 26
assert (curr_df[curr_df['nuccore_accession'] == 'CP084655.1']['variant_ir_pair_repr'].nunique() + 1) == num_of_identified_pilv_phage_tail_collar_variants
print(f'made sure that including the reference variant, we identified {num_of_identified_pilv_phage_tail_collar_variants} '
      'variants for the gene encoding PilV and phage tail collar fusion protein')

write_xlsx_for_df(None, 'Supplementary Table S7', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)


curr_df = ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF.merge(pd.DataFrame(anomalous_read_flat_dicts))
curr_df = add_location_str_repr_columns(curr_df)
verify_sra_accession_and_short_read_name_matches_read_name(curr_df)
ordered_column_name_and_final_name_and_width = (
    ('nuccore_accession', 'NCBI Nucleotide accession', 22),
    ('longest target CDS location', 'longest target CDS location', 22),
    ('sra_accession', 'NCBI SRA accession', 20),
    ('short_read_name', 'read name', 20),
    ('anomaly_description', 'anomaly description', 80),
)
write_xlsx_for_df(None, 'Supplementary Table S8', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)
    

curr_df = ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF.merge(pd.DataFrame(non_ref_read_flat_dicts))
assert (len(curr_df[list(ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF)].drop_duplicates()) == 
        len(ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF))
curr_df = add_location_str_repr_columns(curr_df)
verify_sra_accession_and_short_read_name_matches_read_name(curr_df)
ordered_column_name_and_final_name_and_width = (
    ('nuccore_accession', 'NCBI Nucleotide accession', 22),
    ('longest target CDS location', 'longest target CDS location', 22),
    ('variant_ir_pair_repr', 'inverted repeats (presumably) promoting programmed inversions allowing switching from the reference variant to this variant', 50),
    ('sra_accession', 'NCBI SRA accession', 20),
    ('short_read_name', 'read name', 20),
)
write_xlsx_for_df(None, 'Supplementary Table S9', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)
    
    
products_series = pd.concat(
    [
        repeat_cds_product_fisher_or_g_result_df[repeat_cds_product_fisher_or_g_result_df['corrected_pvalue'] <= search_for_pis_args[
            'enrichment_analysis']['max_corrected_pvalue_for_product_to_be_considered_significantly_enriched']]['product'],
        adjacent_repeat_cds_product_fisher_or_g_result_df[adjacent_repeat_cds_product_fisher_or_g_result_df['corrected_pvalue'] <= search_for_pis_args[
            'enrichment_analysis']['max_corrected_pvalue_for_product_to_be_considered_significantly_enriched']]['product'],
    ] + [pd.read_csv(x, sep='\t', low_memory=False)['product'] for x in all_raw_read_alignment_results_df['filtered_extended_cds_df_csv_file_path'].unique()],
    ignore_index=True,
).drop_duplicates().sort_values()
product_classes_df = products_series.to_frame()
product_classes_df = add_product_class_column(product_classes_df, 'product', 'product_class')

product_classes_df = product_classes_df[(product_classes_df['product_class'] != 'other')].copy()
product_classes_df.sort_values(['product_class', 'product'], inplace=True)

ordered_column_name_and_final_name_and_width = (
    ('product', 'GenBank CDS product annotation', 60),
    ('product_class', 'gene family', max(len(x) for x in product_classes_df['product_class'].unique())),
)

write_xlsx_for_df(None, 'Supplementary Table S10', product_classes_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)


curr_df = all_homologs_extended_df.copy()

def get_homolog_ir_pair_str_repr(homolog_df_row):
    ir_pair = tuple(homolog_df_row[[
        'inverted_repeat_in_homolog_left',
        'inverted_repeat_in_homolog_right',
        'inverted_repeat_in_margin_left',
        'inverted_repeat_in_margin_right',
    ]].astype(int))
    if ir_pair[2] < ir_pair[1]:
        ir_pair = ir_pair[2:] + ir_pair[:2]
    assert ir_pair[0] < ir_pair[1] < ir_pair[2] < ir_pair[3]
    return massive_screening_configuration.get_ir_pair_str_repr(ir_pair)[1:-1]

curr_df['min_evalue_repr'] = curr_df['min_evalue_of_alignment_to_inverted_repeat_in_margin'].apply(
    lambda x: '' if np.isposinf(x) else f'{x:.2E}')
curr_df['inverted_repeat_repr'] = curr_df.apply(
    lambda row: ('no inverted repeats found' if row['min_evalue_repr'] == '' 
                 else get_homolog_ir_pair_str_repr(row)), 
    axis=1,
)
curr_df = add_location_str_repr_columns(curr_df)

curr_df.sort_values(['taxon_phylum', 'taxon_species', 'nuccore_accession', 'CDS location'], inplace=True)
all_cds_context_names_and_longer_linked_cds_loci_ordered_df_with_curr_column_names = (
    ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF.rename(columns={
        'nuccore_accession': 'longer_linked_repeat_cds_nuccore_accession',
    }))
curr_df = all_cds_context_names_and_longer_linked_cds_loci_ordered_df_with_curr_column_names.merge(curr_df)
ordered_column_name_and_final_name_and_width = (
    ('longer_linked_repeat_cds_nuccore_accession', 'programmed inversion NCBI Nucleotide accession', 22),
    ('longest target CDS location', 'programmed inversion longest target CDS location', 22),
    ('taxon_phylum', 'homolog CDS phylum', 22),
    ('taxon_species', 'homolog CDS species', 22),
    ('nuccore_accession', 'homolog CDS NCBI Nucleotide accession', 24),
    ('CDS location', 'homolog CDS location', 22),
    ('product', 'homolog GenBank CDS product annotation', 60),
    ('is_homolog_potentially_modified_by_pi', 'is homolog considered as potentially targeted by programmed inversions', 20),
    ('inverted_repeat_repr', 'best inverted repeats between homolog CDS and its margins', 50),
    ('min_evalue_repr', 'alignment evalue of best inverted repeats between homolog CDS and its margins', 20),
)

write_xlsx_for_df(None, 'Supplementary Table S11', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)

shufflon_protein_df = pd.DataFrame(
    [
        ('WP_001389385.1', 'Salmonella enterica subsp. enterica serovar Typhimurium plasmid R64', 'AP005147.1', '104790-106214'),
        ('WP_001389385.1', 'Shigella sonnei plasmid P9', 'NC_002122.1', '77407-78831'),
        ('WP_010895887.1', 'Escherichia coli plasmid R721', 'NC_002525.1', '44733-45986'),
        ('AAO71709.1', 'Salmonella enterica subsp. enterica serovar Typhi Ty2', 'AE014613.1', '4418334-4419641'),
    ],
    columns=['NCBI Protein accession', 'Genome description', 'NCBI Nucleotide accession', 'CDS location'], 
)

ordered_column_name_and_final_name_and_width = (
    ('NCBI Protein accession', 'NCBI Protein accession', 16),
    ('Genome description', 'Genome description', 60),
    ('NCBI Nucleotide accession', 'NCBI Nucleotide accession', 16),
    ('CDS location', 'CDS location', 19),
)

write_xlsx_for_df(None, 'Supplementary Table S12', shufflon_protein_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)


curr_df = ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF.merge(semi_min_rna_seq_summary_df)
curr_df = add_location_str_repr_columns(curr_df)
ordered_column_name_and_final_name_and_width = (
    ('nuccore_accession', 'NCBI Nucleotide accession', 22),
    ('longest target CDS location', 'longest target CDS location', 22),
    ('sra_accession', 'NCBI SRA accession', 20),
    ('in_silico_inverted_region', 'predicted invertible region', 22),
    ('num_of_reads_matching_ref_better', 'number of read pairs matching the reference substantially better than the alternative reference (>=100 difference between bowtie2 alignment scores to reference and alternative reference)', 26),
    ('num_of_reads_matching_non_ref_better', 'number of read pairs matching the alternative reference substantially better than the reference (>=100 difference between bowtie2 alignment scores to alternative reference and reference)', 26),
)
write_xlsx_for_df(None, 'Supplementary Table S13', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)

curr_df = ALL_CDS_CONTEXT_NAMES_AND_LONGER_LINKED_CDS_LOCI_ORDERED_DF.merge(semi_min_all_all_diff_score_alignments_df)
curr_df = add_location_str_repr_columns(curr_df)
for score_column_name in ('pair_total_align_score_ref', 'pair_total_align_score_non_ref'):
    curr_df[score_column_name].replace(-np.inf, 'no concordant alignment identified', inplace=True)
    curr_df[score_column_name].replace(0, '0 (sequences match perfectly)', inplace=True)
verify_sra_accession_and_short_read_name_matches_read_name(curr_df)
ordered_column_name_and_final_name_and_width = (
    ('nuccore_accession', 'NCBI Nucleotide accession', 22),
    ('longest target CDS location', 'longest target CDS location', 22),
    ('sra_accession', 'NCBI SRA accession', 20),
    ('in_silico_inverted_region', 'predicted invertible region', 22),
    ('short_read_name', 'read name', 20),
    ('pair_total_align_score_ref', 'score of concordant alignment to the reference', 35),
    ('pair_total_align_score_non_ref', 'score of concordant alignment to the alternative reference', 35),
    # pair_total_align_score_ref  pair_total_align_score_non_ref  score_diff  max_score  is_read_matching_ref_better in_silico_inverted_region
)
write_xlsx_for_df(None, 'Supplementary Table S14', curr_df, ordered_column_name_and_final_name_and_width, writer=excel_writer)


excel_writer.save()


In [ ]:
num_of_ref_rhamnosus_reads = all_raw_read_alignment_results_df[
    (all_raw_read_alignment_results_df['nuccore_accession'] == 'NC_013198.1') & 
    (all_raw_read_alignment_results_df['cds_context_name'] == 'MTase: BREX type 1') & 
    (all_raw_read_alignment_results_df['read_evidence_type'] == 'ref_variant')
]['sseqid'].nunique()
total_num_of_rhamnosus_reads = 176
print(f'num_of_ref_rhamnosus_reads: {num_of_ref_rhamnosus_reads}')

rhamnosus_alignment_regions_raw_read_alignment_args = search_for_pis_args['stage6'][
    'cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args']['MTase: BREX type 1']['NC_013198.1'][0]
sra_accession_to_variants_and_reads_info = rhamnosus_alignment_regions_raw_read_alignment_args['sra_accession_to_variants_and_reads_info']
assert len(sra_accession_to_variants_and_reads_info) == 1
reads_info = next(iter(sra_accession_to_variants_and_reads_info.values()))
non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref = reads_info[
    'non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref']
non_ref_variant_ir_pairs_to_num_of_evidence_reads = get_variant_ir_pairs_to_num_of_evidence_reads(
    non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref)
assert sum(non_ref_variant_ir_pairs_to_num_of_evidence_reads.values()) + num_of_ref_rhamnosus_reads == total_num_of_rhamnosus_reads

all_rhamnosus_ir_pairs = set()
for ir_pairs in non_ref_variant_ir_pairs_to_num_of_evidence_reads:
    all_rhamnosus_ir_pairs |= set(ir_pairs)
min_left2 = min(x[2] for x in all_rhamnosus_ir_pairs)
max_right2 = max(x[3] for x in all_rhamnosus_ir_pairs)
(rhamnosus_pglx_start, rhamnosus_pglx_end), rhamnosus_pglx_protein_id = rhamnosus_alignment_regions_raw_read_alignment_args[
    'longest_linked_repeat_cds_region_and_protein_id']
long_region_upstream_to_all_repeats_len = rhamnosus_pglx_end - max_right2
long_region_downstream_to_all_repeats_len = min_left2 - rhamnosus_pglx_start
rhamnosus_pglx_gene_len = rhamnosus_pglx_end - rhamnosus_pglx_start + 1
assert (rhamnosus_pglx_gene_len % 3) == 0
rhamnosus_pglx_len = rhamnosus_pglx_gene_len // 3 - 1 # don't count stop codon
rhamnosus_n_terminus_len = long_region_upstream_to_all_repeats_len // 3
rhamnosus_c_terminus_len = long_region_downstream_to_all_repeats_len // 3 - 1 # don't count stop codon
rhamnosus_var_part_len = rhamnosus_pglx_len - rhamnosus_n_terminus_len - rhamnosus_c_terminus_len
rhamnosus_var_part_start = rhamnosus_n_terminus_len + 1
rhamnosus_var_part_end = rhamnosus_var_part_start + rhamnosus_var_part_len - 1
rhamnosus_c_terminus_start = rhamnosus_var_part_end + 1

print(f'1-{rhamnosus_n_terminus_len}')
print(f'{rhamnosus_var_part_start}-{rhamnosus_var_part_end}')
print(f'{rhamnosus_c_terminus_start}-{rhamnosus_pglx_len}')

casei_alignment_regions_raw_read_alignment_args = search_for_pis_args['stage6'][
        'cds_context_name_to_nuccore_accession_to_alignment_regions_raw_read_alignment_args'][
    'MTase: BREX type 1, downstream extra short PglX']['NZ_CP068173.1'][0]
casei_sra_accession_to_variants_and_reads_info = casei_alignment_regions_raw_read_alignment_args['sra_accession_to_variants_and_reads_info']
assert len(casei_sra_accession_to_variants_and_reads_info) == 1
casei_reads_info = next(iter(casei_sra_accession_to_variants_and_reads_info.values()))
casei_non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref = casei_reads_info[
    'non_ref_variant_read_name_to_possible_ir_pairs_used_to_reach_from_ref']
casei_non_ref_variant_ir_pairs_to_num_of_evidence_reads = get_variant_ir_pairs_to_num_of_evidence_reads(
    casei_non_ref_read_name_to_possible_ir_pairs_used_to_reach_from_ref)
all_casei_ir_pairs = set()
for ir_pairs in casei_non_ref_variant_ir_pairs_to_num_of_evidence_reads:
    all_casei_ir_pairs |= set(ir_pairs)
casei_max_right2 = max(x[3] for x in all_casei_ir_pairs)
(casei_pglx_start, casei_pglx_end), casei_pglx_protein_id = casei_alignment_regions_raw_read_alignment_args[
    'longest_linked_repeat_cds_region_and_protein_id']
casei_region_upstream_to_all_repeats_len = casei_pglx_end - casei_max_right2
casei_n_terminus_len = casei_region_upstream_to_all_repeats_len // 3
casei_pglx_gene_len = casei_pglx_end - casei_pglx_start + 1
assert (casei_pglx_gene_len % 3) == 0
casei_pglx_len = casei_pglx_gene_len // 3 - 1 # don't count stop codon
casei_var_part_len = casei_pglx_len - casei_n_terminus_len
casei_var_part_start = casei_n_terminus_len + 1
casei_downstream_pglx_gene_len = 948
assert (casei_downstream_pglx_gene_len % 3) == 0
casei_downstream_pglx_len = casei_downstream_pglx_gene_len // 3

# online blastp rhamnosus long PglX (WP_015765014.1) to casei long PglX (WP_201670672.1)
main_casei_alignment_start_in_rhamnosus = 1
main_casei_alignment_start_in_casei = 1
main_casei_alignment_end_in_rhamnosus = 878
main_casei_alignment_end_in_casei = 859

# online blastp rhamnosus long PglX (WP_015765014.1) to casei downstream PglX (WP_236586838.1)
downstream_casei_alignment_start_in_rhamnosus = 885
downstream_casei_alignment_start_in_casei = 3
downstream_casei_alignment_end_in_rhamnosus = 1172
downstream_casei_alignment_end_in_casei = 299
downstream_casei_protein_id = 'WP_236586838.1'

rhamnosus_pglx_start = 74
rhamnosus_top = 0.8
protein_height = 0.1
dist_between_rhamnosus_and_casei = 0.3
dist_between_casei_pglx_proteins = 150
show_n_and_c_for_each_protein = True
show_protein_ids = True
y_dist_between_protein_and_id = 0.02
species_text_start = -80

if 1:
    plt.close('all')
fig, ax = plt.subplots(figsize=(8,4))

fig.subplots_adjust(
    top=1,
    bottom=0.025,
    left=0.27,
    right=0.99,
    # hspace=0.2,
    # wspace=0.2
)

remove_all_ax_spines_an_x_and_y_axes(ax)


rhamnosus_bottom = rhamnosus_top - protein_height
rhamnosus_middle_y = np.mean((rhamnosus_bottom, rhamnosus_top))
ax.add_patch(matplotlib.patches.Rectangle(
    (rhamnosus_pglx_start, rhamnosus_bottom), rhamnosus_pglx_len, protein_height,
    fill=False, 
))
if show_n_and_c_for_each_protein:
    ax.text(rhamnosus_pglx_start, rhamnosus_middle_y, 'N-', va='center', ha='right')
    ax.text(rhamnosus_pglx_start + rhamnosus_pglx_len, rhamnosus_middle_y, '-C', va='center', ha='left')
if show_protein_ids:
    ax.text(np.mean((rhamnosus_pglx_start, rhamnosus_pglx_start + rhamnosus_pglx_len)), 
            rhamnosus_top + y_dist_between_protein_and_id, rhamnosus_pglx_protein_id, va='bottom', ha='center')
ax.add_patch(matplotlib.patches.Rectangle(
    (rhamnosus_pglx_start + rhamnosus_var_part_start, rhamnosus_bottom), rhamnosus_var_part_len, protein_height,
    fill=False, 
    hatch='.....',
    linewidth=0, 
))

casei_top = rhamnosus_bottom - dist_between_rhamnosus_and_casei
casei_bottom = casei_top - protein_height
casei_downstream_pglx_start = casei_pglx_len + dist_between_casei_pglx_proteins
casei_downstream_pglx_end = casei_downstream_pglx_start + casei_downstream_pglx_len
casei_middle_y = np.mean((casei_bottom, casei_top))

ax.add_patch(matplotlib.patches.Rectangle(
    (1, casei_bottom), casei_pglx_len, protein_height,
    fill=False, 
))
var_part_handle = ax.add_patch(matplotlib.patches.Rectangle(
    (casei_var_part_start, casei_bottom), casei_var_part_len, protein_height,
    fill=False, 
    hatch='.....',
    linewidth=0, 
))
if show_n_and_c_for_each_protein:
    ax.text(1, casei_middle_y, 'N-', va='center', ha='right')
    ax.text(1 + casei_pglx_len, casei_middle_y, '-C', va='center', ha='left')
if show_protein_ids:
    ax.text(np.mean((1, 1 + casei_pglx_len)), 
            casei_bottom - y_dist_between_protein_and_id, casei_pglx_protein_id, va='top', ha='center')
ax.add_patch(matplotlib.patches.Rectangle(
    (casei_downstream_pglx_start, casei_bottom), casei_downstream_pglx_len, protein_height,
    fill=False, 
))
if show_n_and_c_for_each_protein:
    ax.text(casei_downstream_pglx_start, casei_middle_y, 'N-', va='center', ha='right')
    ax.text(casei_downstream_pglx_start + casei_downstream_pglx_len, casei_middle_y, '-C', va='center', ha='left')
if show_protein_ids:
    ax.text(np.mean((casei_downstream_pglx_start, casei_downstream_pglx_start + casei_downstream_pglx_len)), 
            casei_bottom - y_dist_between_protein_and_id, downstream_casei_protein_id, va='top', ha='center')
    


ax.add_patch(matplotlib.patches.Polygon([
    (rhamnosus_pglx_start + main_casei_alignment_start_in_rhamnosus, rhamnosus_bottom), 
    (rhamnosus_pglx_start + main_casei_alignment_end_in_rhamnosus, rhamnosus_bottom), 
    (main_casei_alignment_end_in_casei, casei_top), 
    (main_casei_alignment_start_in_casei, casei_top), 
], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
    linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))

ax.add_patch(matplotlib.patches.Polygon([
    (rhamnosus_pglx_start + downstream_casei_alignment_start_in_rhamnosus, rhamnosus_bottom), 
    (rhamnosus_pglx_start + downstream_casei_alignment_end_in_rhamnosus, rhamnosus_bottom), 
    (downstream_casei_alignment_end_in_casei + casei_downstream_pglx_start, casei_top), 
    (downstream_casei_alignment_start_in_casei + casei_downstream_pglx_start, casei_top), 
], closed=True, fill=True, facecolor=ALIGNMENT_PROJECTION_COLOR, alpha=ALIGNMENT_PROJECTION_ALPHA, edgecolor=ALIGNMENT_PROJECTION_EDGE_COLOR,
    linewidth=ALIGNMENT_PROJECTION_LINE_WIDTH))


ax.set_xlim(-100, casei_downstream_pglx_end + 100)
ax.text(species_text_start, rhamnosus_middle_y, rhamnosus_alignment_regions_raw_read_alignment_args['locus_definition'], va='center', ha='right')
ax.text(species_text_start, casei_middle_y, casei_alignment_regions_raw_read_alignment_args['locus_definition'], va='center', ha='right')

ax.legend(
    handles=[var_part_handle],
    labels=['variable region'],
    # fontsize='small', 
    bbox_to_anchor=(
        0.977, 
        0.05,
    ),
    handleheight=2, handlelength=4,
    loc='lower right',
)
if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s33.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s33.pdf'))


total_num_of_even_non_ref_reads = sum(
    num_of_evidence_reads for non_ref_variant_ir_pairs, num_of_evidence_reads in non_ref_variant_ir_pairs_to_num_of_evidence_reads.items()
    if (len(non_ref_variant_ir_pairs) % 2) == 0
)
total_num_of_odd_non_ref_reads = sum(
    num_of_evidence_reads for non_ref_variant_ir_pairs, num_of_evidence_reads in non_ref_variant_ir_pairs_to_num_of_evidence_reads.items()
    if (len(non_ref_variant_ir_pairs) % 2) == 1
)
total_num_of_non_ref_reads = total_num_of_even_non_ref_reads + total_num_of_odd_non_ref_reads
assert total_num_of_non_ref_reads + num_of_ref_rhamnosus_reads == total_num_of_rhamnosus_reads

print(f'all_rhamnosus_ir_pairs: {all_rhamnosus_ir_pairs}')
print(f'num_of_ref_rhamnosus_reads: {num_of_ref_rhamnosus_reads}')
print(f'total_num_of_non_ref_reads: {total_num_of_non_ref_reads}')
print(f'total_num_of_even_non_ref_reads: {total_num_of_even_non_ref_reads}')
print(f'total_num_of_odd_non_ref_reads: {total_num_of_odd_non_ref_reads}')
print(f'long_region_upstream_to_all_repeats_len: {long_region_upstream_to_all_repeats_len}')
print(f'long_region_downstream_to_all_repeats_len: {long_region_downstream_to_all_repeats_len}')

binom_test_result_for_all_reads = scipy.stats.binom_test(
    x=total_num_of_odd_non_ref_reads, 
    n=total_num_of_rhamnosus_reads, 
    p=0.5, alternative='two-sided',
)
binom_test_result_for_non_ref_reads = scipy.stats.binom_test(
    x=total_num_of_odd_non_ref_reads, 
    n=total_num_of_non_ref_reads, 
    p=0.5, alternative='two-sided',
)
print(f'binom_test_result_for_all_reads: {binom_test_result_for_all_reads}')
print(f'binom_test_result_for_non_ref_reads: {binom_test_result_for_non_ref_reads}')



In [ ]:
if 1:
    plt.close('all')

curr_df = pairs_linked_with_evidence_for_repeat_overlapping_mobile_element_df.copy()
column_name = 'repeat_estimated_copy_num'

curr_df = curr_df[~(curr_df[column_name].isna())]
fig, ax = plt.subplots(figsize=(6,4))

orig_sample_size = len(curr_df)


min_val = curr_df[column_name].min()
max_val = curr_df[column_name].max()
print(f'max_val: {max_val}')


real_bins = list(range(31))
first_real_bin_start = real_bins[0]
last_real_bin_end = real_bins[-1]
bin_size = 1
fake_last_bin_end = last_real_bin_end + bin_size
fake_last_bin_middle = fake_last_bin_end - bin_size / 2

# print(curr_df[column_name].describe())

curr_df.loc[curr_df[column_name] > last_real_bin_end, column_name] = fake_last_bin_middle

bins = real_bins + [fake_last_bin_end]
assert curr_df[column_name].min() >= bins[0]
assert curr_df[column_name].max() <= bins[-1]
assert ((curr_df[column_name] <= bins[-1]) & (curr_df[column_name] >= bins[0])).sum() == orig_sample_size

hist_color = (*matplotlib.colors.to_rgb('grey'), 1)
hist_color = 'grey'
hist_color = (0.7,0.7,0.7)
hist_color = GREY_HIST_COLOR

ax.hist(
    curr_df[column_name],
    bins=bins,
    # density=True,
    # histtype='step', 
    # linewidth=8,
    color=hist_color,
    edgecolor=hist_color,
)
ax.set_yscale('log')
# ax.set_xlabel('repeat estimated copy number outside CDS pair locus')
ax.set_xlabel('repeat estimated copy number outside inverted repeat pair locus')
# ax.set_ylabel('# CDS pairs')
ax.set_ylabel('# coding sequence pairs')
# orig_xlim = ax.get_xlim()
ax.set_xlim(bins[0], bins[-1])

add_fake_bin_tick_labels_and_dashed_lines_to_histogram(
    ax, df=curr_df, column_name=column_name, bin_size=bin_size, 
    last_real_bin_end=last_real_bin_end, 
    first_real_bin_start=first_real_bin_start,
)

ylim = ax.get_ylim()
ax.plot([search_for_pis_args['stage3']['min_max_estimated_copy_num_to_classify_as_mobile_element']] * 2, ylim,
        linestyle='--',color='red',
        # label=f'x=0',
        alpha=THRESHOLD_DASHED_LINE_ALPHA)
ax.set_ylim(ylim)

if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s1.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s1.pdf'))

In [ ]:
if 1:
    plt.close('all')

curr_df = lens_of_spanning_regions_ratios_df.copy()

curr_df['lens_of_spanning_regions_ratio_and_1'] = (curr_df['lens_of_spanning_regions_ratio'] - 1).abs()
column_name = 'lens_of_spanning_regions_ratio_and_1'

assert not curr_df[column_name].isna().any()
fig, ax = plt.subplots(figsize=(6,4))

orig_sample_size = len(curr_df)

min_val = curr_df[column_name].min()
max_val = curr_df[column_name].max()

bin_size = 0.01
first_real_bin_start = 0
last_real_bin_end = 0.5
fake_last_bin_end = last_real_bin_end + bin_size
fake_last_bin_middle = fake_last_bin_end - bin_size / 2
real_bins = list(np.arange(first_real_bin_start, last_real_bin_end + bin_size, bin_size))

# print(curr_df[column_name].describe())

curr_df.loc[curr_df[column_name] > last_real_bin_end, column_name] = fake_last_bin_middle

bins = real_bins + [fake_last_bin_end]
assert curr_df[column_name].min() >= bins[0]
assert curr_df[column_name].max() <= bins[-1]
assert ((curr_df[column_name] <= bins[-1]) & (curr_df[column_name] >= bins[0])).sum() == orig_sample_size

hist_color = GREY_HIST_COLOR

ax.hist(
    curr_df[column_name],
    bins=bins,
    # density=True,
    # histtype='step', 
    # linewidth=8,
    color=hist_color,
    edgecolor=hist_color,
)
ax.set_yscale('log')
ax.set_xlabel('relative locus length difference')
ax.set_ylabel(
    # '# collinear left and right margin alignment pairs',
    # '# collinear alignment pairs',
    '# similar length locus candidates',
)
# orig_xlim = ax.get_xlim()
ax.set_xlim(bins[0], bins[-1])

add_fake_bin_tick_labels_and_dashed_lines_to_histogram(
    ax, df=curr_df, column_name=column_name, bin_size=bin_size, 
    last_real_bin_end=last_real_bin_end, 
    first_real_bin_start=first_real_bin_start,
)

ylim = ax.get_ylim()
ax.plot([search_for_pis_args['stage5']['blast_margins_and_identify_regions_in_other_nuccores'][
    'max_dist_between_lens_of_spanning_regions_ratio_and_1']] * 2, ylim,
        linestyle='--',color='red',
        # label=f'x=0',
        alpha=THRESHOLD_DASHED_LINE_ALPHA)
ax.set_ylim(ylim)


if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s2.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s2.pdf'))

In [ ]:
if 1:
    plt.close('all')

curr_df = region_in_other_nuccore_df.copy()

column_name = 'mauve_total_match_proportion'

assert not curr_df[column_name].isna().any()
fig, ax = plt.subplots(figsize=(6,4))

bin_size = 0.01
bins = list(np.arange(0, 1 + bin_size, bin_size))
assert bins[-1] == 1

# print(curr_df[column_name].describe())

hist_color = GREY_HIST_COLOR

ax.hist(
    curr_df[column_name],
    bins=bins,
    # density=True,
    # histtype='step', 
    # linewidth=8,
    color=hist_color,
    edgecolor=hist_color,
)
ax.set_yscale('log')
ax.set_xlabel('alignment match proportion')
ax.set_ylabel('# homologous locus candidates')
# orig_xlim = ax.get_xlim()
ax.set_xlim(bins[0], bins[-1])


ylim = ax.get_ylim()
ax.plot([search_for_pis_args['stage5']['min_mauve_total_match_proportion']] * 2, ylim,
        linestyle='--',color='red',
        # label=f'x=0',
        alpha=THRESHOLD_DASHED_LINE_ALPHA)
ax.set_ylim(ylim)


ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))

if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s3.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s3.pdf'))

In [ ]:
if 1:
    plt.close('all')

curr_df = region_in_other_nuccore_df.copy()
curr_df = curr_df[curr_df['mauve_total_match_proportion'] >= search_for_pis_args['stage5']['min_mauve_total_match_proportion']]

column_name = 'min_sub_alignment_min_match_proportion'

assert not curr_df[column_name].isna().any()
fig, ax = plt.subplots(figsize=(6,4))

bin_size = 0.01
bins = list(np.arange(0, 1 + bin_size, bin_size))
assert bins[-1] == 1

# print(curr_df[column_name].describe())

hist_color = GREY_HIST_COLOR

ax.hist(
    curr_df[column_name],
    bins=bins,
    # density=True,
    # histtype='step', 
    # linewidth=8,
    color=hist_color,
    edgecolor=hist_color,
)
ax.set_yscale('log')
ax.set_xlabel('lowest sub-alignment match proportion')
ax.set_ylabel('# homologous locus candidates')
# orig_xlim = ax.get_xlim()
ax.set_xlim(bins[0], bins[-1])


ylim = ax.get_ylim()
ax.plot([search_for_pis_args['stage5']['min_min_sub_alignment_min_match_proportion']] * 2, ylim,
        linestyle='--',color='red',
        # label=f'x=0',
        alpha=THRESHOLD_DASHED_LINE_ALPHA)
ax.set_ylim(ylim)

ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))

if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s4.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s4.pdf'))

In [ ]:
curr_df = minimal_ir_pairs_linked_to_breakpoint_pairs_df.copy()

curr_df = curr_df.sort_values(by='max_dist_between_potential_breakpoint_containing_interval_and_repeat', ascending=True).drop_duplicates(
    [
        'nuccore_accession',
        'index_in_nuccore_ir_pairs_df_csv_file',
    ],
    keep='first'
).rename(columns={
    'max_dist_between_potential_breakpoint_containing_interval_and_repeat': 'min_max_dist_between_potential_breakpoint_containing_interval_and_repeat'})
column_name = 'min_max_dist_between_potential_breakpoint_containing_interval_and_repeat'
# print(curr_df[column_name].describe())

curr_df.loc[curr_df['min_max_dist_between_potential_breakpoint_containing_interval_and_repeat'] < 0, 
            'min_max_dist_between_potential_breakpoint_containing_interval_and_repeat'] = 0

assert not curr_df[column_name].isna().any()
orig_sample_size = len(curr_df)

fig, ax = plt.subplots(figsize=(7,4))

bin_size = 2
# bin_size = 5
real_bins = list(range(
    0, 
    # 401, 
    201, 
    # 141, 
    bin_size,
))
first_real_bin_start = real_bins[0]
last_real_bin_end = real_bins[-1]
fake_last_bin_end = last_real_bin_end + bin_size
fake_last_bin_middle = fake_last_bin_end - bin_size / 2


curr_df.loc[curr_df[column_name] > last_real_bin_end, column_name] = fake_last_bin_middle

bins = real_bins + [fake_last_bin_end]
assert curr_df[column_name].min() >= bins[0]
assert curr_df[column_name].max() <= bins[-1]
assert ((curr_df[column_name] <= bins[-1]) & (curr_df[column_name] >= bins[0])).sum() == orig_sample_size


# print(curr_df[column_name].describe())

# hist_color = (0.7,0.7,0.7)
hist_color = GREY_HIST_COLOR

ax.hist(
    curr_df[column_name],
    bins=bins,
    # density=True,
    # histtype='step', 
    # linewidth=8,
    color=hist_color,
    edgecolor=hist_color,
)
ax.set_yscale('log')
# ax.set_xlabel('longer distance between repeat and closest breakpoint-containing region')
# ax.set_xlabel('longer distance between repeat and closest breakpoint')
# ax.set_xlabel('distance between further repeat and closest breakpoint')
ax.set_xlabel('higher distance between repeat and closest breakpoint')
ax.set_ylabel('# inverted repeat pairs')
# orig_xlim = ax.get_xlim()
ax.set_xlim(bins[0], bins[-1])

add_fake_bin_tick_labels_and_dashed_lines_to_histogram(
    ax, df=curr_df, column_name=column_name, bin_size=bin_size, 
    last_real_bin_end=last_real_bin_end, 
    first_real_bin_start=first_real_bin_start,
)


ylim = ax.get_ylim()
ax.plot([search_for_pis_args['stage5']['max_max_dist_between_potential_breakpoint_containing_interval_and_repeat']] * 2, ylim,
        linestyle='--',color='red',
        # label=f'x=0',
        alpha=THRESHOLD_DASHED_LINE_ALPHA)
ax.set_ylim(ylim)


if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s5.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s5.pdf'))

In [ ]:
# plot_roc_curve_for_each_sim = True
plot_roc_curve_for_each_sim = False

plot_mean_roc_curve = True
# plot_mean_roc_curve = False

curr_df = unified_roc_curve_df.copy()

curr_df['tpr'] = 1 - curr_df['fnr']


all_fprs_sorted = sorted(set(curr_df['fpr']))

sim_all_fprs_and_interp_tprs_dfs = []
for sim_i, sim_roc_curve_df in curr_df.groupby('sim_i'):
    interp_func = scipy.interpolate.interp1d(sim_roc_curve_df['fpr'], sim_roc_curve_df['tpr'], kind='linear')
    sim_all_fprs_and_interp_tprs_dfs.append(pd.DataFrame({'fpr': all_fprs_sorted, 'interp_tpr': interp_func(all_fprs_sorted), 'sim_i': sim_i}))

all_fprs_and_interp_tprs_df = pd.concat(sim_all_fprs_and_interp_tprs_dfs, ignore_index=True)

# fpr_and_tpr_stat_df = all_fprs_and_interp_tprs_df.groupby('fpr', as_index=False, sort=False).agg({'interp_tpr': np.mean, 'interp_tpr': np.std})

interp_tpr_grouped_by_fpr = all_fprs_and_interp_tprs_df.groupby('fpr')['interp_tpr']
fpr_and_tpr_stat_df = interp_tpr_grouped_by_fpr.mean().reset_index(name='interp_tpr_mean').merge(
    interp_tpr_grouped_by_fpr.std().reset_index(name='interp_tpr_std')).merge(
    interp_tpr_grouped_by_fpr.size().reset_index(name='num_of_interp_tprs'))

num_of_sims = curr_df['sim_i'].nunique()
print(f'num_of_sims: {num_of_sims}')
assert next(iter(set(fpr_and_tpr_stat_df['num_of_interp_tprs']))) == num_of_sims
    
chosen_threshold = search_for_pis_args['enrichment_analysis']['min_predicted_rearrangement_probability']

if 1:
    plt.close('all')

    
fig, ax = plt.subplots(figsize=(6,4))

aucs = []
for sim_i, sim_roc_curve_df in curr_df.groupby('sim_i'):
    sim_curr_df = sim_roc_curve_df.sort_values(by='fpr')
    xs = sim_curr_df['fpr']
    ys = sim_curr_df['tpr']
    if plot_roc_curve_for_each_sim:
        ax.plot(xs, ys)
    auc = scipy.integrate.trapz(ys, xs)
    aucs.append(auc)
    # print(f'auc: {auc}')

if plot_mean_roc_curve:
    ax.plot(
        fpr_and_tpr_stat_df['fpr'], 
        fpr_and_tpr_stat_df['interp_tpr_mean'],
    )
    ax.fill_between(
        fpr_and_tpr_stat_df['fpr'], 
        fpr_and_tpr_stat_df['interp_tpr_mean'] - fpr_and_tpr_stat_df['interp_tpr_std'],
        fpr_and_tpr_stat_df['interp_tpr_mean'] + fpr_and_tpr_stat_df['interp_tpr_std'],
        alpha=0.3,
    )
    
    
chosen_threshold_sim_results_df = curr_df[curr_df['new_threshold'] == chosen_threshold]
chosen_threshold_fpr_mean = chosen_threshold_sim_results_df['fpr'].mean()
chosen_threshold_tpr_mean = chosen_threshold_sim_results_df['tpr'].mean()
chosen_threshold_fpr_std = chosen_threshold_sim_results_df['fpr'].std()
chosen_threshold_tpr_std = chosen_threshold_sim_results_df['tpr'].std()
ax.errorbar(
    x=chosen_threshold_fpr_mean, 
    y=chosen_threshold_tpr_mean, 
    xerr=chosen_threshold_fpr_std, 
    yerr=chosen_threshold_tpr_std, 
    zorder=100, color='red',
    capsize=2.5,
)
print(f'chosen_threshold_fpr_mean: {chosen_threshold_fpr_mean}')
print(f'chosen_threshold_tpr_mean: {chosen_threshold_tpr_mean}')

ax.plot([0,1], [0,1], color='grey', linestyle='--', alpha=0.4)
ax.set_xlabel('false positive rate')
ax.set_ylabel('true positive rate')
ax.set_xlim(-0.005, 1.005)
ax.set_ylim(-0.005, 1.005)
ax.set_aspect('equal')

ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))

auc_mean = np.mean(aucs)
auc_std = np.std(aucs)
print(f'auc_mean: {auc_mean}')
print(f'auc_std: {auc_std}')

if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s8.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s8.pdf'))

In [ ]:
if 1:
    plt.close('all')

proportions_as_series = pd.Series(all_all_repeat_cds_covered_bases_proportions)

assert not proportions_as_series.isna().any()
fig, ax = plt.subplots(figsize=(6,4))

bin_size = 0.02
bins = list(np.arange(0, 1 + bin_size, bin_size))
assert bins[-1] == 1

# print(proportions_as_series.describe())

hist_color = GREY_HIST_COLOR

ax.hist(
    proportions_as_series,
    bins=bins,
    # density=True,
    # histtype='step', 
    # linewidth=8,
    color=hist_color,
    edgecolor=hist_color,
)
ax.set_yscale('log')
# ax.set_xlabel('proportion of target coding sequence covered by alignment')
ax.set_xlabel('fraction of target coding sequence covered by alignment')
ax.set_ylabel('# homologous sequences')
# orig_xlim = ax.get_xlim()
ax.set_xlim(bins[0], bins[-1])


ylim = ax.get_ylim()
ax.plot([search_for_pis_args['stage6']['blast_longer_linked_repeat_cds_to_each_taxon_genome']['min_repeat_cds_covered_bases_proportion']] * 2, ylim,
        linestyle='--',color='red',
        # label=f'x=0',
        alpha=THRESHOLD_DASHED_LINE_ALPHA)
ax.set_ylim(ylim)

# fig.set_size_inches(12, 6, forward=True)
ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))

if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s30.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s30.pdf'))

In [ ]:
if 1:
    plt.close('all')

proportions_as_series = pd.Series(all_all_alignment_bases_covered_by_cds_proportions)

assert not proportions_as_series.isna().any()
fig, ax = plt.subplots(figsize=(6,4))

bin_size = 0.02
bins = list(np.arange(0, 1 + bin_size, bin_size))
assert bins[-1] == 1

# print(proportions_as_series.describe())

hist_color = GREY_HIST_COLOR

ax.hist(
    proportions_as_series,
    bins=bins,
    # density=True,
    # histtype='step', 
    # linewidth=8,
    color=hist_color,
    edgecolor=hist_color,
)
ax.set_yscale('log')
# ax.set_xlabel('proportion of homologous sequence covered by coding sequence')
ax.set_xlabel('fraction of homologous sequence covered by coding sequence')
ax.set_ylabel('# homologous sequences')
# orig_xlim = ax.get_xlim()
ax.set_xlim(bins[0], bins[-1])


ylim = ax.get_ylim()
ax.plot([search_for_pis_args['stage6']['blast_longer_linked_repeat_cds_to_each_taxon_genome']['min_alignment_bases_covered_by_cds_proportion']] * 2, ylim,
        linestyle='--',color='red',
        # label=f'x=0',
        alpha=THRESHOLD_DASHED_LINE_ALPHA)
ax.set_ylim(ylim)

# fig.set_size_inches(12, 6, forward=True)
ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))

if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s31.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s31.pdf'))

In [ ]:
if 1:
    plt.close('all')

fig = plt.figure(
    figsize=(6,9.8), 
)
fig_gridspec = fig.add_gridspec(
    # 3, 1, 
    5, 1, 
    # wspace=0.0001,
    hspace=0.0001,
    # width_ratios=[3,10,0.43,1.2], 
    height_ratios=[4,3,4,3,4], 
)

fig.subplots_adjust(
    # top=0.96 if (num_of_long_read_rows == 2) else 0.945,
    top=0.91,
    bottom=0.06,
    left=0.07,
    right=0.97,
    # hspace=0.2,
    # wspace=0.2
)

upper_subgridspec = fig_gridspec[0].subgridspec(1, 5, width_ratios=[0.2,1,0.7,1,0.2], wspace=0)
middle_subgridspec = fig_gridspec[2].subgridspec(1, 5, width_ratios=[0.2,1,0.7,1,0.2], wspace=0)
lower_subgridspec = fig_gridspec[4].subgridspec(1, 5, width_ratios=[0.2,1,0.7,1,0.2], wspace=0)

sra_accession_and_inv_region_to_ax_info = {
    ('SRR10883644', '3712356-3713584'): {'ax': fig.add_subplot(upper_subgridspec[1]), 'show_y_label': True, 'panel_label': 'A'},
    ('SRR8867158', '5327747-5328904'):  {'ax': fig.add_subplot(upper_subgridspec[3]), 'show_y_label': True, 'panel_label': 'B'},
    ('SRR6322568', '2158761-2164817'):  {'ax': fig.add_subplot(middle_subgridspec[1]), 'show_y_label': True, 'panel_label': 'C'},
    ('SRR6322568', '2160408-2163224'):  {'ax': fig.add_subplot(middle_subgridspec[3]), 'show_y_label': True, 'panel_label': 'D'},
    ('SRR6322568', '2160750-2162885'):  {'ax': fig.add_subplot(lower_subgridspec[1]), 'show_y_label': True, 'panel_label': 'E'},
}
    
    
curr_df = min_all_all_diff_score_alignments_df.copy()

jitter_sd = 1
for sra_accession, curr_sra_df in curr_df.groupby('sra_accession'):
    curr_sra_df = curr_sra_df.copy()
    
    for in_silico_inverted_region, curr_inv_region_df in curr_sra_df.groupby('in_silico_inverted_region'):
        curr_inv_region_df = curr_inv_region_df.copy()
        
        assert curr_inv_region_df['longer_linked_repeat_cds_nuccore_accession'].nunique() == 1
        longer_linked_repeat_cds_nuccore_accession = curr_inv_region_df['longer_linked_repeat_cds_nuccore_accession'].iloc[0]
        
        num_of_reads = len(curr_inv_region_df)
        # print(sra_accession, in_silico_inverted_region, num_of_reads)
        
        
        min_finite_score = min((set(curr_inv_region_df['pair_total_align_score_ref']) | set(curr_inv_region_df['pair_total_align_score_non_ref'])) - {-np.inf})
        assert not pd.isna(min_finite_score)
        fake_minus_inf_score = min_finite_score - abs(min_finite_score * 0.1)
        fake_minus_inf_score_thresh = min_finite_score - abs(min_finite_score * 0.05)
        curr_inv_region_df['pair_total_align_score_ref'].replace(-np.inf, fake_minus_inf_score, inplace=True)
        curr_inv_region_df['pair_total_align_score_non_ref'].replace(-np.inf, fake_minus_inf_score, inplace=True)
        
        
        predicted_invertible_region_label = f'predicted invertible region:\n{in_silico_inverted_region}'
        
        ax_info = sra_accession_and_inv_region_to_ax_info[(sra_accession, in_silico_inverted_region)]
        ax = ax_info['ax']
        show_y_label = ax_info['show_y_label']
        panel_label = ax_info['panel_label']
        
        big_diff_df = curr_inv_region_df[curr_inv_region_df['big_abs_score_diff']]
        non_big_diff_df = curr_inv_region_df[~curr_inv_region_df['big_abs_score_diff']]
        
        num_of_big_diff_reads = len(big_diff_df)
        num_of_non_big_diff_reads = len(non_big_diff_df)
        
        assert not big_diff_df['pair_total_align_score_ref'].isna().any()
        assert not big_diff_df['pair_total_align_score_non_ref'].isna().any()
        ax.scatter(
            big_diff_df['pair_total_align_score_ref'] + np.random.normal(scale=jitter_sd, size=num_of_big_diff_reads),
            big_diff_df['pair_total_align_score_non_ref'] + np.random.normal(scale=jitter_sd, size=num_of_big_diff_reads),
            label=predicted_invertible_region_label,
            alpha=0.3,
            facecolor='none',
            # edgecolor='tab:blue',
            # edgecolor='tab:green',
            edgecolor='magenta',
            # edgecolor='red',
        )
        
        assert not non_big_diff_df['pair_total_align_score_ref'].isna().any()
        assert not non_big_diff_df['pair_total_align_score_non_ref'].isna().any()
        ax.scatter(
            non_big_diff_df['pair_total_align_score_ref'] + np.random.normal(scale=jitter_sd, size=num_of_non_big_diff_reads),
            non_big_diff_df['pair_total_align_score_non_ref'] + np.random.normal(scale=jitter_sd, size=num_of_non_big_diff_reads),
            label=predicted_invertible_region_label,
            alpha=0.3,
            s=14,
            facecolor='none',
            edgecolor='grey',
        )
        
        min_lim = fake_minus_inf_score * 1.06
        max_lim = abs(fake_minus_inf_score) * 0.06
        
        # print(min_lim, max_lim)
        ax.plot((min_lim, max_lim), (min_lim, max_lim),
            linestyle='--',color='grey',
            alpha=0.3)
        ax.plot((min_lim, max_lim), [fake_minus_inf_score_thresh] * 2,
                linestyle=FAKE_VAL_THRESHOLD_LINE_STYLE,
                color=FAKE_VAL_THRESHOLD_LINE_COLOR,
                alpha=FAKE_VAL_THRESHOLD_LINE_ALPHA)
        ax.plot([fake_minus_inf_score_thresh] * 2, (min_lim, max_lim),
                linestyle=FAKE_VAL_THRESHOLD_LINE_STYLE,
                color=FAKE_VAL_THRESHOLD_LINE_COLOR,
                alpha=FAKE_VAL_THRESHOLD_LINE_ALPHA)

        
        orig_y_tick_positions = ax.get_yticks()
        new_y_tick_positions = [fake_minus_inf_score] + [y for y in orig_y_tick_positions if y > fake_minus_inf_score + 30]
        ax.set_yticks(new_y_tick_positions)
        new_y_tick_labels = ['NA'] + [remove_trailing_zeros(str(x)) for x in new_y_tick_positions[1:]]
        ax.set_yticklabels(
            new_y_tick_labels,
            fontsize='small',
        )
        
        orig_x_tick_positions = ax.get_xticks()
        new_x_tick_positions = [fake_minus_inf_score] + [x for x in orig_x_tick_positions if x > fake_minus_inf_score + 30]
        ax.set_xticks(new_x_tick_positions)
        new_x_tick_labels = ['NA'] + [remove_trailing_zeros(str(x)) for x in new_x_tick_positions[1:]]
        ax.set_xticklabels(
            new_x_tick_labels,
            fontsize='small',
        )
        
        ax.set_xlim((min_lim, max_lim))
        ax.set_ylim((min_lim, max_lim))
        
        ax.set_aspect('equal')
        ax.set_xlabel(
            'score of paired read\nalignment to ref',
            fontsize='small',
        )
        if show_y_label:
            ax.set_ylabel(
                'score of paired read\nalignment to alternative ref',
                fontsize='small',
            )
        ax.set_title(f'ref: {longer_linked_repeat_cds_nuccore_accession}\nSRA: {sra_accession}\n{predicted_invertible_region_label}')

        ax.text(-0.35, 1.39, panel_label, va='center', ha='center', transform=ax.transAxes, fontweight='bold')
if 1:
    fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s32.png'))
    # fig.savefig(os.path.join(paper_figs_and_tables_dir_path, 'sup_fig_s32.pdf'))